<a href="https://colab.research.google.com/github/tatsuhiko-suyama/Something-/blob/main/4_16_v2_%E3%82%B5%E3%83%9D%E3%83%BC%E3%83%88%E3%82%B5%E3%82%A4%E3%82%BA%EF%BC%93%E4%BB%A5%E4%B8%8A%E3%82%92%E6%8E%A2%E3%81%99%E6%97%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""
4_14_full_range_activeset_mu003.py

Analyzes the change of w*(alpha), H*(alpha), pi*(alpha), grad H*(alpha),
lambda*(alpha), and active_set*(alpha) over the full alpha range [0.0, 1.5],
using the original active set method for the inner QP solve, with mu_tilde = 0.03.
Includes final solve step in FW for consistency and relaxed norm check in gradient.
"""

import numpy as np
import matplotlib.pyplot as plt
import warnings
from scipy.linalg import solve, LinAlgError, eigh, inv, qr
from scipy.optimize import linprog, OptimizeWarning
import traceback
import time
import itertools
import os
import math
from collections import Counter # オプション

# pandas がなければインストールを促す
try:
    import pandas as pd
    PANDAS_AVAILABLE = True
    pd.set_option('display.width', 160)
    pd.set_option('display.float_format', '{:.6e}'.format) # 科学技術表記
    pd.set_option('display.max_rows', 200) # 表示行数を増やす
except ImportError:
    PANDAS_AVAILABLE = False
    print("Warning: pandas library not found. Output formatting will be basic. CSV export disabled.")

# === ここから必要な関数定義 ===
DEFAULT_TOLERANCE = 1e-9

# --- OptimizationResult クラス (active_set_opt を追加) ---
class OptimizationResult:
    def __init__(self, success, message, w_opt=None, pi_opt=None, lambda_opt=None,
                 H_opt=None, grad_H_opt=None, iterations=None, fw_gap=None,
                 active_set_opt=None): # <-- active_set_opt 追加
        self.success = success; self.message = message; self.w_opt = w_opt; self.pi_opt = pi_opt
        self.lambda_opt = lambda_opt; self.H_opt = H_opt; self.grad_H_opt = grad_H_opt
        self.iterations = iterations; self.fw_gap = fw_gap
        self.active_set_opt = active_set_opt # <-- active_set_opt を格納

# --- find_feasible_initial_pi 関数 ---
def find_feasible_initial_pi(R, mu_tilde, K, tolerance=1e-8):
    M = R.shape[1]; c = np.zeros(K + 1); c[K] = 1.0
    A_ub = np.hstack((-R.T, -np.ones((M, 1)))); b_ub = -mu_tilde * np.ones(M)
    bounds = [(None, None)] * K + [(0, None)]; opts = {'tol': tolerance, 'disp': False, 'presolve': True}
    result = None; methods_to_try = ['highs', 'highs-ipm', 'highs-ds', 'simplex']
    for method in methods_to_try:
        try:
            with warnings.catch_warnings(): warnings.filterwarnings("ignore"); result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method=method, options=opts)
            if result.success: break
        except ValueError: continue
        except Exception as e: return None, False, f"Phase 1 LP failed: {e}"
    if result is None or not result.success: msg = result.message if result else "No solver"; status = result.status if result else -1; return None, False, f"Phase 1 LP solver failed: {msg} (status={status})"
    s = result.x[K]; pi = result.x[:K]
    if np.isnan(pi).any(): return None, False, "Phase 1 LP resulted in NaN values for pi."
    if s <= tolerance * 1000:
        G = -R.T; h = -mu_tilde * np.ones(M); violation = np.max(G @ pi - h)
        if violation <= tolerance * 10000: return pi, True, f"Phase 1 OK (s*={s:.1e}, vio={violation:.1e})"
        else: return pi, True, f"Phase 1 OK (s*={s:.1e}), WARN: violation {violation:.1e}"
    else: return None, False, f"Phase 1 Infeasible (s* = {s:.1e})"

# --- solve_kkt_system 関数 ---
def solve_kkt_system(Q, G_W, g, tolerance=DEFAULT_TOLERANCE):
    K = Q.shape[0]; n_act = G_W.shape[0] if G_W is not None and G_W.ndim == 2 and G_W.shape[0] > 0 else 0
    if n_act == 0:
        try: p = solve(Q, -g, assume_a='sym'); l = np.array([])
        except LinAlgError: return None, None, False
        res_norm = np.linalg.norm(Q @ p + g); g_norm = np.linalg.norm(g); solved_ok = res_norm <= tolerance * 1e3 * (1 + g_norm)
        return p, l, solved_ok
    else:
        kkt_mat = None; rhs = None
        try: kkt_mat = np.block([[Q, G_W.T], [G_W, np.zeros((n_act, n_act))]]) ; rhs = np.concatenate([-g, np.zeros(n_act)])
        except ValueError as e: return None, None, False
        try: sol = solve(kkt_mat, rhs, assume_a='sym'); p = sol[:K]; l = sol[K:]
        except LinAlgError: return None, None, False
        except ValueError as e: return None, None, False
        res_norm = np.linalg.norm(kkt_mat @ sol - rhs); rhs_norm = np.linalg.norm(rhs); solved_ok = res_norm <= tolerance * 1e3 * (1 + rhs_norm)
        return p, l, solved_ok

# --- solve_inner_qp_active_set 関数 ---
def solve_inner_qp_active_set(Vw, R, mu_tilde, initial_pi, max_iter=350, tolerance=DEFAULT_TOLERANCE, regularization_epsilon=1e-10):
    K = Vw.shape[0]; M = R.shape[1]; Q_reg = 2 * Vw + 2 * regularization_epsilon * np.eye(K); G = -R.T; h = -mu_tilde * np.ones(M)
    if initial_pi is None: return None, None, None, None, None, False, "No initial pi"
    pi_k = np.copy(initial_pi); lam_opt = np.zeros(M); W = set()
    active_tol = tolerance * 10
    initial_violations = G @ pi_k - h; W = set(j for j, viol in enumerate(initial_violations) if viol > -active_tol)
    active_indices_opt = None; kkt_matrix_opt = None
    if np.any(initial_violations > active_tol * 10): warnings.warn(f"Initial pi infeasible (max viol: {np.max(initial_violations):.2e}).")

    for i in range(max_iter):
        g_k = Q_reg @ pi_k; act = sorted(list(W)); n_act = len(act); G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))
        p_k, lam_Wk, solved = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)
        if not solved or p_k is None: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: KKT solve failed. ActiveSet={act}"
        if np.linalg.norm(p_k) <= tolerance * 10 * (1 + np.linalg.norm(pi_k)):
            is_optimal_point = True; blocking_constraint_idx = -1; min_negative_lambda = float('inf')
            dual_feas_tol = -tolerance * 10
            if W:
                if lam_Wk is None or len(lam_Wk) != n_act: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: lam_Wk inconsistent? ActiveSet={act}"
                lambda_map = dict(zip(act, lam_Wk))
                for constraint_idx, lagrange_multiplier in lambda_map.items():
                    if lagrange_multiplier < dual_feas_tol: is_optimal_point = False;
                    if lagrange_multiplier < min_negative_lambda: min_negative_lambda = lagrange_multiplier; blocking_constraint_idx = constraint_idx
            if is_optimal_point:
                lam_opt.fill(0.0);
                if W and len(lam_Wk) == n_act: lam_opt[act] = np.maximum(lam_Wk, 0)
                final_infeas = np.max(G @ pi_k - h); msg = f"Optimal found at iter {i+1}."
                if final_infeas > active_tol: msg += f" (WARN: violation {final_infeas:.1e})"
                active_indices_opt = act
                # Vw_reg を返すように変更
                return pi_k, lam_opt, active_indices_opt, None, Q_reg / 2.0, True, msg
            else:
                if blocking_constraint_idx in W: W.remove(blocking_constraint_idx); continue
                else: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: Neg lambda idx {blocking_constraint_idx}, not in W={act}"
        else:
            alpha_k = 1.0; blocking_constraint_idx = -1; min_step_length = float('inf')
            step_tol = tolerance * 10
            for j in range(M):
                if j not in W:
                    constraint_gradient_dot_p = G[j, :] @ p_k
                    if constraint_gradient_dot_p > step_tol:
                        distance_to_boundary = h[j] - (G[j, :] @ pi_k)
                        if abs(constraint_gradient_dot_p) > 1e-15:
                            alpha_j = distance_to_boundary / constraint_gradient_dot_p
                            step_j = max(0.0, alpha_j)
                            if step_j < min_step_length:
                                min_step_length = step_j; blocking_constraint_idx = j
            alpha_k = min(1.0, min_step_length); pi_k += alpha_k * p_k
            if alpha_k < 1.0 - step_tol and blocking_constraint_idx != -1:
                if blocking_constraint_idx not in W: W.add(blocking_constraint_idx)
            continue
    msg = f"Max iter ({max_iter}) reached."; final_infeas = np.max(G @ pi_k - h)
    if final_infeas > active_tol * 10: return None, None, None, None, None, False, f"{msg} Final infeasible. ActiveSet={sorted(list(W))}"
    act = sorted(list(W)); n_act = len(act); G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))
    is_likely_optimal = False; active_constraints_opt = act
    g_k = Q_reg @ pi_k; p_f, lam_f, solved_f = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)
    final_lambda_estimate = np.zeros(M)
    if solved_f and p_f is not None and np.linalg.norm(p_f) <= tolerance * 100 * (1 + np.linalg.norm(pi_k)):
        if n_act > 0 and lam_f is not None and len(lam_f) == n_act:
             try: final_lambda_estimate[act] = lam_f
             except IndexError: pass
        active_lambdas = final_lambda_estimate[act] if n_act > 0 else np.array([])
        if n_act == 0 or np.all(active_lambdas >= -tolerance * 100): is_likely_optimal = True; msg += " Final KKT check approx OK."
        else: msg += " Final KKT check fails (dual infeasible)."
    else: msg += " Final KKT check fails (stationarity or solve error)."
    lam_opt = final_lambda_estimate
    # Vw_reg を返すように変更
    return pi_k, lam_opt, active_constraints_opt, None, Q_reg / 2.0, is_likely_optimal, msg

# --- make_psd 関数 ---
def make_psd(matrix, tolerance=1e-8):
    sym = (matrix + matrix.T) / 2
    try: eigenvalues, eigenvectors = np.linalg.eigh(sym); min_eigenvalue = np.min(eigenvalues)
    except LinAlgError: warnings.warn("..."); return sym
    if min_eigenvalue < tolerance: eigenvalues[eigenvalues < tolerance] = tolerance; psd_matrix = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T; return (psd_matrix + psd_matrix.T) / 2
    else: return sym

# --- calculate_Vw 関数 ---
def calculate_Vw(w, R, SecondMoments_a_array, tolerance=DEFAULT_TOLERANCE, psd_tolerance=1e-8):
    K, M = R.shape; w_sum = np.sum(w); w_norm = w
    EwX = R @ w_norm; EwXXT = np.zeros((K, K));
    for m in range(M): EwXXT += w_norm[m] * SecondMoments_a_array[m]
    Vw = EwXXT - np.outer(EwX, EwX); Vw_psd = make_psd(Vw, psd_tolerance)
    return Vw_psd, EwX, EwXXT

# --- calculate_H_gradient 関数 (ノルムチェック緩和) ---
def calculate_H_gradient(pi_star, w, R, SecondMoments_a_array, EwX, EwXXT, tolerance=1e-9, debug_print=False):
    M = w.shape[0]; K = R.shape[0]; grad = np.zeros(M)
    norm_tolerance = 1e-12 # Allow very small pi_star

    if pi_star is None or EwX is None:
        if debug_print: print("DEBUG grad_H: Returning NaN due to None input (pi_star or EwX).")
        return np.full(M, np.nan)
    if np.isnan(pi_star).any() or np.isinf(pi_star).any():
         if debug_print: print("DEBUG grad_H: Returning NaN because pi_star contains NaN/Inf.")
         return np.full(M, np.nan)
    pi_norm = np.linalg.norm(pi_star)
    if pi_norm < norm_tolerance:
         if debug_print: print(f"DEBUG grad_H: Returning NaN because pi_star norm {pi_norm:.2e} < {norm_tolerance:.1e}")
         return np.full(M, np.nan)
    if np.isnan(EwX).any() or np.isinf(EwX).any():
        if debug_print: print("DEBUG grad_H: Returning NaN because EwX contains NaN/Inf.")
        return np.full(M, np.nan)


    try: # 計算中のエラーを捕捉
        pi_T_EwX = pi_star.T @ EwX
        if np.isnan(pi_T_EwX) or np.isinf(pi_T_EwX):
             if debug_print: print(f"DEBUG grad_H: pi_T_EwX is NaN/Inf: {pi_T_EwX}")
             return np.full(M, np.nan)

        for j in range(M):
            Sigma_j = SecondMoments_a_array[j]; r_j = R[:, j]
            pi_T_Sigma_j_pi = pi_star.T @ Sigma_j @ pi_star
            pi_T_r_j = pi_star.T @ r_j

            if np.isnan(pi_T_Sigma_j_pi) or np.isinf(pi_T_Sigma_j_pi):
                 if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_Sigma_j_pi is NaN/Inf: {pi_T_Sigma_j_pi}")
                 grad[j] = np.nan; continue
            if np.isnan(pi_T_r_j) or np.isinf(pi_T_r_j):
                 if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_r_j is NaN/Inf: {pi_T_r_j}")
                 grad[j] = np.nan; continue

            term2 = 2 * pi_T_r_j * pi_T_EwX
            if np.isnan(term2) or np.isinf(term2):
                 if debug_print: print(f"DEBUG grad_H (j={j}): term2 (2*pi_T_r_j*pi_T_EwX) is NaN/Inf: {term2}, pi_T_r_j={pi_T_r_j}, pi_T_EwX={pi_T_EwX}")
                 grad[j] = np.nan; continue

            grad[j] = pi_T_Sigma_j_pi - term2
            if np.isnan(grad[j]) or np.isinf(grad[j]):
                 if debug_print: print(f"DEBUG grad_H (j={j}): final grad[{j}] is NaN/Inf")

    except Exception as e_calc:
         print(f"ERROR in calculate_H_gradient calculation: {e_calc}")
         print(traceback.format_exc())
         return np.full(M, np.nan)

    if np.any(np.isnan(grad)) or np.any(np.isinf(grad)):
        warnings.warn(f"NaN or Inf detected in calculated gradient for w={w}")
        if debug_print: print(f"DEBUG grad_H: Final check found NaN/Inf in gradient: {grad}")
        return np.full(M, np.nan)
    return grad

# --- project_to_simplex 関数 ---
def project_to_simplex(v, z=1):
    n_features = v.shape[0];
    if n_features == 0: return np.array([])
    v_arr = np.asarray(v)
    if np.all(v_arr >= -1e-9) and np.isclose(np.sum(v_arr), z): return np.maximum(v_arr, 0)
    u = np.sort(v_arr)[::-1]; cssv = np.cumsum(u) - z; ind = np.arange(n_features) + 1; cond = u - cssv / ind > 0
    if np.any(cond): rho = ind[cond][-1]; theta = cssv[rho - 1] / float(rho); w = np.maximum(v_arr - theta, 0)
    else:
         w = np.zeros(n_features)
         if z > 0: w[np.argmax(v_arr)] = z
    w_sum = np.sum(w)
    if not np.isclose(w_sum, z):
        if w_sum > 1e-9: w = w * (z / w_sum)
        elif z > 0 :
            w = np.zeros(n_features)
            w[np.argmax(v_arr)] = z
    return np.maximum(w, 0)

# --- frank_wolfe_optimizer 関数 (ActiveSet法を使用、最終解の整合性を高める修正済み) ---
def frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde, initial_w=None, max_outer_iter=250, fw_gap_tol=1e-7, inner_max_iter=350, tolerance=1e-9, psd_make_tolerance=1e-8, qp_regularization=1e-10, debug_print=False, force_iterations=0, return_history=False):
    K, M = R_alpha.shape
    if initial_w is None: w_k = np.ones(M) / M
    else: w_k = project_to_simplex(np.copy(initial_w))
    if w_k is None:
        result = OptimizationResult(False, "Initial projection failed", w_opt=initial_w)
        return (result, []) if return_history else result
    fw_gap = float('inf'); best_w = np.copy(w_k); best_pi = None; best_lam = np.zeros(M); best_H = -float('inf'); final_gHk = np.zeros(M); best_active_set = None
    pi0, ok, p1msg = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance)
    if not ok:
        result = OptimizationResult(False, f"Phase 1 failed: {p1msg}", w_opt=initial_w)
        return (result, []) if return_history else result
    current_w=np.copy(w_k); current_H=-float('inf'); current_pi=None; current_lam=None; current_active_set=None
    inner_solver_args_for_loop = {'max_iter': inner_max_iter, 'tolerance': tolerance, 'regularization_epsilon': qp_regularization}
    history = []

    last_successful_pi = pi0
    last_successful_lam = np.zeros(M)
    last_successful_active_set = tuple()
    last_successful_gHk = np.zeros(M)
    last_successful_fw_gap = float('inf')

    for k in range(max_outer_iter):
        iter_data = {'k': k + 1}
        if return_history: iter_data['w_k'] = np.copy(w_k)
        try: Vk, Ex, ExxT = calculate_Vw(w_k, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
        except Exception as e:
            result = OptimizationResult(False, f"Outer iter {k+1}: Vw failed: {e}", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, active_set_opt=best_active_set)
            if return_history: history.append(iter_data); return (result, history)
            else: return result

        # === Use Active Set for inner solve ===
        pi_init_inner = last_successful_pi if last_successful_pi is not None else pi0
        pk, lk, act_idx_k, _, Vw_reg_k, inner_ok, inner_msg = solve_inner_qp_active_set(
            Vk, R_alpha, mu_tilde, pi_init_inner, **inner_solver_args_for_loop
        )
        # ===================================

        if not inner_ok or pk is None:
            warnings.warn(f"Outer iter {k+1}: Inner QP failed: {inner_msg}. Using last successful state if available.")
            if last_successful_pi is None:
                 result = OptimizationResult(False, f"Outer iter {k+1}: Inner QP failed and no prior success: {inner_msg}",
                                             w_opt=current_w, iterations=k)
                 if return_history: history.append(iter_data); return (result, history)
                 else: return result
            pk = last_successful_pi
            lk = last_successful_lam
            act_idx_k = list(last_successful_active_set)
            final_gHk = last_successful_gHk
            fw_gap = last_successful_fw_gap
            try: Hk = pk.T @ Vk @ pk
            except: Hk = current_H
        else:
            last_successful_pi = pk
            last_successful_lam = lk if lk is not None else np.zeros(M)
            last_successful_active_set = tuple(sorted(act_idx_k)) if act_idx_k is not None else tuple()
            Hk = pk.T @ Vk @ pk
            try:
                gHk = calculate_H_gradient(pk, w_k, R_alpha, SecondMoments_alpha_array, Ex, ExxT, tolerance, debug_print=debug_print)
            except Exception as e:
                result = OptimizationResult(False, f"Outer iter {k+1}: Grad failed: {e}", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
                if return_history: history.append(iter_data); return (result, history)
                else: return result

            final_gHk = gHk if gHk is not None else np.full(M, np.nan) # Return NaN if calculation fails
            last_successful_gHk = final_gHk if not np.isnan(final_gHk).any() else last_successful_gHk # Only update if valid

        if return_history: iter_data['H_k'] = Hk
        if return_history: iter_data['grad_H_k_norm'] = np.linalg.norm(final_gHk) if not np.isnan(final_gHk).any() else np.nan

        current_w = np.copy(w_k); current_H = Hk; current_pi = pk; current_lam = last_successful_lam; current_active_set = last_successful_active_set

        if Hk >= best_H - tolerance*1000:
             best_H = Hk; best_w = np.copy(w_k); best_pi = np.copy(pk); best_lam = np.copy(current_lam); best_active_set = current_active_set

        # --- Check for NaN gradient ---
        if final_gHk is None or np.isnan(final_gHk).any():
            result = OptimizationResult(False, f"Outer iter {k+1}: Grad NaN.", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
            if return_history: history.append(iter_data); return (result, history)
            else: return result
        # --- End Check ---

        grad_norm = np.linalg.norm(final_gHk)
        if grad_norm < tolerance * 10: sk = w_k; sk_idx = -1
        else: sk_idx = np.argmax(final_gHk); sk = np.zeros(M); sk[sk_idx] = 1.0
        if return_history: iter_data['s_k_index'] = sk_idx

        if w_k is None:
             result = OptimizationResult(False, f"k={k} w_k None", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
             if return_history: history.append(iter_data); return (result, history)
             else: return result

        fw_gap = final_gHk.T @ (w_k - sk)
        last_successful_fw_gap = fw_gap
        if return_history: iter_data['fw_gap'] = fw_gap
        gamma = 2.0 / (k + 3.0)
        if return_history: iter_data['gamma_k'] = gamma
        if return_history: history.append(iter_data)

        converged = False
        if k >= force_iterations and not np.isnan(fw_gap):
            if abs(fw_gap) <= fw_gap_tol: converged = True; conv_msg = f"Converged (Gap {abs(fw_gap):.2e})"
        if converged:
            # --- Final Inner Solve for Consistency ---
            final_w = current_w
            final_pi_result = current_pi
            final_lam_result = current_lam
            final_active_set_result = current_active_set
            final_H_result = current_H
            final_grad_result = final_gHk
            final_fw_gap_result = fw_gap

            try:
                final_Vk, final_Ex, final_ExxT = calculate_Vw(final_w, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
                pi0_final, ok_final, _ = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance)
                if ok_final:
                    pi_s_final, lam_s_final, act_idx_final, _, _, inner_ok_final, msg_final = solve_inner_qp_active_set(
                        final_Vk, R_alpha, mu_tilde, pi0_final, # Use stable pi0 for final check
                        **inner_solver_args_for_loop
                    )
                    if inner_ok_final and pi_s_final is not None and lam_s_final is not None:
                        final_pi_result = pi_s_final
                        final_lam_result = lam_s_final
                        final_active_set_result = tuple(sorted(act_idx_final)) if act_idx_final is not None else tuple()
                        final_grad_result = calculate_H_gradient(final_pi_result, final_w, R_alpha, SecondMoments_alpha_array, final_Ex, final_ExxT, tolerance, debug_print=False)
                        if final_grad_result is None or np.isnan(final_grad_result).any(): final_grad_result = np.zeros(M)
                        grad_norm_final = np.linalg.norm(final_grad_result) if not np.isnan(final_grad_result).any() else np.nan
                        if grad_norm_final < tolerance * 10: sk_final = final_w
                        else: sk_idx_final = np.argmax(final_grad_result); sk_final = np.zeros(M); sk_final[sk_idx_final] = 1.0
                        final_fw_gap_result = final_grad_result.T @ (final_w - sk_final) if not np.isnan(final_grad_result).any() else np.nan
                    else:
                        warnings.warn(f"Warning: Final inner QP solve failed after convergence: {msg_final}. Using last iteration's values.")
                else:
                    warnings.warn(f"Warning: Could not find feasible pi0 for final solve after convergence. Using last iteration's values.")

            except Exception as e:
                warnings.warn(f"Error during final inner solve after convergence: {e}. Using last iteration's values.")
            # --- End of Final Inner Solve ---

            result = OptimizationResult(True, conv_msg, w_opt=final_w, pi_opt=final_pi_result, lambda_opt=final_lam_result,
                                        H_opt=final_H_result, grad_H_opt=final_grad_result, iterations=k + 1,
                                        fw_gap=final_fw_gap_result, active_set_opt=final_active_set_result)
            return (result, history) if return_history else result

        # --- Prepare for next iteration ---
        if w_k is None or sk is None:
             result = OptimizationResult(False, f"k={k} w_k/sk None before update", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
             return (result, history) if return_history else result
        w_k_next = (1.0 - gamma) * w_k + gamma * sk; w_k = project_to_simplex(w_k_next)
        if w_k is None:
             result = OptimizationResult(False, f"k={k} proj None", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
             return (result, history) if return_history else result
        pi0 = pk # Use last inner solution as initial guess

    # Max iter reached, perform final solve for consistency
    final_w = current_w
    final_pi_result = current_pi
    final_lam_result = current_lam
    final_active_set_result = current_active_set
    final_H_result = current_H
    final_grad_result = last_successful_gHk
    final_fw_gap_result = last_successful_fw_gap

    try:
        final_Vk, final_Ex, final_ExxT = calculate_Vw(final_w, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
        pi0_final, ok_final, _ = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance) # Use stable pi0
        if ok_final:
            pi_s_final, lam_s_final, act_idx_final, _, _, inner_ok_final, msg_final = solve_inner_qp_active_set(
                final_Vk, R_alpha, mu_tilde, pi0_final, # <-- Use stable pi0_final
                **inner_solver_args_for_loop
            )
            if inner_ok_final and pi_s_final is not None and lam_s_final is not None:
                final_pi_result = pi_s_final
                final_lam_result = lam_s_final
                final_active_set_result = tuple(sorted(act_idx_final)) if act_idx_final is not None else tuple()
                # Recalculate gradient and gap
                final_grad_result = calculate_H_gradient(final_pi_result, final_w, R_alpha, SecondMoments_alpha_array, final_Ex, final_ExxT, tolerance, debug_print=False)
                if final_grad_result is None or np.isnan(final_grad_result).any(): final_grad_result = np.zeros(M)
                grad_norm_final = np.linalg.norm(final_grad_result) if not np.isnan(final_grad_result).any() else np.nan
                if grad_norm_final < tolerance * 10: sk_final = final_w
                else: sk_idx_final = np.argmax(final_grad_result); sk_final = np.zeros(M); sk_final[sk_idx_final] = 1.0
                final_fw_gap_result = final_grad_result.T @ (final_w - sk_final) if not np.isnan(final_grad_result).any() else np.nan
            else:
                warnings.warn(f"Warning: Final inner QP solve failed after max iter: {msg_final}. Using last iteration's values.")
        else:
             warnings.warn(f"Warning: Could not find feasible pi0 for final solve after max iter. Using last iteration's values.")

    except Exception as e:
        warnings.warn(f"Error during final inner solve after max iter: {e}. Using last iteration's values.")

    if return_history:
        grad_norm_final = np.linalg.norm(final_grad_result) if not np.isnan(final_grad_result).any() else np.nan
        if grad_norm_final < tolerance * 10: sk_idx_final = -1
        else: sk_idx_final = np.argmax(final_grad_result) if not np.isnan(final_grad_result).any() else -1
        history.append({
            'k': max_outer_iter + 1, 'w_k': np.copy(final_w), 'H_k': final_H_result,
            'grad_H_k_norm': grad_norm_final, 's_k_index': sk_idx_final, 'fw_gap': final_fw_gap_result,
            'gamma_k': np.nan
        })

    result = OptimizationResult(True, f"Max Iter ({max_outer_iter})", w_opt=final_w, pi_opt=final_pi_result, lambda_opt=final_lam_result,
                                H_opt=final_H_result, grad_H_opt=final_grad_result, iterations=max_outer_iter,
                                fw_gap=final_fw_gap_result, active_set_opt=final_active_set_result)
    return (result, history) if return_history else result


# --- generate_params_profile_switching 関数 ---
def generate_params_profile_switching(alpha, alpha_max, K=5, M=3, R_base=np.array([0.08, 0.07, 0.06, 0.05, 0.04]), sigma_base=np.array([0.18, 0.15, 0.20, 0.12, 0.10]), Corr_base=np.array([[1.0, 0.4, 0.3, 0.2, 0.1], [0.4, 1.0, 0.5, 0.3, 0.2], [0.3, 0.5, 1.0, 0.6, 0.4], [0.2, 0.3, 0.6, 1.0, 0.5], [0.1, 0.2, 0.4, 0.5, 1.0]]), good_prof_r_factor=0.05, good_prof_r_offset=0.002, good_prof_s_factor=-0.02, good_prof_corr_factor=1.0, bad_prof_r_factor=-0.02, bad_prof_r_offset=-0.001, bad_prof_s_factor=0.02, bad_prof_corr_offset=0.0, sigma_min_epsilon=1e-4, psd_tolerance=1e-9):
    assert alpha >= 0 and K == len(R_base) and K == len(sigma_base) and K == Corr_base.shape[0] and M == 3, "Dimension mismatch"
    non_diag_indices = np.where(~np.eye(K, dtype=bool)); R_neutral = R_base; sigma_neutral = np.maximum(sigma_min_epsilon, sigma_base); Corr_neutral = make_psd(Corr_base, psd_tolerance)
    R_good = R_base + (R_base * good_prof_r_factor + good_prof_r_offset); sigma_good = np.maximum(sigma_min_epsilon, sigma_base + (sigma_base * good_prof_s_factor)); Corr_good_target = np.copy(Corr_base); Corr_good_target[non_diag_indices] *= good_prof_corr_factor; Corr_good = make_psd(Corr_good_target, psd_tolerance)
    R_bad = R_base + (R_base * bad_prof_r_factor + bad_prof_r_offset); sigma_bad = np.maximum(sigma_min_epsilon, sigma_base + (sigma_base * bad_prof_s_factor)); Corr_bad_target = np.copy(Corr_base); Corr_bad_target[non_diag_indices] = np.clip(Corr_bad_target[non_diag_indices] + bad_prof_corr_offset, -1.0+psd_tolerance, 1.0-psd_tolerance); Corr_bad = make_psd(Corr_bad_target, psd_tolerance)
    beta = np.clip(alpha / alpha_max if alpha_max > 0 else (1.0 if alpha > 0 else 0.0), 0.0, 1.0)
    R_alpha = np.zeros((K, M)); sigma_alpha = np.zeros((K, M)); Corr_alpha = [np.eye(K) for _ in range(M)]; Cov_alpha = [np.eye(K) for _ in range(M)]; SecondMoments_a_array = np.zeros((M, K, K))
    R_alpha[:, 0] = (1 - beta) * R_good + beta * R_neutral; sigma_alpha[:, 0] = np.maximum(sigma_min_epsilon, (1 - beta) * sigma_good + beta * sigma_neutral); Corr_alpha[0] = make_psd((1 - beta) * Corr_good + beta * Corr_neutral, psd_tolerance)
    R_alpha[:, 1] = (1 - beta) * R_bad + beta * R_good; sigma_alpha[:, 1] = np.maximum(sigma_min_epsilon, (1 - beta) * sigma_bad + beta * sigma_good); Corr_alpha[1] = make_psd((1 - beta) * Corr_bad + beta * Corr_good, psd_tolerance)
    R_alpha[:, 2] = (1 - beta) * R_neutral + beta * R_bad; sigma_alpha[:, 2] = np.maximum(sigma_min_epsilon, (1 - beta) * sigma_neutral + beta * sigma_bad); Corr_alpha[2] = make_psd((1 - beta) * Corr_neutral + beta * Corr_bad, psd_tolerance)
    for m in range(M): sigma_diag_m = np.diag(sigma_alpha[:, m]); Cov_alpha[m] = sigma_diag_m @ Corr_alpha[m] @ sigma_diag_m; SecondMoments_a_array[m, :, :] = Cov_alpha[m] + np.outer(R_alpha[:, m], R_alpha[:, m])
    return R_alpha, SecondMoments_a_array

# === α依存性分析関数 (ActiveSet法を使用) ===
def analyze_alpha_full_results(alpha_range, param_gen_kwargs, optimizer_kwargs, mu_tilde):
    """ 指定された alpha 範囲で FW解、H*、pi*、grad H*、lambda*、active_set* を追跡する """
    results_over_alpha = []
    K = param_gen_kwargs.get('K', 5); M = param_gen_kwargs.get('M', 3)

    print("\n--- Starting Full Results Analysis over Alpha Range (using Active Set Method) ---")
    total_alphas = len(alpha_range)
    start_loop_time = time.time()

    for idx, alpha in enumerate(alpha_range):
        loop_start_time = time.time()
        print(f"\rAnalyzing alpha = {alpha:.6f} ({idx+1}/{total_alphas}) ... ", end="")

        R_alpha, SecondMoments_alpha_array = generate_params_profile_switching(alpha, **param_gen_kwargs)
        alpha_result = {'alpha': alpha}

        w_fw_default = np.full(M, np.nan)
        H_star_fw = np.nan
        pi_opt = np.full(K, np.nan)
        grad_H_opt = np.full(M, np.nan)
        lambda_opt = np.full(M, np.nan)
        active_set_opt = tuple()

        try:
            # --- FW Optimizer呼び出し (Active Setを使用) ---
            fw_result = frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde,
                                              initial_w=None, return_history=False,
                                              debug_print=False, # デバッグOFF
                                              **optimizer_kwargs)
            if fw_result.success:
                 w_fw_default = fw_result.w_opt if fw_result.w_opt is not None else np.full(M, np.nan)
                 H_star_fw = fw_result.H_opt if fw_result.H_opt is not None else np.nan
                 pi_opt = fw_result.pi_opt if fw_result.pi_opt is not None else np.full(K, np.nan)
                 grad_H_opt = fw_result.grad_H_opt if fw_result.grad_H_opt is not None else np.full(M, np.nan)
                 lambda_opt = fw_result.lambda_opt if fw_result.lambda_opt is not None else np.full(M, np.nan)
                 active_set_opt = fw_result.active_set_opt if fw_result.active_set_opt is not None else tuple()
            else:
                 w_fw_default = np.full(M, np.nan)
                 H_star_fw = np.nan
                 pi_opt = np.full(K, np.nan)
                 grad_H_opt = np.full(M, np.nan)
                 lambda_opt = np.full(M, np.nan)
                 active_set_opt = tuple()
                 print(f"\n  Warn: FW failed for alpha={alpha:.6f}: {fw_result.message}")

        except Exception as e:
            print(f"\n  Error during FW run for alpha={alpha:.6f}: {e}")
            w_fw_default = np.full(M, np.nan)
            H_star_fw = np.nan
            pi_opt = np.full(K, np.nan)
            grad_H_opt = np.full(M, np.nan)
            lambda_opt = np.full(M, np.nan)
            active_set_opt = tuple()

        alpha_result['w_fw_default'] = w_fw_default
        alpha_result['H_star'] = H_star_fw
        alpha_result['pi_opt'] = pi_opt
        alpha_result['grad_H_opt'] = grad_H_opt
        alpha_result['lambda_opt'] = lambda_opt
        alpha_result['active_set_opt'] = active_set_opt

        results_over_alpha.append(alpha_result)

    print("\n--- Finished Full Results Analysis ---")
    df_results = pd.DataFrame(results_over_alpha)
    if PANDAS_AVAILABLE:
        fw_vecs = np.stack([res.get('w_fw_default', np.full(M, np.nan)) for res in results_over_alpha])
        pi_vecs = np.stack([res.get('pi_opt', np.full(K, np.nan)) for res in results_over_alpha])
        grad_vecs = np.stack([res.get('grad_H_opt', np.full(M, np.nan)) for res in results_over_alpha])
        lambda_vecs = np.stack([res.get('lambda_opt', np.full(M, np.nan)) for res in results_over_alpha])
        active_sets = [str(res.get('active_set_opt', tuple())) for res in results_over_alpha]

        for m in range(M):
            df_results[f'w_fw_{m}'] = fw_vecs[:, m]
            df_results[f'grad_H_{m}'] = grad_vecs[:, m]
            df_results[f'lambda_{m}'] = lambda_vecs[:, m]
        for k in range(K):
            df_results[f'pi_{k}'] = pi_vecs[:, k]

        df_results['active_set'] = active_sets

        df_results = df_results.drop(columns=['w_fw_default', 'pi_opt', 'grad_H_opt', 'lambda_opt', 'active_set_opt'], errors='ignore')

    return df_results if PANDAS_AVAILABLE else results_over_alpha


# === メイン実行ブロック ===
if __name__ == '__main__':
    start_time_main = time.time()
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    warnings.filterwarnings('ignore', category=OptimizeWarning)

    # --- 設定 ---
    K = 5; M = 3;
    # ★★★ mu_tilde を 0.02 に設定 ★★★
    mu_tilde = 0.02
    alpha_max = 1.5
    profile_adjustments = { 'good_prof_r_factor': 0.05, 'good_prof_r_offset': 0.002,'good_prof_s_factor': -0.02, 'good_prof_corr_factor': 1.0,'bad_prof_r_factor': -0.02, 'bad_prof_r_offset': -0.001,'bad_prof_s_factor': 0.02, 'bad_prof_corr_offset': 0.0, }
    param_gen_kwargs_main = {'K': K, 'M': M, 'alpha_max': alpha_max, **profile_adjustments}
    solver_settings = { # 精度設定
        'max_outer_iter': 500, 'fw_gap_tol': 1e-9,
        'inner_max_iter': 600, # ActiveSet法で使用
        'tolerance': 1e-11, # FW法および内側ソルバーの許容誤差
        'psd_make_tolerance': 1e-9,
        'qp_regularization': 1e-10, # ActiveSet法の正則化
        'force_iterations': 20
    }
    # ★★★ 分析する alpha の範囲 (全範囲 [0.0, 1.5]) ★★★
    alpha_start = 0.0
    alpha_end = 1.5
    alpha_step = 0.01 # 0.01刻み
    num_alpha_steps = int(round((alpha_end - alpha_start) / alpha_step)) + 1
    alpha_range_analyze = np.linspace(alpha_start, alpha_end, num_alpha_steps)
    unique_pt_tolerance = 1e-5 # KKTチェック用 (main スコープで定義)

    # ★★★ CSV保存用の alpha リスト (0.1刻み) ★★★
    alpha_step_for_csv = 0.1
    alphas_for_csv = np.arange(alpha_start, alpha_end + alpha_step_for_csv / 2, alpha_step_for_csv)

    output_csv_filename = f"alpha_full_results_{alpha_start:.1f}_{alpha_end:.1f}_activeset_mu002_sparse.csv" # ファイル名変更

    print("-" * 60)
    print(f"--- Starting Full Range Analysis [{alpha_start:.1f}, {alpha_end:.1f}] (using ActiveSet, mu_tilde={mu_tilde}) ---")
    print(f"--- (CSV will be saved for alpha approx every {alpha_step_for_csv}) ---")
    print("-" * 60)

    # --- 分析実行 ---
    results_df = analyze_alpha_full_results( # 修正された関数を呼び出し
        alpha_range=alpha_range_analyze,
        param_gen_kwargs=param_gen_kwargs_main,
        optimizer_kwargs=solver_settings,
        mu_tilde=mu_tilde # ★★★ 変更した mu_tilde を渡す ★★★
    )

    # --- 結果表示 & CSV保存 & KKTチェック ---
    if PANDAS_AVAILABLE and isinstance(results_df, pd.DataFrame):
        print("\n--- Analysis Results Summary (DataFrame - First 5 & Last 5 rows) ---")
        float_format_func = lambda x: f"{x:.4e}" if pd.notna(x) and isinstance(x, (float, np.number)) else 'NaN'
        pd.options.display.float_format = float_format_func
        cols_to_show = ['alpha', 'H_star', 'active_set'] + \
                       [f'w_fw_{m}' for m in range(M) if f'w_fw_{m}' in results_df.columns] + \
                       [f'pi_{k}' for k in range(K) if f'pi_{k}' in results_df.columns] + \
                       [f'grad_H_{m}' for m in range(M) if f'grad_H_{m}' in results_df.columns] + \
                       [f'lambda_{m}' for m in range(M) if f'lambda_{m}' in results_df.columns]
        cols_to_show = [col for col in cols_to_show if col in results_df.columns]
        with pd.option_context('display.max_rows', 10): # 表示行数を制限
            print(results_df[cols_to_show].to_string(na_rep='NaN'))
        pd.reset_option('display.float_format')

        # CSVファイルに保存 (フィルタリング)
        try:
            csv_save_indices = []
            alpha_csv_tol = alpha_step / 2.1
            processed_targets = set()
            for alpha_target_raw in alphas_for_csv:
                 alpha_target = round(alpha_target_raw, 8)
                 if alpha_target in processed_targets: continue
                 diffs = (results_df['alpha'] - alpha_target).abs()
                 valid_diffs = diffs.dropna()
                 if not valid_diffs.empty and valid_diffs.min() <= alpha_csv_tol:
                      closest_idx = valid_diffs.idxmin()
                      if np.isclose(results_df.loc[closest_idx, 'alpha'], alpha_target, atol=alpha_csv_tol):
                           csv_save_indices.append(closest_idx)
                           processed_targets.add(alpha_target)

            if csv_save_indices:
                 unique_indices = sorted(list(set(csv_save_indices)))
                 df_to_save = results_df.loc[unique_indices]

                 cols_order = ['alpha', 'H_star', 'active_set'] + \
                              [f'w_fw_{m}' for m in range(M) if f'w_fw_{m}' in df_to_save.columns] + \
                              [f'pi_{k}' for k in range(K) if f'pi_{k}' in df_to_save.columns] + \
                              [f'grad_H_{m}' for m in range(M) if f'grad_H_{m}' in df_to_save.columns] + \
                              [f'lambda_{m}' for m in range(M) if f'lambda_{m}' in df_to_save.columns]
                 cols_order = [col for col in cols_order if col in df_to_save.columns]
                 df_to_save = df_to_save[cols_order]

                 df_to_save.to_csv(output_csv_filename, index=False, float_format='%.8e')
                 print(f"\nFiltered results ({len(df_to_save)} points) saved to: {os.path.abspath(output_csv_filename)}")
                 print("Saved alpha values:", np.round(df_to_save['alpha'].values, 4))
            else:
                 print("\nWarning: No data points found matching the sparse alpha values for CSV saving.")

        except Exception as e:
            print(f"\nError saving filtered results to CSV: {e}")

        # KKT条件のチェック (簡易版)
        print("\n--- Quick KKT Check (Stationarity w.r.t. w) ---")
        kkt_violations = []
        # unique_pt_tolerance は main スコープで定義済み
        for index, row in results_df.iterrows():
            alpha = row['alpha']
            w_star = np.array([row.get(f'w_fw_{m}', np.nan) for m in range(M)]) # Use NaN if not found
            grad_h = np.array([row.get(f'grad_H_{m}', np.nan) for m in range(M)])
            active_set_str = row.get('active_set', '()') # Active set from inner solver

            # Check if necessary data is available
            if np.isnan(w_star).any() or np.isnan(grad_h).any() or np.isclose(np.sum(w_star), 0):
                kkt_violations.append({
                    'alpha': alpha,
                    'active_set_report': active_set_str,
                    'active_set_kkt': 'NaN',
                    'max_violation': np.nan,
                    'grad_consistency_violation': np.nan,
                    'check_result': 'Skipped (NaN or zero w)'
                })
                continue

            check_tol = solver_settings['tolerance'] * 1e3

            max_grad_h = np.max(grad_h)
            nu_estimate = max_grad_h # Estimate nu*

            stationarity_violation = 0.0
            active_indices_kkt = set() # Indices where w_m > tol
            for m in range(M):
                if w_star[m] > unique_pt_tolerance:
                    active_indices_kkt.add(m)

            # Check gradient consistency for active indices based on w_star
            active_grads = grad_h[list(active_indices_kkt)] if active_indices_kkt else []
            grad_consistency_violation = 0.0
            if len(active_grads) > 1:
                grad_consistency_violation = np.max(active_grads) - np.min(active_grads)
                if not np.isclose(grad_consistency_violation, 0.0, atol=check_tol):
                     stationarity_violation = max(stationarity_violation, grad_consistency_violation)

            # Check if inactive components have smaller gradients (dual feasibility check)
            for m in range(M):
                if m not in active_indices_kkt: # If w_m is near zero
                    nu_for_check = np.min(active_grads) if active_grads else max_grad_h
                    eta_m_estimate = nu_for_check - grad_h[m]
                    if eta_m_estimate < -check_tol: # Check dual feasibility (eta_m >= 0)
                        stationarity_violation = max(stationarity_violation, abs(eta_m_estimate))

            kkt_violations.append({
                'alpha': alpha,
                'active_set_report': active_set_str, # From inner solver
                'active_set_kkt': str(tuple(sorted(active_indices_kkt))), # From w*
                'max_violation': stationarity_violation,
                'grad_consistency_violation': grad_consistency_violation,
                'check_result': 'OK' if stationarity_violation < check_tol * 10 else 'WARN' # Simple check result
            })

        if PANDAS_AVAILABLE and kkt_violations:
            df_kkt = pd.DataFrame(kkt_violations)
            kkt_cols_order = ['alpha', 'active_set_report','active_set_kkt', 'max_violation', 'grad_consistency_violation', 'check_result']
            kkt_cols_order = [col for col in kkt_cols_order if col in df_kkt.columns]
            with pd.option_context('display.float_format', '{:.4e}'.format):
                print(df_kkt[kkt_cols_order].to_string(index=False, na_rep='NaN'))
            print("Notes on KKT Check:")
            print(" - active_set_report: Active set reported by inner solver")
            print(" - active_set_kkt: Active set inferred from w* > tol")
            print(" - max_violation: Max KKT violation (dual infeasibility or comp. slackness based on eta estimate)")
            print(" - grad_consistency_violation: max(active_grads) - min(active_grads) for w_m > tol")
        else:
            print("Could not perform KKT check or pandas not available.")


        # プロットは今回は行わない

    elif isinstance(results_df, list):
        print("\n--- Analysis Results Summary (List - First 5 & Last 5) ---")
        # (表示省略)
        print("\n(Pandas not available, skipping CSV export)")

    end_time_main = time.time()
    print(f"\nTotal execution time: {end_time_main - start_time_main:.2f} seconds")

------------------------------------------------------------
--- Starting Full Range Analysis [0.0, 1.5] (using ActiveSet, mu_tilde=0.02) ---
--- (CSV will be saved for alpha approx every 0.1) ---
------------------------------------------------------------

--- Starting Full Results Analysis over Alpha Range (using Active Set Method) ---
Analyzing alpha = 1.500000 (151/151) ... 
--- Finished Full Results Analysis ---

--- Analysis Results Summary (DataFrame - First 5 & Last 5 rows) ---
         alpha     H_star active_set     w_fw_0     w_fw_1     w_fw_2       pi_0       pi_1        pi_2       pi_3       pi_4   grad_H_0   grad_H_1   grad_H_2   lambda_0   lambda_1   lambda_2
0   0.0000e+00 1.0393e-03       (1,) 8.7835e-06 9.9998e-01 8.7835e-06 8.1006e-02 1.0352e-01 -4.9906e-02 1.0154e-01 1.2416e-01 5.6578e-04 6.3926e-04 5.9951e-04 0.0000e+00 1.0393e-01 0.0000e+00
1   1.0000e-02 1.0370e-03       (1,) 8.9124e-06 9.9998e-01 8.9124e-06 8.0927e-02 1.0343e-01 -4.9868e-02 1.0145e-01 1.2409e-0

In [2]:
# -*- coding: utf-8 -*-
"""
4_14_find_internal_solution.py

Attempts to find an alpha and parameter setting where the optimal w*
has full support (w*_m > 0 for all m), focusing on alpha around 0.5
with milder profile differences and mu_tilde = 0.02.
Uses the Active Set method for the inner QP solve.
"""

import numpy as np
import matplotlib.pyplot as plt
import warnings
from scipy.linalg import solve, LinAlgError, eigh, inv, qr
from scipy.optimize import linprog, OptimizeWarning
import traceback
import time
import itertools
import os
import math
from collections import Counter # オプション

# pandas がなければインストールを促す
try:
    import pandas as pd
    PANDAS_AVAILABLE = True
    pd.set_option('display.width', 160)
    pd.set_option('display.float_format', '{:.6e}'.format) # 科学技術表記
    pd.set_option('display.max_rows', 200) # 表示行数を増やす
except ImportError:
    PANDAS_AVAILABLE = False
    print("Warning: pandas library not found. Output formatting will be basic. CSV export disabled.")

# === ここから必要な関数定義 ===
DEFAULT_TOLERANCE = 1e-9

# --- OptimizationResult クラス (active_set_opt を追加) ---
class OptimizationResult:
    def __init__(self, success, message, w_opt=None, pi_opt=None, lambda_opt=None,
                 H_opt=None, grad_H_opt=None, iterations=None, fw_gap=None,
                 active_set_opt=None):
        self.success = success; self.message = message; self.w_opt = w_opt; self.pi_opt = pi_opt
        self.lambda_opt = lambda_opt; self.H_opt = H_opt; self.grad_H_opt = grad_H_opt
        self.iterations = iterations; self.fw_gap = fw_gap
        self.active_set_opt = active_set_opt

# --- find_feasible_initial_pi 関数 ---
def find_feasible_initial_pi(R, mu_tilde, K, tolerance=1e-8):
    M = R.shape[1]; c = np.zeros(K + 1); c[K] = 1.0
    A_ub = np.hstack((-R.T, -np.ones((M, 1)))); b_ub = -mu_tilde * np.ones(M)
    bounds = [(None, None)] * K + [(0, None)]; opts = {'tol': tolerance, 'disp': False, 'presolve': True}
    result = None; methods_to_try = ['highs', 'highs-ipm', 'highs-ds', 'simplex']
    for method in methods_to_try:
        try:
            with warnings.catch_warnings(): warnings.filterwarnings("ignore"); result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method=method, options=opts)
            if result.success: break
        except ValueError: continue
        except Exception as e: return None, False, f"Phase 1 LP failed: {e}"
    if result is None or not result.success: msg = result.message if result else "No solver"; status = result.status if result else -1; return None, False, f"Phase 1 LP solver failed: {msg} (status={status})"
    s = result.x[K]; pi = result.x[:K]
    if np.isnan(pi).any(): return None, False, "Phase 1 LP resulted in NaN values for pi."
    if s <= tolerance * 1000:
        G = -R.T; h = -mu_tilde * np.ones(M); violation = np.max(G @ pi - h)
        if violation <= tolerance * 10000: return pi, True, f"Phase 1 OK (s*={s:.1e}, vio={violation:.1e})"
        else: return pi, True, f"Phase 1 OK (s*={s:.1e}), WARN: violation {violation:.1e}"
    else: return None, False, f"Phase 1 Infeasible (s* = {s:.1e})"

# --- solve_kkt_system 関数 ---
def solve_kkt_system(Q, G_W, g, tolerance=DEFAULT_TOLERANCE):
    K = Q.shape[0]; n_act = G_W.shape[0] if G_W is not None and G_W.ndim == 2 and G_W.shape[0] > 0 else 0
    if n_act == 0:
        try: p = solve(Q, -g, assume_a='sym'); l = np.array([])
        except LinAlgError: return None, None, False
        res_norm = np.linalg.norm(Q @ p + g); g_norm = np.linalg.norm(g); solved_ok = res_norm <= tolerance * 1e3 * (1 + g_norm)
        return p, l, solved_ok
    else:
        kkt_mat = None; rhs = None
        try: kkt_mat = np.block([[Q, G_W.T], [G_W, np.zeros((n_act, n_act))]]) ; rhs = np.concatenate([-g, np.zeros(n_act)])
        except ValueError as e: return None, None, False
        try: sol = solve(kkt_mat, rhs, assume_a='sym'); p = sol[:K]; l = sol[K:]
        except LinAlgError: return None, None, False
        except ValueError as e: return None, None, False
        res_norm = np.linalg.norm(kkt_mat @ sol - rhs); rhs_norm = np.linalg.norm(rhs); solved_ok = res_norm <= tolerance * 1e3 * (1 + rhs_norm)
        return p, l, solved_ok

# --- solve_inner_qp_active_set 関数 ---
def solve_inner_qp_active_set(Vw, R, mu_tilde, initial_pi, max_iter=350, tolerance=DEFAULT_TOLERANCE, regularization_epsilon=1e-10):
    K = Vw.shape[0]; M = R.shape[1]; Q_reg = 2 * Vw + 2 * regularization_epsilon * np.eye(K); G = -R.T; h = -mu_tilde * np.ones(M)
    if initial_pi is None: return None, None, None, None, None, False, "No initial pi"
    pi_k = np.copy(initial_pi); lam_opt = np.zeros(M); W = set()
    active_tol = tolerance * 10
    initial_violations = G @ pi_k - h; W = set(j for j, viol in enumerate(initial_violations) if viol > -active_tol)
    active_indices_opt = None; kkt_matrix_opt = None
    if np.any(initial_violations > active_tol * 10): warnings.warn(f"Initial pi infeasible (max viol: {np.max(initial_violations):.2e}).")

    for i in range(max_iter):
        g_k = Q_reg @ pi_k; act = sorted(list(W)); n_act = len(act); G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))
        p_k, lam_Wk, solved = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)
        if not solved or p_k is None: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: KKT solve failed. ActiveSet={act}"
        if np.linalg.norm(p_k) <= tolerance * 10 * (1 + np.linalg.norm(pi_k)):
            is_optimal_point = True; blocking_constraint_idx = -1; min_negative_lambda = float('inf')
            dual_feas_tol = -tolerance * 10
            if W:
                if lam_Wk is None or len(lam_Wk) != n_act: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: lam_Wk inconsistent? ActiveSet={act}"
                lambda_map = dict(zip(act, lam_Wk))
                for constraint_idx, lagrange_multiplier in lambda_map.items():
                    if lagrange_multiplier < dual_feas_tol: is_optimal_point = False;
                    if lagrange_multiplier < min_negative_lambda: min_negative_lambda = lagrange_multiplier; blocking_constraint_idx = constraint_idx
            if is_optimal_point:
                lam_opt.fill(0.0);
                if W and len(lam_Wk) == n_act: lam_opt[act] = np.maximum(lam_Wk, 0)
                final_infeas = np.max(G @ pi_k - h); msg = f"Optimal found at iter {i+1}."
                if final_infeas > active_tol: msg += f" (WARN: violation {final_infeas:.1e})"
                active_indices_opt = act
                return pi_k, lam_opt, active_indices_opt, None, Q_reg / 2.0, True, msg
            else:
                if blocking_constraint_idx in W: W.remove(blocking_constraint_idx); continue
                else: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: Neg lambda idx {blocking_constraint_idx}, not in W={act}"
        else:
            alpha_k = 1.0; blocking_constraint_idx = -1; min_step_length = float('inf')
            step_tol = tolerance * 10
            for j in range(M):
                if j not in W:
                    constraint_gradient_dot_p = G[j, :] @ p_k
                    if constraint_gradient_dot_p > step_tol:
                        distance_to_boundary = h[j] - (G[j, :] @ pi_k)
                        if abs(constraint_gradient_dot_p) > 1e-15:
                            alpha_j = distance_to_boundary / constraint_gradient_dot_p
                            step_j = max(0.0, alpha_j)
                            if step_j < min_step_length:
                                min_step_length = step_j; blocking_constraint_idx = j
            alpha_k = min(1.0, min_step_length); pi_k += alpha_k * p_k
            if alpha_k < 1.0 - step_tol and blocking_constraint_idx != -1:
                if blocking_constraint_idx not in W: W.add(blocking_constraint_idx)
            continue
    msg = f"Max iter ({max_iter}) reached."; final_infeas = np.max(G @ pi_k - h)
    if final_infeas > active_tol * 10: return None, None, None, None, None, False, f"{msg} Final infeasible. ActiveSet={sorted(list(W))}"
    act = sorted(list(W)); n_act = len(act); G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))
    is_likely_optimal = False; active_constraints_opt = act
    g_k = Q_reg @ pi_k; p_f, lam_f, solved_f = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)
    final_lambda_estimate = np.zeros(M)
    if solved_f and p_f is not None and np.linalg.norm(p_f) <= tolerance * 100 * (1 + np.linalg.norm(pi_k)):
        if n_act > 0 and lam_f is not None and len(lam_f) == n_act:
             try: final_lambda_estimate[act] = lam_f
             except IndexError: pass
        active_lambdas = final_lambda_estimate[act] if n_act > 0 else np.array([])
        if n_act == 0 or np.all(active_lambdas >= -tolerance * 100): is_likely_optimal = True; msg += " Final KKT check approx OK."
        else: msg += " Final KKT check fails (dual infeasible)."
    else: msg += " Final KKT check fails (stationarity or solve error)."
    lam_opt = final_lambda_estimate
    return pi_k, lam_opt, active_constraints_opt, None, Q_reg / 2.0, is_likely_optimal, msg

# --- make_psd 関数 ---
def make_psd(matrix, tolerance=1e-8):
    sym = (matrix + matrix.T) / 2
    try: eigenvalues, eigenvectors = np.linalg.eigh(sym); min_eigenvalue = np.min(eigenvalues)
    except LinAlgError: warnings.warn("..."); return sym
    if min_eigenvalue < tolerance: eigenvalues[eigenvalues < tolerance] = tolerance; psd_matrix = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T; return (psd_matrix + psd_matrix.T) / 2
    else: return sym

# --- calculate_Vw 関数 ---
def calculate_Vw(w, R, SecondMoments_a_array, tolerance=DEFAULT_TOLERANCE, psd_tolerance=1e-8):
    K, M = R.shape; w_sum = np.sum(w); w_norm = w
    EwX = R @ w_norm; EwXXT = np.zeros((K, K));
    for m in range(M): EwXXT += w_norm[m] * SecondMoments_a_array[m]
    Vw = EwXXT - np.outer(EwX, EwX); Vw_psd = make_psd(Vw, psd_tolerance)
    return Vw_psd, EwX, EwXXT

# --- calculate_H_gradient 関数 (ノルムチェック緩和) ---
def calculate_H_gradient(pi_star, w, R, SecondMoments_a_array, EwX, EwXXT, tolerance=1e-9, debug_print=False):
    M = w.shape[0]; K = R.shape[0]; grad = np.zeros(M)
    norm_tolerance = 1e-12 # Allow very small pi_star

    if pi_star is None or EwX is None:
        if debug_print: print("DEBUG grad_H: Returning NaN due to None input (pi_star or EwX).")
        return np.full(M, np.nan)
    if np.isnan(pi_star).any() or np.isinf(pi_star).any():
         if debug_print: print("DEBUG grad_H: Returning NaN because pi_star contains NaN/Inf.")
         return np.full(M, np.nan)
    pi_norm = np.linalg.norm(pi_star)
    if pi_norm < norm_tolerance:
         if debug_print: print(f"DEBUG grad_H: Returning NaN because pi_star norm {pi_norm:.2e} < {norm_tolerance:.1e}")
         return np.full(M, np.nan)
    if np.isnan(EwX).any() or np.isinf(EwX).any():
        if debug_print: print("DEBUG grad_H: Returning NaN because EwX contains NaN/Inf.")
        return np.full(M, np.nan)

    try:
        pi_T_EwX = pi_star.T @ EwX
        if np.isnan(pi_T_EwX) or np.isinf(pi_T_EwX):
             if debug_print: print(f"DEBUG grad_H: pi_T_EwX is NaN/Inf: {pi_T_EwX}")
             return np.full(M, np.nan)

        for j in range(M):
            Sigma_j = SecondMoments_a_array[j]; r_j = R[:, j]
            pi_T_Sigma_j_pi = pi_star.T @ Sigma_j @ pi_star
            pi_T_r_j = pi_star.T @ r_j

            if np.isnan(pi_T_Sigma_j_pi) or np.isinf(pi_T_Sigma_j_pi):
                 if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_Sigma_j_pi is NaN/Inf: {pi_T_Sigma_j_pi}")
                 grad[j] = np.nan; continue
            if np.isnan(pi_T_r_j) or np.isinf(pi_T_r_j):
                 if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_r_j is NaN/Inf: {pi_T_r_j}")
                 grad[j] = np.nan; continue

            term2 = 2 * pi_T_r_j * pi_T_EwX
            if np.isnan(term2) or np.isinf(term2):
                 if debug_print: print(f"DEBUG grad_H (j={j}): term2 (2*pi_T_r_j*pi_T_EwX) is NaN/Inf: {term2}, pi_T_r_j={pi_T_r_j}, pi_T_EwX={pi_T_EwX}")
                 grad[j] = np.nan; continue

            grad[j] = pi_T_Sigma_j_pi - term2
            if np.isnan(grad[j]) or np.isinf(grad[j]):
                 if debug_print: print(f"DEBUG grad_H (j={j}): final grad[{j}] is NaN/Inf")

    except Exception as e_calc:
         print(f"ERROR in calculate_H_gradient calculation: {e_calc}")
         print(traceback.format_exc())
         return np.full(M, np.nan)

    if np.any(np.isnan(grad)) or np.any(np.isinf(grad)):
        warnings.warn(f"NaN or Inf detected in calculated gradient for w={w}")
        if debug_print: print(f"DEBUG grad_H: Final check found NaN/Inf in gradient: {grad}")
        return np.full(M, np.nan)
    return grad

# --- project_to_simplex 関数 ---
def project_to_simplex(v, z=1):
    n_features = v.shape[0];
    if n_features == 0: return np.array([])
    v_arr = np.asarray(v)
    if np.all(v_arr >= -1e-9) and np.isclose(np.sum(v_arr), z): return np.maximum(v_arr, 0)
    u = np.sort(v_arr)[::-1]; cssv = np.cumsum(u) - z; ind = np.arange(n_features) + 1; cond = u - cssv / ind > 0
    if np.any(cond): rho = ind[cond][-1]; theta = cssv[rho - 1] / float(rho); w = np.maximum(v_arr - theta, 0)
    else:
         w = np.zeros(n_features)
         if z > 0: w[np.argmax(v_arr)] = z
    w_sum = np.sum(w)
    if not np.isclose(w_sum, z):
        if w_sum > 1e-9: w = w * (z / w_sum)
        elif z > 0 :
            w = np.zeros(n_features)
            w[np.argmax(v_arr)] = z
    return np.maximum(w, 0)

# --- frank_wolfe_optimizer 関数 (ActiveSet法を使用、最終解の整合性を高める修正済み) ---
def frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde, initial_w=None, max_outer_iter=250, fw_gap_tol=1e-7, inner_max_iter=350, tolerance=1e-9, psd_make_tolerance=1e-8, qp_regularization=1e-10, debug_print=False, force_iterations=0, return_history=False):
    K, M = R_alpha.shape
    if initial_w is None: w_k = np.ones(M) / M
    else: w_k = project_to_simplex(np.copy(initial_w))
    if w_k is None:
        result = OptimizationResult(False, "Initial projection failed", w_opt=initial_w)
        return (result, []) if return_history else result
    fw_gap = float('inf'); best_w = np.copy(w_k); best_pi = None; best_lam = np.zeros(M); best_H = -float('inf'); final_gHk = np.zeros(M); best_active_set = None
    pi0, ok, p1msg = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance)
    if not ok:
        result = OptimizationResult(False, f"Phase 1 failed: {p1msg}", w_opt=initial_w)
        return (result, []) if return_history else result
    current_w=np.copy(w_k); current_H=-float('inf'); current_pi=None; current_lam=None; current_active_set=None
    inner_solver_args_for_loop = {'max_iter': inner_max_iter, 'tolerance': tolerance, 'regularization_epsilon': qp_regularization}
    history = []

    last_successful_pi = pi0
    last_successful_lam = np.zeros(M)
    last_successful_active_set = tuple()
    last_successful_gHk = np.zeros(M)
    last_successful_fw_gap = float('inf')

    for k in range(max_outer_iter):
        iter_data = {'k': k + 1}
        if return_history: iter_data['w_k'] = np.copy(w_k)
        try: Vk, Ex, ExxT = calculate_Vw(w_k, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
        except Exception as e:
            result = OptimizationResult(False, f"Outer iter {k+1}: Vw failed: {e}", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, active_set_opt=best_active_set)
            if return_history: history.append(iter_data); return (result, history)
            else: return result

        # === Use Active Set for inner solve ===
        pi_init_inner = last_successful_pi if last_successful_pi is not None else pi0
        pk, lk, act_idx_k, _, Vw_reg_k, inner_ok, inner_msg = solve_inner_qp_active_set(
            Vk, R_alpha, mu_tilde, pi_init_inner, **inner_solver_args_for_loop
        )
        # ===================================

        if not inner_ok or pk is None:
            warnings.warn(f"Outer iter {k+1}: Inner QP failed: {inner_msg}. Using last successful state if available.")
            if last_successful_pi is None:
                 result = OptimizationResult(False, f"Outer iter {k+1}: Inner QP failed and no prior success: {inner_msg}",
                                             w_opt=current_w, iterations=k)
                 if return_history: history.append(iter_data); return (result, history)
                 else: return result
            pk = last_successful_pi
            lk = last_successful_lam
            act_idx_k = list(last_successful_active_set)
            final_gHk = last_successful_gHk
            fw_gap = last_successful_fw_gap
            try: Hk = pk.T @ Vk @ pk
            except: Hk = current_H
        else:
            last_successful_pi = pk
            last_successful_lam = lk if lk is not None else np.zeros(M)
            last_successful_active_set = tuple(sorted(act_idx_k)) if act_idx_k is not None else tuple()
            Hk = pk.T @ Vk @ pk
            try:
                gHk = calculate_H_gradient(pk, w_k, R_alpha, SecondMoments_alpha_array, Ex, ExxT, tolerance, debug_print=debug_print)
            except Exception as e:
                result = OptimizationResult(False, f"Outer iter {k+1}: Grad failed: {e}", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
                if return_history: history.append(iter_data); return (result, history)
                else: return result

            final_gHk = gHk if gHk is not None else np.full(M, np.nan)
            last_successful_gHk = final_gHk if not np.isnan(final_gHk).any() else last_successful_gHk

        if return_history: iter_data['H_k'] = Hk
        if return_history: iter_data['grad_H_k_norm'] = np.linalg.norm(final_gHk) if not np.isnan(final_gHk).any() else np.nan

        current_w = np.copy(w_k); current_H = Hk; current_pi = pk; current_lam = last_successful_lam; current_active_set = last_successful_active_set

        if Hk >= best_H - tolerance*1000:
             best_H = Hk; best_w = np.copy(w_k); best_pi = np.copy(pk); best_lam = np.copy(current_lam); best_active_set = current_active_set

        if final_gHk is None or np.isnan(final_gHk).any():
            result = OptimizationResult(False, f"Outer iter {k+1}: Grad NaN.", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
            if return_history: history.append(iter_data); return (result, history)
            else: return result

        grad_norm = np.linalg.norm(final_gHk)
        if grad_norm < tolerance * 10: sk = w_k; sk_idx = -1
        else: sk_idx = np.argmax(final_gHk); sk = np.zeros(M); sk[sk_idx] = 1.0
        if return_history: iter_data['s_k_index'] = sk_idx

        if w_k is None:
             result = OptimizationResult(False, f"k={k} w_k None", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
             if return_history: history.append(iter_data); return (result, history)
             else: return result

        fw_gap = final_gHk.T @ (w_k - sk)
        last_successful_fw_gap = fw_gap
        if return_history: iter_data['fw_gap'] = fw_gap
        gamma = 2.0 / (k + 3.0)
        if return_history: iter_data['gamma_k'] = gamma
        if return_history: history.append(iter_data)

        converged = False
        if k >= force_iterations and not np.isnan(fw_gap):
            if abs(fw_gap) <= fw_gap_tol: converged = True; conv_msg = f"Converged (Gap {abs(fw_gap):.2e})"
        if converged:
            # --- Final Inner Solve for Consistency ---
            final_w = current_w
            final_pi_result = current_pi
            final_lam_result = current_lam
            final_active_set_result = current_active_set
            final_H_result = current_H
            final_grad_result = final_gHk
            final_fw_gap_result = fw_gap

            try:
                final_Vk, final_Ex, final_ExxT = calculate_Vw(final_w, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
                pi0_final, ok_final, _ = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance)
                if ok_final:
                    pi_s_final, lam_s_final, act_idx_final, _, _, inner_ok_final, msg_final = solve_inner_qp_active_set(
                        final_Vk, R_alpha, mu_tilde, pi0_final, # Use stable pi0 for final check
                        **inner_solver_args_for_loop
                    )
                    if inner_ok_final and pi_s_final is not None and lam_s_final is not None:
                        final_pi_result = pi_s_final
                        final_lam_result = lam_s_final
                        final_active_set_result = tuple(sorted(act_idx_final)) if act_idx_final is not None else tuple()
                        final_grad_result = calculate_H_gradient(final_pi_result, final_w, R_alpha, SecondMoments_alpha_array, final_Ex, final_ExxT, tolerance, debug_print=False)
                        if final_grad_result is None or np.isnan(final_grad_result).any(): final_grad_result = np.zeros(M)
                        grad_norm_final = np.linalg.norm(final_grad_result) if not np.isnan(final_grad_result).any() else np.nan
                        if grad_norm_final < tolerance * 10: sk_final = final_w
                        else: sk_idx_final = np.argmax(final_grad_result); sk_final = np.zeros(M); sk_final[sk_idx_final] = 1.0
                        final_fw_gap_result = final_grad_result.T @ (final_w - sk_final) if not np.isnan(final_grad_result).any() else np.nan
                    else:
                        warnings.warn(f"Warning: Final inner QP solve failed after convergence: {msg_final}. Using last iteration's values.")
                else:
                     warnings.warn(f"Warning: Could not find feasible pi0 for final solve after convergence. Using last iteration's values.")
            except Exception as e:
                warnings.warn(f"Error during final inner solve after convergence: {e}. Using last iteration's values.")
            # --- End of Final Inner Solve ---

            result = OptimizationResult(True, conv_msg, w_opt=final_w, pi_opt=final_pi_result, lambda_opt=final_lam_result,
                                        H_opt=final_H_result, grad_H_opt=final_grad_result, iterations=k + 1,
                                        fw_gap=final_fw_gap_result, active_set_opt=final_active_set_result)
            return (result, history) if return_history else result

        # --- Prepare for next iteration ---
        if w_k is None or sk is None:
             result = OptimizationResult(False, f"k={k} w_k/sk None before update", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
             return (result, history) if return_history else result
        w_k_next = (1.0 - gamma) * w_k + gamma * sk; w_k = project_to_simplex(w_k_next)
        if w_k is None:
             result = OptimizationResult(False, f"k={k} proj None", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
             return (result, history) if return_history else result
        pi0 = pk # Use last inner solution as initial guess

    # Max iter reached, perform final solve for consistency
    final_w = current_w
    final_pi_result = current_pi
    final_lam_result = current_lam
    final_active_set_result = current_active_set
    final_H_result = current_H
    final_grad_result = last_successful_gHk
    final_fw_gap_result = last_successful_fw_gap

    try:
        final_Vk, final_Ex, final_ExxT = calculate_Vw(final_w, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
        pi0_final, ok_final, _ = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance) # Use stable pi0
        if ok_final:
            pi_s_final, lam_s_final, act_idx_final, _, _, inner_ok_final, msg_final = solve_inner_qp_active_set(
                final_Vk, R_alpha, mu_tilde, pi0_final, # <-- Use stable pi0_final
                **inner_solver_args_for_loop
            )
            if inner_ok_final and pi_s_final is not None and lam_s_final is not None:
                final_pi_result = pi_s_final
                final_lam_result = lam_s_final
                final_active_set_result = tuple(sorted(act_idx_final)) if act_idx_final is not None else tuple()
                # Recalculate gradient and gap
                final_grad_result = calculate_H_gradient(final_pi_result, final_w, R_alpha, SecondMoments_alpha_array, final_Ex, final_ExxT, tolerance, debug_print=False)
                if final_grad_result is None or np.isnan(final_grad_result).any(): final_grad_result = np.zeros(M)
                grad_norm_final = np.linalg.norm(final_grad_result) if not np.isnan(final_grad_result).any() else np.nan
                if grad_norm_final < tolerance * 10: sk_final = final_w
                else: sk_idx_final = np.argmax(final_grad_result); sk_final = np.zeros(M); sk_final[sk_idx_final] = 1.0
                final_fw_gap_result = final_grad_result.T @ (final_w - sk_final) if not np.isnan(final_grad_result).any() else np.nan
            else:
                warnings.warn(f"Warning: Final inner QP solve failed after max iter: {msg_final}. Using last iteration's values.")
        else:
             warnings.warn(f"Warning: Could not find feasible pi0 for final solve after max iter. Using last iteration's values.")
    except Exception as e:
        warnings.warn(f"Error during final inner solve after max iter: {e}. Using last iteration's values.")

    if return_history:
        grad_norm_final = np.linalg.norm(final_grad_result) if not np.isnan(final_grad_result).any() else np.nan
        if grad_norm_final < tolerance * 10: sk_idx_final = -1
        else: sk_idx_final = np.argmax(final_grad_result) if not np.isnan(final_grad_result).any() else -1
        history.append({
            'k': max_outer_iter + 1, 'w_k': np.copy(final_w), 'H_k': final_H_result,
            'grad_H_k_norm': grad_norm_final, 's_k_index': sk_idx_final, 'fw_gap': final_fw_gap_result,
            'gamma_k': np.nan
        })

    result = OptimizationResult(True, f"Max Iter ({max_outer_iter})", w_opt=final_w, pi_opt=final_pi_result, lambda_opt=final_lam_result,
                                H_opt=final_H_result, grad_H_opt=final_grad_result, iterations=max_outer_iter,
                                fw_gap=final_fw_gap_result, active_set_opt=final_active_set_result)
    return (result, history) if return_history else result


# --- generate_params_profile_switching 関数 (マイルドな設定) ---
def generate_params_profile_switching(alpha, alpha_max, K=5, M=3, R_base=np.array([0.08, 0.07, 0.06, 0.05, 0.04]), sigma_base=np.array([0.18, 0.15, 0.20, 0.12, 0.10]), Corr_base=np.array([[1.0, 0.4, 0.3, 0.2, 0.1], [0.4, 1.0, 0.5, 0.3, 0.2], [0.3, 0.5, 1.0, 0.6, 0.4], [0.2, 0.3, 0.6, 1.0, 0.5], [0.1, 0.2, 0.4, 0.5, 1.0]]),
                                      good_prof_r_factor=0.01, good_prof_r_offset=0.0005, # <-- 変更
                                      good_prof_s_factor=-0.005, good_prof_corr_factor=1.0, # <-- 変更
                                      bad_prof_r_factor=-0.005, bad_prof_r_offset=-0.0002, # <-- 変更
                                      bad_prof_s_factor=0.005, bad_prof_corr_offset=0.0, # <-- 変更
                                      sigma_min_epsilon=1e-4, psd_tolerance=1e-9):
    assert alpha >= 0 and K == len(R_base) and K == len(sigma_base) and K == Corr_base.shape[0] and M == 3, "Dimension mismatch"
    non_diag_indices = np.where(~np.eye(K, dtype=bool)); R_neutral = R_base; sigma_neutral = np.maximum(sigma_min_epsilon, sigma_base); Corr_neutral = make_psd(Corr_base, psd_tolerance)
    R_good = R_base + (R_base * good_prof_r_factor + good_prof_r_offset); sigma_good = np.maximum(sigma_min_epsilon, sigma_base + (sigma_base * good_prof_s_factor)); Corr_good_target = np.copy(Corr_base); Corr_good_target[non_diag_indices] *= good_prof_corr_factor; Corr_good = make_psd(Corr_good_target, psd_tolerance)
    R_bad = R_base + (R_base * bad_prof_r_factor + bad_prof_r_offset); sigma_bad = np.maximum(sigma_min_epsilon, sigma_base + (sigma_base * bad_prof_s_factor)); Corr_bad_target = np.copy(Corr_base); Corr_bad_target[non_diag_indices] = np.clip(Corr_bad_target[non_diag_indices] + bad_prof_corr_offset, -1.0+psd_tolerance, 1.0-psd_tolerance); Corr_bad = make_psd(Corr_bad_target, psd_tolerance)
    beta = np.clip(alpha / alpha_max if alpha_max > 0 else (1.0 if alpha > 0 else 0.0), 0.0, 1.0)
    R_alpha = np.zeros((K, M)); sigma_alpha = np.zeros((K, M)); Corr_alpha = [np.eye(K) for _ in range(M)]; Cov_alpha = [np.eye(K) for _ in range(M)]; SecondMoments_a_array = np.zeros((M, K, K))
    R_alpha[:, 0] = (1 - beta) * R_good + beta * R_neutral; sigma_alpha[:, 0] = np.maximum(sigma_min_epsilon, (1 - beta) * sigma_good + beta * sigma_neutral); Corr_alpha[0] = make_psd((1 - beta) * Corr_good + beta * Corr_neutral, psd_tolerance)
    R_alpha[:, 1] = (1 - beta) * R_bad + beta * R_good; sigma_alpha[:, 1] = np.maximum(sigma_min_epsilon, (1 - beta) * sigma_bad + beta * sigma_good); Corr_alpha[1] = make_psd((1 - beta) * Corr_bad + beta * Corr_good, psd_tolerance)
    R_alpha[:, 2] = (1 - beta) * R_neutral + beta * R_bad; sigma_alpha[:, 2] = np.maximum(sigma_min_epsilon, (1 - beta) * sigma_neutral + beta * sigma_bad); Corr_alpha[2] = make_psd((1 - beta) * Corr_neutral + beta * Corr_bad, psd_tolerance)
    for m in range(M): sigma_diag_m = np.diag(sigma_alpha[:, m]); Cov_alpha[m] = sigma_diag_m @ Corr_alpha[m] @ sigma_diag_m; SecondMoments_a_array[m, :, :] = Cov_alpha[m] + np.outer(R_alpha[:, m], R_alpha[:, m])
    return R_alpha, SecondMoments_a_array

# === α依存性分析関数 (ActiveSet法を使用) ===
def analyze_alpha_full_results(alpha_range, param_gen_kwargs, optimizer_kwargs, mu_tilde):
    """ 指定された alpha 範囲で FW解、H*、pi*、grad H*、lambda*、active_set* を追跡する """
    results_over_alpha = []
    K = param_gen_kwargs.get('K', 5); M = param_gen_kwargs.get('M', 3)

    print("\n--- Starting Full Results Analysis over Alpha Range (using Active Set Method) ---")
    total_alphas = len(alpha_range)
    start_loop_time = time.time()

    for idx, alpha in enumerate(alpha_range):
        loop_start_time = time.time()
        print(f"\rAnalyzing alpha = {alpha:.6f} ({idx+1}/{total_alphas}) ... ", end="")

        # ★★★ 変更されたパラメータ生成 kwargs を使う ★★★
        R_alpha, SecondMoments_alpha_array = generate_params_profile_switching(alpha, **param_gen_kwargs)
        alpha_result = {'alpha': alpha}

        w_fw_default = np.full(M, np.nan)
        H_star_fw = np.nan
        pi_opt = np.full(K, np.nan)
        grad_H_opt = np.full(M, np.nan)
        lambda_opt = np.full(M, np.nan)
        active_set_opt = tuple()

        try:
            # --- FW Optimizer呼び出し (Active Setを使用) ---
            fw_result = frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde,
                                              initial_w=None, return_history=False,
                                              debug_print=False, # デバッグOFF
                                              **optimizer_kwargs)
            if fw_result.success:
                 w_fw_default = fw_result.w_opt if fw_result.w_opt is not None else np.full(M, np.nan)
                 H_star_fw = fw_result.H_opt if fw_result.H_opt is not None else np.nan
                 pi_opt = fw_result.pi_opt if fw_result.pi_opt is not None else np.full(K, np.nan)
                 grad_H_opt = fw_result.grad_H_opt if fw_result.grad_H_opt is not None else np.full(M, np.nan)
                 lambda_opt = fw_result.lambda_opt if fw_result.lambda_opt is not None else np.full(M, np.nan)
                 active_set_opt = fw_result.active_set_opt if fw_result.active_set_opt is not None else tuple()
            else:
                 w_fw_default = np.full(M, np.nan)
                 H_star_fw = np.nan
                 pi_opt = np.full(K, np.nan)
                 grad_H_opt = np.full(M, np.nan)
                 lambda_opt = np.full(M, np.nan)
                 active_set_opt = tuple()
                 print(f"\n  Warn: FW failed for alpha={alpha:.6f}: {fw_result.message}")

        except Exception as e:
            print(f"\n  Error during FW run for alpha={alpha:.6f}: {e}")
            w_fw_default = np.full(M, np.nan)
            H_star_fw = np.nan
            pi_opt = np.full(K, np.nan)
            grad_H_opt = np.full(M, np.nan)
            lambda_opt = np.full(M, np.nan)
            active_set_opt = tuple()

        alpha_result['w_fw_default'] = w_fw_default
        alpha_result['H_star'] = H_star_fw
        alpha_result['pi_opt'] = pi_opt
        alpha_result['grad_H_opt'] = grad_H_opt
        alpha_result['lambda_opt'] = lambda_opt
        alpha_result['active_set_opt'] = active_set_opt

        results_over_alpha.append(alpha_result)

    print("\n--- Finished Full Results Analysis ---")
    df_results = pd.DataFrame(results_over_alpha)
    if PANDAS_AVAILABLE:
        fw_vecs = np.stack([res.get('w_fw_default', np.full(M, np.nan)) for res in results_over_alpha])
        pi_vecs = np.stack([res.get('pi_opt', np.full(K, np.nan)) for res in results_over_alpha])
        grad_vecs = np.stack([res.get('grad_H_opt', np.full(M, np.nan)) for res in results_over_alpha])
        lambda_vecs = np.stack([res.get('lambda_opt', np.full(M, np.nan)) for res in results_over_alpha])
        active_sets = [str(res.get('active_set_opt', tuple())) for res in results_over_alpha]

        for m in range(M):
            df_results[f'w_fw_{m}'] = fw_vecs[:, m]
            df_results[f'grad_H_{m}'] = grad_vecs[:, m]
            df_results[f'lambda_{m}'] = lambda_vecs[:, m]
        for k in range(K):
            df_results[f'pi_{k}'] = pi_vecs[:, k]

        df_results['active_set'] = active_sets

        df_results = df_results.drop(columns=['w_fw_default', 'pi_opt', 'grad_H_opt', 'lambda_opt', 'active_set_opt'], errors='ignore')

    return df_results if PANDAS_AVAILABLE else results_over_alpha


# === メイン実行ブロック ===
if __name__ == '__main__':
    start_time_main = time.time()
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    warnings.filterwarnings('ignore', category=OptimizeWarning)

    # --- 設定 ---
    K = 5; M = 3;
    # ★★★ mu_tilde を 0.02 に設定 ★★★
    mu_tilde = 0.02
    alpha_max = 1.5

    # ★★★ プロファイル間の差を小さくした設定 ★★★
    profile_adjustments_mild = {
        'good_prof_r_factor': 0.01, 'good_prof_r_offset': 0.0005,
        'good_prof_s_factor': -0.005, 'good_prof_corr_factor': 1.0,
        'bad_prof_r_factor': -0.005, 'bad_prof_r_offset': -0.0002,
        'bad_prof_s_factor': 0.005, 'bad_prof_corr_offset': 0.0,
    }
    param_gen_kwargs_main = {'K': K, 'M': M, 'alpha_max': alpha_max, **profile_adjustments_mild} # <-- 変更後の設定を使用

    solver_settings = { # 精度設定
        'max_outer_iter': 500, 'fw_gap_tol': 1e-9,
        'inner_max_iter': 600,
        'tolerance': 1e-11,
        'psd_make_tolerance': 1e-9,
        'qp_regularization': 1e-10, # 元の正則化値
        'force_iterations': 20
    }
    # ★★★ 分析する alpha の範囲 (0.4 から 0.6 まで 0.001 刻み) ★★★
    alpha_start = 0.400
    alpha_end = 0.600
    alpha_step = 0.001
    num_alpha_steps = int(round((alpha_end - alpha_start) / alpha_step)) + 1
    alpha_range_analyze = np.linspace(alpha_start, alpha_end, num_alpha_steps)
    unique_pt_tolerance = 1e-5 # KKTチェック用

    output_csv_filename = f"alpha_internal_sol_search_{alpha_start:.3f}_{alpha_end:.3f}_activeset_mu002_mild.csv" # ファイル名変更

    print("-" * 60)
    print(f"--- Searching for Internal Solution around alpha=0.5 [{alpha_start:.3f}, {alpha_end:.3f}] ---")
    print(f"--- (Using ActiveSet, mu_tilde={mu_tilde}, Milder Profiles) ---")
    print("-" * 60)

    # --- 分析実行 ---
    results_df = analyze_alpha_full_results(
        alpha_range=alpha_range_analyze,
        param_gen_kwargs=param_gen_kwargs_main, # <-- 変更後の設定を渡す
        optimizer_kwargs=solver_settings,
        mu_tilde=mu_tilde
    )

    # --- 結果表示 & CSV保存 & KKTチェック ---
    if PANDAS_AVAILABLE and isinstance(results_df, pd.DataFrame):
        print("\n--- Analysis Results Summary (DataFrame) ---")
        float_format_func = lambda x: f"{x:.4e}" if pd.notna(x) and isinstance(x, (float, np.number)) else x
        pd.options.display.float_format = float_format_func
        cols_to_show = ['alpha', 'H_star', 'active_set'] + \
                       [f'w_fw_{m}' for m in range(M) if f'w_fw_{m}' in results_df.columns] + \
                       [f'pi_{k}' for k in range(K) if f'pi_{k}' in results_df.columns] + \
                       [f'grad_H_{m}' for m in range(M) if f'grad_H_{m}' in results_df.columns] + \
                       [f'lambda_{m}' for m in range(M) if f'lambda_{m}' in results_df.columns]
        cols_to_show = [col for col in cols_to_show if col in results_df.columns]
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 200): # 全行表示
             print(results_df[cols_to_show].to_string(index=False, na_rep='NaN'))
        pd.reset_option('display.float_format')

        # CSVファイルに保存 (フィルタリングなし、全データ)
        try:
            df_to_save = results_df.copy()
            cols_order = ['alpha', 'H_star', 'active_set'] + \
                         [f'w_fw_{m}' for m in range(M) if f'w_fw_{m}' in df_to_save.columns] + \
                         [f'pi_{k}' for k in range(K) if f'pi_{k}' in df_to_save.columns] + \
                         [f'grad_H_{m}' for m in range(M) if f'grad_H_{m}' in df_to_save.columns] + \
                         [f'lambda_{m}' for m in range(M) if f'lambda_{m}' in df_to_save.columns]
            cols_order = [col for col in cols_order if col in df_to_save.columns]
            df_to_save = df_to_save[cols_order]

            df_to_save.to_csv(output_csv_filename, index=False, float_format='%.8e')
            print(f"\nFull detailed results ({len(df_to_save)} points) saved to: {os.path.abspath(output_csv_filename)}")
        except Exception as e:
            print(f"\nError saving results to CSV: {e}")

        # KKT条件のチェック (簡易版)
        print("\n--- Quick KKT Check (Stationarity w.r.t. w) ---")
        kkt_violations = []
        # unique_pt_tolerance は main スコープで定義済み
        for index, row in results_df.iterrows():
            alpha = row['alpha']
            w_star = np.array([row.get(f'w_fw_{m}', np.nan) for m in range(M)])
            grad_h = np.array([row.get(f'grad_H_{m}', np.nan) for m in range(M)])
            active_set_str = row.get('active_set', '()')

            if np.isnan(w_star).any() or np.isnan(grad_h).any() or np.isclose(np.sum(w_star), 0):
                kkt_violations.append({
                    'alpha': alpha,
                    'active_set_report': active_set_str,
                    'active_set_kkt': 'NaN',
                    'max_violation': np.nan,
                    'grad_consistency_violation': np.nan,
                    'check_result': 'Skipped (NaN or zero w)'
                })
                continue

            check_tol = solver_settings['tolerance'] * 1e3

            max_grad_h = np.max(grad_h)
            nu_estimate = max_grad_h

            stationarity_violation = 0.0
            active_indices_kkt = set()
            for m in range(M):
                if w_star[m] > unique_pt_tolerance:
                    active_indices_kkt.add(m)

            active_grads = grad_h[list(active_indices_kkt)] if active_indices_kkt else []
            grad_consistency_violation = 0.0
            if len(active_grads) > 1:
                grad_consistency_violation = np.max(active_grads) - np.min(active_grads)
                if not np.isclose(grad_consistency_violation, 0.0, atol=check_tol):
                     stationarity_violation = max(stationarity_violation, grad_consistency_violation)

            for m in range(M):
                if m not in active_indices_kkt:
                    nu_for_check = np.min(active_grads) if active_grads else max_grad_h
                    eta_m_estimate = nu_for_check - grad_h[m]
                    if eta_m_estimate < -check_tol:
                        stationarity_violation = max(stationarity_violation, abs(eta_m_estimate))

            kkt_violations.append({
                'alpha': alpha,
                'active_set_report': active_set_str,
                'active_set_kkt': str(tuple(sorted(active_indices_kkt))),
                'max_violation': stationarity_violation,
                'grad_consistency_violation': grad_consistency_violation,
                'check_result': 'OK' if stationarity_violation < check_tol * 10 else 'WARN'
            })

        if PANDAS_AVAILABLE and kkt_violations:
            df_kkt = pd.DataFrame(kkt_violations)
            kkt_cols_order = ['alpha', 'active_set_report','active_set_kkt', 'max_violation', 'grad_consistency_violation', 'check_result']
            kkt_cols_order = [col for col in kkt_cols_order if col in df_kkt.columns]
            with pd.option_context('display.float_format', '{:.4e}'.format):
                 print(df_kkt[kkt_cols_order].to_string(index=False, na_rep='NaN'))
            print("Notes on KKT Check:")
            print(" - active_set_report: Active set reported by inner solver")
            print(" - active_set_kkt: Active set inferred from w* > tol")
            print(" - max_violation: Max KKT violation (dual infeasibility or comp. slackness based on eta estimate)")
            print(" - grad_consistency_violation: max(active_grads) - min(active_grads) for w_m > tol")
        else:
            print("Could not perform KKT check or pandas not available.")

    elif isinstance(results_df, list):
        print("\n--- Analysis Results Summary (List - First 5 & Last 5) ---")
        # (表示省略)
        print("\n(Pandas not available, skipping CSV export)")

    end_time_main = time.time()
    print(f"\nTotal execution time: {end_time_main - start_time_main:.2f} seconds")

------------------------------------------------------------
--- Searching for Internal Solution around alpha=0.5 [0.400, 0.600] ---
--- (Using ActiveSet, mu_tilde=0.02, Milder Profiles) ---
------------------------------------------------------------

--- Starting Full Results Analysis over Alpha Range (using Active Set Method) ---
Analyzing alpha = 0.600000 (201/201) ... 
--- Finished Full Results Analysis ---

--- Analysis Results Summary (DataFrame) ---
     alpha     H_star active_set     w_fw_0     w_fw_1     w_fw_2       pi_0       pi_1        pi_2       pi_3       pi_4   grad_H_0   grad_H_1   grad_H_2   lambda_0   lambda_1   lambda_2
4.0000e-01 9.3420e-04       (2,) 7.6260e-05 9.9985e-01 7.6260e-05 7.7666e-02 9.9536e-02 -4.8332e-02 9.8170e-02 1.2140e-01 5.2231e-04 5.3338e-04 5.3151e-04 0.0000e+00 0.0000e+00 9.3420e-02
4.0100e-01 9.3420e-04       (2,) 7.6260e-05 9.9985e-01 7.6260e-05 7.7667e-02 9.9537e-02 -4.8333e-02 9.8170e-02 1.2140e-01 5.2231e-04 5.3335e-04 5.3151e-04 0.0000e

In [3]:
# -*- coding: utf-8 -*-
"""
4_14_find_internal_solution_symmetric.py

Attempts to find an internal solution w* by using symmetric parameters
around alpha=0.75 (beta=0.5) and mu_tilde = 0.01.
Uses the Active Set method for the inner QP solve.
"""

import numpy as np
import matplotlib.pyplot as plt
import warnings
from scipy.linalg import solve, LinAlgError, eigh, inv, qr
from scipy.optimize import linprog, OptimizeWarning
import traceback
import time
import itertools
import os
import math
from collections import Counter # オプション

# pandas がなければインストールを促す
try:
    import pandas as pd
    PANDAS_AVAILABLE = True
    pd.set_option('display.width', 160)
    pd.set_option('display.float_format', '{:.6e}'.format) # 科学技術表記
    pd.set_option('display.max_rows', 200) # 表示行数を増やす
except ImportError:
    PANDAS_AVAILABLE = False
    print("Warning: pandas library not found. Output formatting will be basic. CSV export disabled.")

# === ここから必要な関数定義 ===
DEFAULT_TOLERANCE = 1e-9

# --- OptimizationResult クラス ---
class OptimizationResult:
    def __init__(self, success, message, w_opt=None, pi_opt=None, lambda_opt=None,
                 H_opt=None, grad_H_opt=None, iterations=None, fw_gap=None,
                 active_set_opt=None):
        self.success = success; self.message = message; self.w_opt = w_opt; self.pi_opt = pi_opt
        self.lambda_opt = lambda_opt; self.H_opt = H_opt; self.grad_H_opt = grad_H_opt
        self.iterations = iterations; self.fw_gap = fw_gap
        self.active_set_opt = active_set_opt

# --- find_feasible_initial_pi 関数 ---
def find_feasible_initial_pi(R, mu_tilde, K, tolerance=1e-8):
    M = R.shape[1]; c = np.zeros(K + 1); c[K] = 1.0
    A_ub = np.hstack((-R.T, -np.ones((M, 1)))); b_ub = -mu_tilde * np.ones(M)
    bounds = [(None, None)] * K + [(0, None)]; opts = {'tol': tolerance, 'disp': False, 'presolve': True}
    result = None; methods_to_try = ['highs', 'highs-ipm', 'highs-ds', 'simplex']
    for method in methods_to_try:
        try:
            with warnings.catch_warnings(): warnings.filterwarnings("ignore"); result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method=method, options=opts)
            if result.success: break
        except ValueError: continue
        except Exception as e: return None, False, f"Phase 1 LP failed: {e}"
    if result is None or not result.success: msg = result.message if result else "No solver"; status = result.status if result else -1; return None, False, f"Phase 1 LP solver failed: {msg} (status={status})"
    s = result.x[K]; pi = result.x[:K]
    if np.isnan(pi).any(): return None, False, "Phase 1 LP resulted in NaN values for pi."
    if s <= tolerance * 1000:
        G = -R.T; h = -mu_tilde * np.ones(M); violation = np.max(G @ pi - h)
        if violation <= tolerance * 10000: return pi, True, f"Phase 1 OK (s*={s:.1e}, vio={violation:.1e})"
        else: return pi, True, f"Phase 1 OK (s*={s:.1e}), WARN: violation {violation:.1e}"
    else: return None, False, f"Phase 1 Infeasible (s* = {s:.1e})"

# --- solve_kkt_system 関数 ---
def solve_kkt_system(Q, G_W, g, tolerance=DEFAULT_TOLERANCE):
    K = Q.shape[0]; n_act = G_W.shape[0] if G_W is not None and G_W.ndim == 2 and G_W.shape[0] > 0 else 0
    if n_act == 0:
        try: p = solve(Q, -g, assume_a='sym'); l = np.array([])
        except LinAlgError: return None, None, False
        res_norm = np.linalg.norm(Q @ p + g); g_norm = np.linalg.norm(g); solved_ok = res_norm <= tolerance * 1e3 * (1 + g_norm)
        return p, l, solved_ok
    else:
        kkt_mat = None; rhs = None
        try: kkt_mat = np.block([[Q, G_W.T], [G_W, np.zeros((n_act, n_act))]]) ; rhs = np.concatenate([-g, np.zeros(n_act)])
        except ValueError as e: return None, None, False
        try: sol = solve(kkt_mat, rhs, assume_a='sym'); p = sol[:K]; l = sol[K:]
        except LinAlgError: return None, None, False
        except ValueError as e: return None, None, False
        res_norm = np.linalg.norm(kkt_mat @ sol - rhs); rhs_norm = np.linalg.norm(rhs); solved_ok = res_norm <= tolerance * 1e3 * (1 + rhs_norm)
        return p, l, solved_ok

# --- solve_inner_qp_active_set 関数 ---
def solve_inner_qp_active_set(Vw, R, mu_tilde, initial_pi, max_iter=350, tolerance=DEFAULT_TOLERANCE, regularization_epsilon=1e-10):
    K = Vw.shape[0]; M = R.shape[1]; Q_reg = 2 * Vw + 2 * regularization_epsilon * np.eye(K); G = -R.T; h = -mu_tilde * np.ones(M)
    if initial_pi is None: return None, None, None, None, None, False, "No initial pi"
    pi_k = np.copy(initial_pi); lam_opt = np.zeros(M); W = set()
    active_tol = tolerance * 10
    initial_violations = G @ pi_k - h; W = set(j for j, viol in enumerate(initial_violations) if viol > -active_tol)
    active_indices_opt = None; kkt_matrix_opt = None
    if np.any(initial_violations > active_tol * 10): warnings.warn(f"Initial pi infeasible (max viol: {np.max(initial_violations):.2e}).")

    for i in range(max_iter):
        g_k = Q_reg @ pi_k; act = sorted(list(W)); n_act = len(act); G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))
        p_k, lam_Wk, solved = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)
        if not solved or p_k is None: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: KKT solve failed. ActiveSet={act}"
        if np.linalg.norm(p_k) <= tolerance * 10 * (1 + np.linalg.norm(pi_k)):
            is_optimal_point = True; blocking_constraint_idx = -1; min_negative_lambda = float('inf')
            dual_feas_tol = -tolerance * 10
            if W:
                if lam_Wk is None or len(lam_Wk) != n_act: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: lam_Wk inconsistent? ActiveSet={act}"
                lambda_map = dict(zip(act, lam_Wk))
                for constraint_idx, lagrange_multiplier in lambda_map.items():
                    if lagrange_multiplier < dual_feas_tol: is_optimal_point = False;
                    if lagrange_multiplier < min_negative_lambda: min_negative_lambda = lagrange_multiplier; blocking_constraint_idx = constraint_idx
            if is_optimal_point:
                lam_opt.fill(0.0);
                if W and len(lam_Wk) == n_act: lam_opt[act] = np.maximum(lam_Wk, 0)
                final_infeas = np.max(G @ pi_k - h); msg = f"Optimal found at iter {i+1}."
                if final_infeas > active_tol: msg += f" (WARN: violation {final_infeas:.1e})"
                active_indices_opt = act
                return pi_k, lam_opt, active_indices_opt, None, Q_reg / 2.0, True, msg
            else:
                if blocking_constraint_idx in W: W.remove(blocking_constraint_idx); continue
                else: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: Neg lambda idx {blocking_constraint_idx}, not in W={act}"
        else:
            alpha_k = 1.0; blocking_constraint_idx = -1; min_step_length = float('inf')
            step_tol = tolerance * 10
            for j in range(M):
                if j not in W:
                    constraint_gradient_dot_p = G[j, :] @ p_k
                    if constraint_gradient_dot_p > step_tol:
                        distance_to_boundary = h[j] - (G[j, :] @ pi_k)
                        if abs(constraint_gradient_dot_p) > 1e-15:
                            alpha_j = distance_to_boundary / constraint_gradient_dot_p
                            step_j = max(0.0, alpha_j)
                            if step_j < min_step_length:
                                min_step_length = step_j; blocking_constraint_idx = j
            alpha_k = min(1.0, min_step_length); pi_k += alpha_k * p_k
            if alpha_k < 1.0 - step_tol and blocking_constraint_idx != -1:
                if blocking_constraint_idx not in W: W.add(blocking_constraint_idx)
            continue
    msg = f"Max iter ({max_iter}) reached."; final_infeas = np.max(G @ pi_k - h)
    if final_infeas > active_tol * 10: return None, None, None, None, None, False, f"{msg} Final infeasible. ActiveSet={sorted(list(W))}"
    act = sorted(list(W)); n_act = len(act); G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))
    is_likely_optimal = False; active_constraints_opt = act
    g_k = Q_reg @ pi_k; p_f, lam_f, solved_f = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)
    final_lambda_estimate = np.zeros(M)
    if solved_f and p_f is not None and np.linalg.norm(p_f) <= tolerance * 100 * (1 + np.linalg.norm(pi_k)):
        if n_act > 0 and lam_f is not None and len(lam_f) == n_act:
             try: final_lambda_estimate[act] = lam_f
             except IndexError: pass
        active_lambdas = final_lambda_estimate[act] if n_act > 0 else np.array([])
        if n_act == 0 or np.all(active_lambdas >= -tolerance * 100): is_likely_optimal = True; msg += " Final KKT check approx OK."
        else: msg += " Final KKT check fails (dual infeasible)."
    else: msg += " Final KKT check fails (stationarity or solve error)."
    lam_opt = final_lambda_estimate
    return pi_k, lam_opt, active_constraints_opt, None, Q_reg / 2.0, is_likely_optimal, msg

# --- make_psd 関数 ---
def make_psd(matrix, tolerance=1e-8):
    sym = (matrix + matrix.T) / 2
    try: eigenvalues, eigenvectors = np.linalg.eigh(sym); min_eigenvalue = np.min(eigenvalues)
    except LinAlgError: warnings.warn("..."); return sym
    if min_eigenvalue < tolerance: eigenvalues[eigenvalues < tolerance] = tolerance; psd_matrix = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T; return (psd_matrix + psd_matrix.T) / 2
    else: return sym

# --- calculate_Vw 関数 ---
def calculate_Vw(w, R, SecondMoments_a_array, tolerance=DEFAULT_TOLERANCE, psd_tolerance=1e-8):
    K, M = R.shape; w_sum = np.sum(w); w_norm = w
    EwX = R @ w_norm; EwXXT = np.zeros((K, K));
    for m in range(M): EwXXT += w_norm[m] * SecondMoments_a_array[m]
    Vw = EwXXT - np.outer(EwX, EwX); Vw_psd = make_psd(Vw, psd_tolerance)
    return Vw_psd, EwX, EwXXT

# --- calculate_H_gradient 関数 (ノルムチェック緩和) ---
def calculate_H_gradient(pi_star, w, R, SecondMoments_a_array, EwX, EwXXT, tolerance=1e-9, debug_print=False):
    M = w.shape[0]; K = R.shape[0]; grad = np.zeros(M)
    norm_tolerance = 1e-12 # Allow very small pi_star

    if pi_star is None or EwX is None:
        if debug_print: print("DEBUG grad_H: Returning NaN due to None input (pi_star or EwX).")
        return np.full(M, np.nan)
    if np.isnan(pi_star).any() or np.isinf(pi_star).any():
         if debug_print: print("DEBUG grad_H: Returning NaN because pi_star contains NaN/Inf.")
         return np.full(M, np.nan)
    pi_norm = np.linalg.norm(pi_star)
    if pi_norm < norm_tolerance:
         if debug_print: print(f"DEBUG grad_H: Returning NaN because pi_star norm {pi_norm:.2e} < {norm_tolerance:.1e}")
         return np.full(M, np.nan)
    if np.isnan(EwX).any() or np.isinf(EwX).any():
        if debug_print: print("DEBUG grad_H: Returning NaN because EwX contains NaN/Inf.")
        return np.full(M, np.nan)

    try:
        pi_T_EwX = pi_star.T @ EwX
        if np.isnan(pi_T_EwX) or np.isinf(pi_T_EwX):
             if debug_print: print(f"DEBUG grad_H: pi_T_EwX is NaN/Inf: {pi_T_EwX}")
             return np.full(M, np.nan)

        for j in range(M):
            Sigma_j = SecondMoments_a_array[j]; r_j = R[:, j]
            pi_T_Sigma_j_pi = pi_star.T @ Sigma_j @ pi_star
            pi_T_r_j = pi_star.T @ r_j

            if np.isnan(pi_T_Sigma_j_pi) or np.isinf(pi_T_Sigma_j_pi):
                 if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_Sigma_j_pi is NaN/Inf: {pi_T_Sigma_j_pi}")
                 grad[j] = np.nan; continue
            if np.isnan(pi_T_r_j) or np.isinf(pi_T_r_j):
                 if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_r_j is NaN/Inf: {pi_T_r_j}")
                 grad[j] = np.nan; continue

            term2 = 2 * pi_T_r_j * pi_T_EwX
            if np.isnan(term2) or np.isinf(term2):
                 if debug_print: print(f"DEBUG grad_H (j={j}): term2 (2*pi_T_r_j*pi_T_EwX) is NaN/Inf: {term2}, pi_T_r_j={pi_T_r_j}, pi_T_EwX={pi_T_EwX}")
                 grad[j] = np.nan; continue

            grad[j] = pi_T_Sigma_j_pi - term2
            if np.isnan(grad[j]) or np.isinf(grad[j]):
                 if debug_print: print(f"DEBUG grad_H (j={j}): final grad[{j}] is NaN/Inf")

    except Exception as e_calc:
         print(f"ERROR in calculate_H_gradient calculation: {e_calc}")
         print(traceback.format_exc())
         return np.full(M, np.nan)

    if np.any(np.isnan(grad)) or np.any(np.isinf(grad)):
        warnings.warn(f"NaN or Inf detected in calculated gradient for w={w}")
        if debug_print: print(f"DEBUG grad_H: Final check found NaN/Inf in gradient: {grad}")
        return np.full(M, np.nan)
    return grad

# --- project_to_simplex 関数 ---
def project_to_simplex(v, z=1):
    n_features = v.shape[0];
    if n_features == 0: return np.array([])
    v_arr = np.asarray(v)
    if np.all(v_arr >= -1e-9) and np.isclose(np.sum(v_arr), z): return np.maximum(v_arr, 0)
    u = np.sort(v_arr)[::-1]; cssv = np.cumsum(u) - z; ind = np.arange(n_features) + 1; cond = u - cssv / ind > 0
    if np.any(cond): rho = ind[cond][-1]; theta = cssv[rho - 1] / float(rho); w = np.maximum(v_arr - theta, 0)
    else:
         w = np.zeros(n_features)
         if z > 0: w[np.argmax(v_arr)] = z
    w_sum = np.sum(w)
    if not np.isclose(w_sum, z):
        if w_sum > 1e-9: w = w * (z / w_sum)
        elif z > 0 :
            w = np.zeros(n_features)
            w[np.argmax(v_arr)] = z
    return np.maximum(w, 0)

# --- frank_wolfe_optimizer 関数 (ActiveSet法を使用、最終解の整合性を高める修正済み) ---
def frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde, initial_w=None, max_outer_iter=250, fw_gap_tol=1e-7, inner_max_iter=350, tolerance=1e-9, psd_make_tolerance=1e-8, qp_regularization=1e-10, debug_print=False, force_iterations=0, return_history=False):
    K, M = R_alpha.shape
    if initial_w is None: w_k = np.ones(M) / M
    else: w_k = project_to_simplex(np.copy(initial_w))
    if w_k is None:
        result = OptimizationResult(False, "Initial projection failed", w_opt=initial_w)
        return (result, []) if return_history else result
    fw_gap = float('inf'); best_w = np.copy(w_k); best_pi = None; best_lam = np.zeros(M); best_H = -float('inf'); final_gHk = np.zeros(M); best_active_set = None
    pi0, ok, p1msg = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance)
    if not ok:
        result = OptimizationResult(False, f"Phase 1 failed: {p1msg}", w_opt=initial_w)
        return (result, []) if return_history else result
    current_w=np.copy(w_k); current_H=-float('inf'); current_pi=None; current_lam=None; current_active_set=None
    inner_solver_args_for_loop = {'max_iter': inner_max_iter, 'tolerance': tolerance, 'regularization_epsilon': qp_regularization}
    history = []

    last_successful_pi = pi0
    last_successful_lam = np.zeros(M)
    last_successful_active_set = tuple()
    last_successful_gHk = np.zeros(M)
    last_successful_fw_gap = float('inf')

    for k in range(max_outer_iter):
        iter_data = {'k': k + 1}
        if return_history: iter_data['w_k'] = np.copy(w_k)
        try: Vk, Ex, ExxT = calculate_Vw(w_k, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
        except Exception as e:
            result = OptimizationResult(False, f"Outer iter {k+1}: Vw failed: {e}", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, active_set_opt=best_active_set)
            if return_history: history.append(iter_data); return (result, history)
            else: return result

        # === Use Active Set for inner solve ===
        pi_init_inner = last_successful_pi if last_successful_pi is not None else pi0
        pk, lk, act_idx_k, _, Vw_reg_k, inner_ok, inner_msg = solve_inner_qp_active_set(
            Vk, R_alpha, mu_tilde, pi_init_inner, **inner_solver_args_for_loop
        )
        # ===================================

        if not inner_ok or pk is None:
            warnings.warn(f"Outer iter {k+1}: Inner QP failed: {inner_msg}. Using last successful state if available.")
            if last_successful_pi is None:
                 result = OptimizationResult(False, f"Outer iter {k+1}: Inner QP failed and no prior success: {inner_msg}",
                                             w_opt=current_w, iterations=k)
                 if return_history: history.append(iter_data); return (result, history)
                 else: return result
            pk = last_successful_pi
            lk = last_successful_lam
            act_idx_k = list(last_successful_active_set)
            final_gHk = last_successful_gHk
            fw_gap = last_successful_fw_gap
            try: Hk = pk.T @ Vk @ pk
            except: Hk = current_H
        else:
            last_successful_pi = pk
            last_successful_lam = lk if lk is not None else np.zeros(M)
            last_successful_active_set = tuple(sorted(act_idx_k)) if act_idx_k is not None else tuple()
            Hk = pk.T @ Vk @ pk
            try:
                gHk = calculate_H_gradient(pk, w_k, R_alpha, SecondMoments_alpha_array, Ex, ExxT, tolerance, debug_print=debug_print)
            except Exception as e:
                result = OptimizationResult(False, f"Outer iter {k+1}: Grad failed: {e}", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
                if return_history: history.append(iter_data); return (result, history)
                else: return result

            final_gHk = gHk if gHk is not None else np.full(M, np.nan)
            last_successful_gHk = final_gHk if not np.isnan(final_gHk).any() else last_successful_gHk

        if return_history: iter_data['H_k'] = Hk
        if return_history: iter_data['grad_H_k_norm'] = np.linalg.norm(final_gHk) if not np.isnan(final_gHk).any() else np.nan

        current_w = np.copy(w_k); current_H = Hk; current_pi = pk; current_lam = last_successful_lam; current_active_set = last_successful_active_set

        if Hk >= best_H - tolerance*1000:
             best_H = Hk; best_w = np.copy(w_k); best_pi = np.copy(pk); best_lam = np.copy(current_lam); best_active_set = current_active_set

        if final_gHk is None or np.isnan(final_gHk).any():
            result = OptimizationResult(False, f"Outer iter {k+1}: Grad NaN.", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
            if return_history: history.append(iter_data); return (result, history)
            else: return result

        grad_norm = np.linalg.norm(final_gHk)
        if grad_norm < tolerance * 10: sk = w_k; sk_idx = -1
        else: sk_idx = np.argmax(final_gHk); sk = np.zeros(M); sk[sk_idx] = 1.0
        if return_history: iter_data['s_k_index'] = sk_idx

        if w_k is None:
             result = OptimizationResult(False, f"k={k} w_k None", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
             if return_history: history.append(iter_data); return (result, history)
             else: return result

        fw_gap = final_gHk.T @ (w_k - sk)
        last_successful_fw_gap = fw_gap
        if return_history: iter_data['fw_gap'] = fw_gap
        gamma = 2.0 / (k + 3.0)
        if return_history: iter_data['gamma_k'] = gamma
        if return_history: history.append(iter_data)

        converged = False
        if k >= force_iterations and not np.isnan(fw_gap):
            if abs(fw_gap) <= fw_gap_tol: converged = True; conv_msg = f"Converged (Gap {abs(fw_gap):.2e})"
        if converged:
            # --- Final Inner Solve for Consistency ---
            final_w = current_w
            final_pi_result = current_pi
            final_lam_result = current_lam
            final_active_set_result = current_active_set
            final_H_result = current_H
            final_grad_result = final_gHk
            final_fw_gap_result = fw_gap

            try:
                final_Vk, final_Ex, final_ExxT = calculate_Vw(final_w, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
                pi0_final, ok_final, _ = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance)
                if ok_final:
                    pi_s_final, lam_s_final, act_idx_final, _, _, inner_ok_final, msg_final = solve_inner_qp_active_set(
                        final_Vk, R_alpha, mu_tilde, pi0_final, # Use stable pi0 for final check
                        **inner_solver_args_for_loop
                    )
                    if inner_ok_final and pi_s_final is not None and lam_s_final is not None:
                        final_pi_result = pi_s_final
                        final_lam_result = lam_s_final
                        final_active_set_result = tuple(sorted(act_idx_final)) if act_idx_final is not None else tuple()
                        final_grad_result = calculate_H_gradient(final_pi_result, final_w, R_alpha, SecondMoments_alpha_array, final_Ex, final_ExxT, tolerance, debug_print=False)
                        if final_grad_result is None or np.isnan(final_grad_result).any(): final_grad_result = np.zeros(M)
                        grad_norm_final = np.linalg.norm(final_grad_result) if not np.isnan(final_grad_result).any() else np.nan
                        if grad_norm_final < tolerance * 10: sk_final = final_w
                        else: sk_idx_final = np.argmax(final_grad_result); sk_final = np.zeros(M); sk_final[sk_idx_final] = 1.0
                        final_fw_gap_result = final_grad_result.T @ (final_w - sk_final) if not np.isnan(final_grad_result).any() else np.nan
                    else:
                        warnings.warn(f"Warning: Final inner QP solve failed after convergence: {msg_final}. Using last iteration's values.")
                else:
                     warnings.warn(f"Warning: Could not find feasible pi0 for final solve after convergence. Using last iteration's values.")
            except Exception as e:
                warnings.warn(f"Error during final inner solve after convergence: {e}. Using last iteration's values.")
            # --- End of Final Inner Solve ---

            result = OptimizationResult(True, conv_msg, w_opt=final_w, pi_opt=final_pi_result, lambda_opt=final_lam_result,
                                        H_opt=final_H_result, grad_H_opt=final_grad_result, iterations=k + 1,
                                        fw_gap=final_fw_gap_result, active_set_opt=final_active_set_result)
            return (result, history) if return_history else result

        # --- Prepare for next iteration ---
        if w_k is None or sk is None:
             result = OptimizationResult(False, f"k={k} w_k/sk None before update", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
             return (result, history) if return_history else result
        w_k_next = (1.0 - gamma) * w_k + gamma * sk; w_k = project_to_simplex(w_k_next)
        if w_k is None:
             result = OptimizationResult(False, f"k={k} proj None", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
             return (result, history) if return_history else result
        pi0 = pk # Use last inner solution as initial guess

    # Max iter reached, perform final solve for consistency
    final_w = current_w
    final_pi_result = current_pi
    final_lam_result = current_lam
    final_active_set_result = current_active_set
    final_H_result = current_H
    final_grad_result = last_successful_gHk
    final_fw_gap_result = last_successful_fw_gap

    try:
        final_Vk, final_Ex, final_ExxT = calculate_Vw(final_w, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
        pi0_final, ok_final, _ = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance) # Use stable pi0
        if ok_final:
            pi_s_final, lam_s_final, act_idx_final, _, _, inner_ok_final, msg_final = solve_inner_qp_active_set(
                final_Vk, R_alpha, mu_tilde, pi0_final, # <-- Use stable pi0_final
                **inner_solver_args_for_loop
            )
            if inner_ok_final and pi_s_final is not None and lam_s_final is not None:
                final_pi_result = pi_s_final
                final_lam_result = lam_s_final
                final_active_set_result = tuple(sorted(act_idx_final)) if act_idx_final is not None else tuple()
                # Recalculate gradient and gap
                final_grad_result = calculate_H_gradient(final_pi_result, final_w, R_alpha, SecondMoments_alpha_array, final_Ex, final_ExxT, tolerance, debug_print=False)
                if final_grad_result is None or np.isnan(final_grad_result).any(): final_grad_result = np.zeros(M)
                grad_norm_final = np.linalg.norm(final_grad_result) if not np.isnan(final_grad_result).any() else np.nan
                if grad_norm_final < tolerance * 10: sk_final = final_w
                else: sk_idx_final = np.argmax(final_grad_result); sk_final = np.zeros(M); sk_final[sk_idx_final] = 1.0
                final_fw_gap_result = final_grad_result.T @ (final_w - sk_final) if not np.isnan(final_grad_result).any() else np.nan
            else:
                warnings.warn(f"Warning: Final inner QP solve failed after max iter: {msg_final}. Using last iteration's values.")
        else:
             warnings.warn(f"Warning: Could not find feasible pi0 for final solve after max iter. Using last iteration's values.")
    except Exception as e:
        warnings.warn(f"Error during final inner solve after max iter: {e}. Using last iteration's values.")

    if return_history:
        grad_norm_final = np.linalg.norm(final_grad_result) if not np.isnan(final_grad_result).any() else np.nan
        if grad_norm_final < tolerance * 10: sk_idx_final = -1
        else: sk_idx_final = np.argmax(final_grad_result) if not np.isnan(final_grad_result).any() else -1
        history.append({
            'k': max_outer_iter + 1, 'w_k': np.copy(final_w), 'H_k': final_H_result,
            'grad_H_k_norm': grad_norm_final, 's_k_index': sk_idx_final, 'fw_gap': final_fw_gap_result,
            'gamma_k': np.nan
        })

    result = OptimizationResult(True, f"Max Iter ({max_outer_iter})", w_opt=final_w, pi_opt=final_pi_result, lambda_opt=final_lam_result,
                                H_opt=final_H_result, grad_H_opt=final_grad_result, iterations=max_outer_iter,
                                fw_gap=final_fw_gap_result, active_set_opt=final_active_set_result)
    return (result, history) if return_history else result

# === generate_params_profile_switching 関数 (対称設定) ---
def generate_params_profile_switching_symmetric(alpha, alpha_max, K=5, M=3,
                                      R_base_sym=np.array([0.02, 0.01, 0.0, -0.01, -0.02]), # Neutral=0中心
                                      sigma_base=np.array([0.18, 0.15, 0.20, 0.12, 0.10]),
                                      Corr_base=np.array([[1.0, 0.4, 0.3, 0.2, 0.1], [0.4, 1.0, 0.5, 0.3, 0.2], [0.3, 0.5, 1.0, 0.6, 0.4], [0.2, 0.3, 0.6, 1.0, 0.5], [0.1, 0.2, 0.4, 0.5, 1.0]]),
                                      r_offset=0.03, # Good/Badのオフセット
                                      s_factor=0.0,  # V^G=V^N=V^B を仮定
                                      corr_factor=1.0, # Corrは同じと仮定
                                      corr_offset=0.0, # Corrは同じと仮定
                                      sigma_min_epsilon=1e-4, psd_tolerance=1e-9):
    """ Generates symmetric parameters assuming V^G=V^N=V^B """
    assert K == len(R_base_sym) and K == len(sigma_base) and K == Corr_base.shape[0] and M == 3, "Dimension mismatch"

    R_neutral = R_base_sym # Neutral expectation is the base
    R_good = R_base_sym + r_offset
    R_bad = R_base_sym - r_offset # Symmetric bad profile

    # Assume V^G = V^N = V^B = V_base
    sigma_neutral = np.maximum(sigma_min_epsilon, sigma_base)
    Corr_neutral = make_psd(Corr_base, psd_tolerance)
    V_base = np.diag(sigma_neutral) @ Corr_neutral @ np.diag(sigma_neutral)

    # Calculate Sigma^m = V_base + r^m * r^m.T
    Sigma_neutral = V_base + np.outer(R_neutral, R_neutral)
    Sigma_good = V_base + np.outer(R_good, R_good)
    Sigma_bad = V_base + np.outer(R_bad, R_bad)

    # Apply profile switching based on alpha
    beta = np.clip(alpha / alpha_max if alpha_max > 0 else (1.0 if alpha > 0 else 0.0), 0.0, 1.0)

    R_alpha = np.zeros((K, M))
    SecondMoments_a_array = np.zeros((M, K, K))

    # Scenario 0: Mix Good and Neutral
    R_alpha[:, 0] = (1 - beta) * R_good + beta * R_neutral
    SecondMoments_a_array[0, :, :] = (1 - beta) * Sigma_good + beta * Sigma_neutral

    # Scenario 1: Mix Bad and Good
    R_alpha[:, 1] = (1 - beta) * R_bad + beta * R_good
    SecondMoments_a_array[1, :, :] = (1 - beta) * Sigma_bad + beta * Sigma_good

    # Scenario 2: Mix Neutral and Bad
    R_alpha[:, 2] = (1 - beta) * R_neutral + beta * R_bad
    SecondMoments_a_array[2, :, :] = (1 - beta) * Sigma_neutral + beta * Sigma_bad

    return R_alpha, SecondMoments_a_array


# === α依存性分析関数 (ActiveSet法を使用) ===
def analyze_alpha_full_results(alpha_range, param_gen_kwargs, optimizer_kwargs, mu_tilde):
    """ 指定された alpha 範囲で FW解、H*、pi*、grad H*、lambda*、active_set* を追跡する """
    results_over_alpha = []
    K = param_gen_kwargs.get('K', 5); M = param_gen_kwargs.get('M', 3)

    print("\n--- Starting Full Results Analysis over Alpha Range (using Active Set Method) ---")
    total_alphas = len(alpha_range)
    start_loop_time = time.time()

    for idx, alpha in enumerate(alpha_range):
        loop_start_time = time.time()
        print(f"\rAnalyzing alpha = {alpha:.6f} ({idx+1}/{total_alphas}) ... ", end="")

        # === パラメータ生成 ===
        # ★★★ 対称パラメータ生成関数を呼び出す ★★★
        R_alpha, SecondMoments_alpha_array = generate_params_profile_switching_symmetric(alpha, **param_gen_kwargs)
        alpha_result = {'alpha': alpha}

        w_fw_default = np.full(M, np.nan)
        H_star_fw = np.nan
        pi_opt = np.full(K, np.nan)
        grad_H_opt = np.full(M, np.nan)
        lambda_opt = np.full(M, np.nan)
        active_set_opt = tuple()

        try:
            # --- FW Optimizer呼び出し (Active Setを使用) ---
            fw_result = frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde,
                                              initial_w=None, return_history=False,
                                              debug_print=False, # デバッグOFF
                                              **optimizer_kwargs)
            if fw_result.success:
                 w_fw_default = fw_result.w_opt if fw_result.w_opt is not None else np.full(M, np.nan)
                 H_star_fw = fw_result.H_opt if fw_result.H_opt is not None else np.nan
                 pi_opt = fw_result.pi_opt if fw_result.pi_opt is not None else np.full(K, np.nan)
                 grad_H_opt = fw_result.grad_H_opt if fw_result.grad_H_opt is not None else np.full(M, np.nan)
                 lambda_opt = fw_result.lambda_opt if fw_result.lambda_opt is not None else np.full(M, np.nan)
                 active_set_opt = fw_result.active_set_opt if fw_result.active_set_opt is not None else tuple()
            else:
                 w_fw_default = np.full(M, np.nan)
                 H_star_fw = np.nan
                 pi_opt = np.full(K, np.nan)
                 grad_H_opt = np.full(M, np.nan)
                 lambda_opt = np.full(M, np.nan)
                 active_set_opt = tuple()
                 print(f"\n  Warn: FW failed for alpha={alpha:.6f}: {fw_result.message}")

        except Exception as e:
            print(f"\n  Error during FW run for alpha={alpha:.6f}: {e}")
            w_fw_default = np.full(M, np.nan)
            H_star_fw = np.nan
            pi_opt = np.full(K, np.nan)
            grad_H_opt = np.full(M, np.nan)
            lambda_opt = np.full(M, np.nan)
            active_set_opt = tuple()

        alpha_result['w_fw_default'] = w_fw_default
        alpha_result['H_star'] = H_star_fw
        alpha_result['pi_opt'] = pi_opt
        alpha_result['grad_H_opt'] = grad_H_opt
        alpha_result['lambda_opt'] = lambda_opt
        alpha_result['active_set_opt'] = active_set_opt

        results_over_alpha.append(alpha_result)

    print("\n--- Finished Full Results Analysis ---")
    df_results = pd.DataFrame(results_over_alpha)
    if PANDAS_AVAILABLE:
        fw_vecs = np.stack([res.get('w_fw_default', np.full(M, np.nan)) for res in results_over_alpha])
        pi_vecs = np.stack([res.get('pi_opt', np.full(K, np.nan)) for res in results_over_alpha])
        grad_vecs = np.stack([res.get('grad_H_opt', np.full(M, np.nan)) for res in results_over_alpha])
        lambda_vecs = np.stack([res.get('lambda_opt', np.full(M, np.nan)) for res in results_over_alpha])
        active_sets = [str(res.get('active_set_opt', tuple())) for res in results_over_alpha]

        for m in range(M):
            df_results[f'w_fw_{m}'] = fw_vecs[:, m]
            df_results[f'grad_H_{m}'] = grad_vecs[:, m]
            df_results[f'lambda_{m}'] = lambda_vecs[:, m]
        for k in range(K):
            df_results[f'pi_{k}'] = pi_vecs[:, k]

        df_results['active_set'] = active_sets

        df_results = df_results.drop(columns=['w_fw_default', 'pi_opt', 'grad_H_opt', 'lambda_opt', 'active_set_opt'], errors='ignore')

    return df_results if PANDAS_AVAILABLE else results_over_alpha


# === メイン実行ブロック ===
if __name__ == '__main__':
    start_time_main = time.time()
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    warnings.filterwarnings('ignore', category=OptimizeWarning)

    # --- 設定 ---
    K = 5; M = 3;
    # ★★★ mu_tilde を 0.01 に設定 ★★★
    mu_tilde = 0.01
    alpha_max = 1.5 # これはbeta計算に使うだけ

    # ★★★ 対称性を意識したパラメータ設定 ★★★
    param_gen_kwargs_symmetric = {
        'K': K, 'M': M, 'alpha_max': alpha_max,
        'R_base_sym': np.array([0.02, 0.01, 0.0, -0.01, -0.02]), # Neutral=0中心
        'sigma_base': np.array([0.18, 0.15, 0.20, 0.12, 0.10]), # 元のまま
        'Corr_base': np.array([[1.0, 0.4, 0.3, 0.2, 0.1], [0.4, 1.0, 0.5, 0.3, 0.2], [0.3, 0.5, 1.0, 0.6, 0.4], [0.2, 0.3, 0.6, 1.0, 0.5], [0.1, 0.2, 0.4, 0.5, 1.0]]), # 元のまま
        'r_offset': 0.03, # Good/Badのオフセット
        's_factor': 0.0,  # V^G=V^N=V^B を仮定
        'corr_factor': 1.0,
        'corr_offset': 0.0,
        'sigma_min_epsilon': 1e-4,
        'psd_tolerance': 1e-9
    }

    solver_settings = { # 精度設定
        'max_outer_iter': 500, 'fw_gap_tol': 1e-9,
        'inner_max_iter': 600,
        'tolerance': 1e-11,
        'psd_make_tolerance': 1e-9,
        'qp_regularization': 1e-10,
        'force_iterations': 20
    }
    # ★★★ 分析する alpha の範囲 (0.75 付近) ★★★
    alpha_start = 0.700
    alpha_end = 0.800
    alpha_step = 0.001
    num_alpha_steps = int(round((alpha_end - alpha_start) / alpha_step)) + 1
    alpha_range_analyze = np.linspace(alpha_start, alpha_end, num_alpha_steps)
    unique_pt_tolerance = 1e-5 # KKTチェック用

    output_csv_filename = f"alpha_internal_sol_search_{alpha_start:.3f}_{alpha_end:.3f}_activeset_mu001_symmetric.csv" # ファイル名変更

    print("-" * 60)
    print(f"--- Searching for Internal Solution around alpha=0.75 [{alpha_start:.3f}, {alpha_end:.3f}] ---")
    print(f"--- (Using ActiveSet, mu_tilde={mu_tilde}, Symmetric Params) ---")
    print("-" * 60)

    # --- 分析実行 ---
    results_df = analyze_alpha_full_results(
        alpha_range=alpha_range_analyze,
        param_gen_kwargs=param_gen_kwargs_symmetric, # <-- 対称設定を使用
        optimizer_kwargs=solver_settings,
        mu_tilde=mu_tilde # <-- 変更した mu_tilde を渡す
    )

    # --- 結果表示 & CSV保存 & KKTチェック ---
    if PANDAS_AVAILABLE and isinstance(results_df, pd.DataFrame):
        print("\n--- Analysis Results Summary (DataFrame) ---")
        float_format_func = lambda x: f"{x:.4e}" if pd.notna(x) and isinstance(x, (float, np.number)) else x
        pd.options.display.float_format = float_format_func
        cols_to_show = ['alpha', 'H_star', 'active_set'] + \
                       [f'w_fw_{m}' for m in range(M) if f'w_fw_{m}' in results_df.columns] + \
                       [f'pi_{k}' for k in range(K) if f'pi_{k}' in results_df.columns] + \
                       [f'grad_H_{m}' for m in range(M) if f'grad_H_{m}' in results_df.columns] + \
                       [f'lambda_{m}' for m in range(M) if f'lambda_{m}' in results_df.columns]
        cols_to_show = [col for col in cols_to_show if col in results_df.columns]
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 200):
             print(results_df[cols_to_show].to_string(index=False, na_rep='NaN'))
        pd.reset_option('display.float_format')

        # CSVファイルに保存 (フィルタリングなし、全データ)
        try:
            df_to_save = results_df.copy()
            cols_order = ['alpha', 'H_star', 'active_set'] + \
                         [f'w_fw_{m}' for m in range(M) if f'w_fw_{m}' in df_to_save.columns] + \
                         [f'pi_{k}' for k in range(K) if f'pi_{k}' in df_to_save.columns] + \
                         [f'grad_H_{m}' for m in range(M) if f'grad_H_{m}' in df_to_save.columns] + \
                         [f'lambda_{m}' for m in range(M) if f'lambda_{m}' in df_to_save.columns]
            cols_order = [col for col in cols_order if col in df_to_save.columns]
            df_to_save = df_to_save[cols_order]

            df_to_save.to_csv(output_csv_filename, index=False, float_format='%.8e')
            print(f"\nFull detailed results ({len(df_to_save)} points) saved to: {os.path.abspath(output_csv_filename)}")
        except Exception as e:
            print(f"\nError saving results to CSV: {e}")

        # KKT条件のチェック (簡易版)
        print("\n--- Quick KKT Check (Stationarity w.r.t. w) ---")
        kkt_violations = []
        # unique_pt_tolerance は main スコープで定義済み
        for index, row in results_df.iterrows():
            alpha = row['alpha']
            w_star = np.array([row.get(f'w_fw_{m}', np.nan) for m in range(M)])
            grad_h = np.array([row.get(f'grad_H_{m}', np.nan) for m in range(M)])
            active_set_str = row.get('active_set', '()')

            if np.isnan(w_star).any() or np.isnan(grad_h).any() or np.isclose(np.sum(w_star), 0):
                kkt_violations.append({
                    'alpha': alpha,
                    'active_set_report': active_set_str,
                    'active_set_kkt': 'NaN',
                    'max_violation': np.nan,
                    'grad_consistency_violation': np.nan,
                    'check_result': 'Skipped (NaN or zero w)'
                })
                continue

            check_tol = solver_settings['tolerance'] * 1e3

            max_grad_h = np.max(grad_h)
            nu_estimate = max_grad_h

            stationarity_violation = 0.0
            active_indices_kkt = set()
            for m in range(M):
                if w_star[m] > unique_pt_tolerance:
                    active_indices_kkt.add(m)

            active_grads = grad_h[list(active_indices_kkt)] if active_indices_kkt else []
            grad_consistency_violation = 0.0
            if len(active_grads) > 1:
                grad_consistency_violation = np.max(active_grads) - np.min(active_grads)
                if not np.isclose(grad_consistency_violation, 0.0, atol=check_tol):
                     stationarity_violation = max(stationarity_violation, grad_consistency_violation)

            for m in range(M):
                if m not in active_indices_kkt:
                    nu_for_check = np.min(active_grads) if active_grads else max_grad_h
                    eta_m_estimate = nu_for_check - grad_h[m]
                    if eta_m_estimate < -check_tol:
                        stationarity_violation = max(stationarity_violation, abs(eta_m_estimate))

            kkt_violations.append({
                'alpha': alpha,
                'active_set_report': active_set_str,
                'active_set_kkt': str(tuple(sorted(active_indices_kkt))),
                'max_violation': stationarity_violation,
                'grad_consistency_violation': grad_consistency_violation,
                'check_result': 'OK' if stationarity_violation < check_tol * 10 else 'WARN'
            })

        if PANDAS_AVAILABLE and kkt_violations:
            df_kkt = pd.DataFrame(kkt_violations)
            kkt_cols_order = ['alpha', 'active_set_report','active_set_kkt', 'max_violation', 'grad_consistency_violation', 'check_result']
            kkt_cols_order = [col for col in kkt_cols_order if col in df_kkt.columns]
            with pd.option_context('display.float_format', '{:.4e}'.format, 'display.max_rows', None): # 全行表示
                print(df_kkt[kkt_cols_order].to_string(index=False, na_rep='NaN'))
            print("Notes on KKT Check:")
            print(" - active_set_report: Active set reported by inner solver")
            print(" - active_set_kkt: Active set inferred from w* > tol")
            print(" - max_violation: Max KKT violation (dual infeasibility or comp. slackness based on eta estimate)")
            print(" - grad_consistency_violation: max(active_grads) - min(active_grads) for w_m > tol")
        else:
            print("Could not perform KKT check or pandas not available.")

    elif isinstance(results_df, list):
        print("\n--- Analysis Results Summary (List) ---")
        # (表示省略)
        print("\n(Pandas not available, skipping CSV export)")

    end_time_main = time.time()
    print(f"\nTotal execution time: {end_time_main - start_time_main:.2f} seconds")

------------------------------------------------------------
--- Searching for Internal Solution around alpha=0.75 [0.700, 0.800] ---
--- (Using ActiveSet, mu_tilde=0.01, Symmetric Params) ---
------------------------------------------------------------

--- Starting Full Results Analysis over Alpha Range (using Active Set Method) ---
Analyzing alpha = 0.800000 (101/101) ... 
--- Finished Full Results Analysis ---

--- Analysis Results Summary (DataFrame) ---
     alpha     H_star active_set     w_fw_0     w_fw_1     w_fw_2       pi_0       pi_1       pi_2       pi_3        pi_4   grad_H_0   grad_H_1   grad_H_2   lambda_0   lambda_1   lambda_2
7.0000e-01 1.9765e-03     (0, 2) 9.9711e-01 1.4430e-03 1.4430e-03 1.4671e-01 1.2588e-01 1.5709e-02 4.1068e-03 -2.9241e-01 1.8765e-03 1.8765e-03 1.8765e-03 3.0587e-01 0.0000e+00 8.9429e-02
7.0100e-01 1.9765e-03     (0, 2) 9.9711e-01 1.4430e-03 1.4430e-03 1.4671e-01 1.2588e-01 1.5709e-02 4.1068e-03 -2.9241e-01 1.8765e-03 1.8765e-03 1.8765e-03 3.061

In [4]:
# -*- coding: utf-8 -*-
"""
4_14_find_internal_solution_quasi_symmetric.py

Attempts to find an internal solution w* by using quasi-symmetric parameters
(slight difference in Vm) around alpha=0.75, testing various mu_tilde values.
Uses the Active Set method for the inner QP solve.
"""

import numpy as np
import matplotlib.pyplot as plt
import warnings
from scipy.linalg import solve, LinAlgError, eigh, inv, qr
from scipy.optimize import linprog, OptimizeWarning
import traceback
import time
import itertools
import os
import math
from collections import Counter # オプション

# pandas がなければインストールを促す
try:
    import pandas as pd
    PANDAS_AVAILABLE = True
    pd.set_option('display.width', 160)
    pd.set_option('display.float_format', '{:.6e}'.format) # 科学技術表記
    pd.set_option('display.max_rows', 200) # 表示行数を増やす
except ImportError:
    PANDAS_AVAILABLE = False
    print("Warning: pandas library not found. Output formatting will be basic. CSV export disabled.")

# === ここから必要な関数定義 ===
DEFAULT_TOLERANCE = 1e-9

# --- OptimizationResult クラス ---
class OptimizationResult:
    def __init__(self, success, message, w_opt=None, pi_opt=None, lambda_opt=None,
                 H_opt=None, grad_H_opt=None, iterations=None, fw_gap=None,
                 active_set_opt=None):
        self.success = success; self.message = message; self.w_opt = w_opt; self.pi_opt = pi_opt
        self.lambda_opt = lambda_opt; self.H_opt = H_opt; self.grad_H_opt = grad_H_opt
        self.iterations = iterations; self.fw_gap = fw_gap
        self.active_set_opt = active_set_opt

# --- find_feasible_initial_pi 関数 ---
def find_feasible_initial_pi(R, mu_tilde, K, tolerance=1e-8):
    M = R.shape[1]; c = np.zeros(K + 1); c[K] = 1.0
    A_ub = np.hstack((-R.T, -np.ones((M, 1)))); b_ub = -mu_tilde * np.ones(M)
    bounds = [(None, None)] * K + [(0, None)]; opts = {'tol': tolerance, 'disp': False, 'presolve': True}
    result = None; methods_to_try = ['highs', 'highs-ipm', 'highs-ds', 'simplex']
    for method in methods_to_try:
        try:
            with warnings.catch_warnings(): warnings.filterwarnings("ignore"); result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method=method, options=opts)
            if result.success: break
        except ValueError: continue
        except Exception as e: return None, False, f"Phase 1 LP failed: {e}"
    if result is None or not result.success: msg = result.message if result else "No solver"; status = result.status if result else -1; return None, False, f"Phase 1 LP solver failed: {msg} (status={status})"
    s = result.x[K]; pi = result.x[:K]
    if np.isnan(pi).any(): return None, False, "Phase 1 LP resulted in NaN values for pi."
    if s <= tolerance * 1000:
        G = -R.T; h = -mu_tilde * np.ones(M); violation = np.max(G @ pi - h)
        if violation <= tolerance * 10000: return pi, True, f"Phase 1 OK (s*={s:.1e}, vio={violation:.1e})"
        else: return pi, True, f"Phase 1 OK (s*={s:.1e}), WARN: violation {violation:.1e}"
    else: return None, False, f"Phase 1 Infeasible (s* = {s:.1e})"

# --- solve_kkt_system 関数 ---
def solve_kkt_system(Q, G_W, g, tolerance=DEFAULT_TOLERANCE):
    K = Q.shape[0]; n_act = G_W.shape[0] if G_W is not None and G_W.ndim == 2 and G_W.shape[0] > 0 else 0
    if n_act == 0:
        try: p = solve(Q, -g, assume_a='sym'); l = np.array([])
        except LinAlgError: return None, None, False
        res_norm = np.linalg.norm(Q @ p + g); g_norm = np.linalg.norm(g); solved_ok = res_norm <= tolerance * 1e3 * (1 + g_norm)
        return p, l, solved_ok
    else:
        kkt_mat = None; rhs = None
        try: kkt_mat = np.block([[Q, G_W.T], [G_W, np.zeros((n_act, n_act))]]) ; rhs = np.concatenate([-g, np.zeros(n_act)])
        except ValueError as e: return None, None, False
        try: sol = solve(kkt_mat, rhs, assume_a='sym'); p = sol[:K]; l = sol[K:]
        except LinAlgError: return None, None, False
        except ValueError as e: return None, None, False
        res_norm = np.linalg.norm(kkt_mat @ sol - rhs); rhs_norm = np.linalg.norm(rhs); solved_ok = res_norm <= tolerance * 1e3 * (1 + rhs_norm)
        return p, l, solved_ok

# --- solve_inner_qp_active_set 関数 ---
def solve_inner_qp_active_set(Vw, R, mu_tilde, initial_pi, max_iter=350, tolerance=DEFAULT_TOLERANCE, regularization_epsilon=1e-10):
    K = Vw.shape[0]; M = R.shape[1]; Q_reg = 2 * Vw + 2 * regularization_epsilon * np.eye(K); G = -R.T; h = -mu_tilde * np.ones(M)
    if initial_pi is None: return None, None, None, None, None, False, "No initial pi"
    pi_k = np.copy(initial_pi); lam_opt = np.zeros(M); W = set()
    active_tol = tolerance * 10
    initial_violations = G @ pi_k - h; W = set(j for j, viol in enumerate(initial_violations) if viol > -active_tol)
    active_indices_opt = None; kkt_matrix_opt = None
    if np.any(initial_violations > active_tol * 10): warnings.warn(f"Initial pi infeasible (max viol: {np.max(initial_violations):.2e}).")

    for i in range(max_iter):
        g_k = Q_reg @ pi_k; act = sorted(list(W)); n_act = len(act); G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))
        p_k, lam_Wk, solved = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)
        if not solved or p_k is None: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: KKT solve failed. ActiveSet={act}"
        if np.linalg.norm(p_k) <= tolerance * 10 * (1 + np.linalg.norm(pi_k)):
            is_optimal_point = True; blocking_constraint_idx = -1; min_negative_lambda = float('inf')
            dual_feas_tol = -tolerance * 10
            if W:
                if lam_Wk is None or len(lam_Wk) != n_act: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: lam_Wk inconsistent? ActiveSet={act}"
                lambda_map = dict(zip(act, lam_Wk))
                for constraint_idx, lagrange_multiplier in lambda_map.items():
                    if lagrange_multiplier < dual_feas_tol: is_optimal_point = False;
                    if lagrange_multiplier < min_negative_lambda: min_negative_lambda = lagrange_multiplier; blocking_constraint_idx = constraint_idx
            if is_optimal_point:
                lam_opt.fill(0.0);
                if W and len(lam_Wk) == n_act: lam_opt[act] = np.maximum(lam_Wk, 0)
                final_infeas = np.max(G @ pi_k - h); msg = f"Optimal found at iter {i+1}."
                if final_infeas > active_tol: msg += f" (WARN: violation {final_infeas:.1e})"
                active_indices_opt = act
                return pi_k, lam_opt, active_indices_opt, None, Q_reg / 2.0, True, msg
            else:
                if blocking_constraint_idx in W: W.remove(blocking_constraint_idx); continue
                else: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: Neg lambda idx {blocking_constraint_idx}, not in W={act}"
        else:
            alpha_k = 1.0; blocking_constraint_idx = -1; min_step_length = float('inf')
            step_tol = tolerance * 10
            for j in range(M):
                if j not in W:
                    constraint_gradient_dot_p = G[j, :] @ p_k
                    if constraint_gradient_dot_p > step_tol:
                        distance_to_boundary = h[j] - (G[j, :] @ pi_k)
                        if abs(constraint_gradient_dot_p) > 1e-15:
                            alpha_j = distance_to_boundary / constraint_gradient_dot_p
                            step_j = max(0.0, alpha_j)
                            if step_j < min_step_length:
                                min_step_length = step_j; blocking_constraint_idx = j
            alpha_k = min(1.0, min_step_length); pi_k += alpha_k * p_k
            if alpha_k < 1.0 - step_tol and blocking_constraint_idx != -1:
                if blocking_constraint_idx not in W: W.add(blocking_constraint_idx)
            continue
    msg = f"Max iter ({max_iter}) reached."; final_infeas = np.max(G @ pi_k - h)
    if final_infeas > active_tol * 10: return None, None, None, None, None, False, f"{msg} Final infeasible. ActiveSet={sorted(list(W))}"
    act = sorted(list(W)); n_act = len(act); G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))
    is_likely_optimal = False; active_constraints_opt = act
    g_k = Q_reg @ pi_k; p_f, lam_f, solved_f = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)
    final_lambda_estimate = np.zeros(M)
    if solved_f and p_f is not None and np.linalg.norm(p_f) <= tolerance * 100 * (1 + np.linalg.norm(pi_k)):
        if n_act > 0 and lam_f is not None and len(lam_f) == n_act:
             try: final_lambda_estimate[act] = lam_f
             except IndexError: pass
        active_lambdas = final_lambda_estimate[act] if n_act > 0 else np.array([])
        if n_act == 0 or np.all(active_lambdas >= -tolerance * 100): is_likely_optimal = True; msg += " Final KKT check approx OK."
        else: msg += " Final KKT check fails (dual infeasible)."
    else: msg += " Final KKT check fails (stationarity or solve error)."
    lam_opt = final_lambda_estimate
    return pi_k, lam_opt, active_constraints_opt, None, Q_reg / 2.0, is_likely_optimal, msg

# --- make_psd 関数 ---
def make_psd(matrix, tolerance=1e-8):
    sym = (matrix + matrix.T) / 2
    try: eigenvalues, eigenvectors = np.linalg.eigh(sym); min_eigenvalue = np.min(eigenvalues)
    except LinAlgError: warnings.warn("..."); return sym
    if min_eigenvalue < tolerance: eigenvalues[eigenvalues < tolerance] = tolerance; psd_matrix = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T; return (psd_matrix + psd_matrix.T) / 2
    else: return sym

# --- calculate_Vw 関数 ---
def calculate_Vw(w, R, SecondMoments_a_array, tolerance=DEFAULT_TOLERANCE, psd_tolerance=1e-8):
    K, M = R.shape; w_sum = np.sum(w); w_norm = w
    EwX = R @ w_norm; EwXXT = np.zeros((K, K));
    for m in range(M): EwXXT += w_norm[m] * SecondMoments_a_array[m]
    Vw = EwXXT - np.outer(EwX, EwX); Vw_psd = make_psd(Vw, psd_tolerance)
    return Vw_psd, EwX, EwXXT

# --- calculate_H_gradient 関数 (ノルムチェック緩和) ---
def calculate_H_gradient(pi_star, w, R, SecondMoments_a_array, EwX, EwXXT, tolerance=1e-9, debug_print=False):
    M = w.shape[0]; K = R.shape[0]; grad = np.zeros(M)
    norm_tolerance = 1e-12 # Allow very small pi_star

    if pi_star is None or EwX is None:
        if debug_print: print("DEBUG grad_H: Returning NaN due to None input (pi_star or EwX).")
        return np.full(M, np.nan)
    if np.isnan(pi_star).any() or np.isinf(pi_star).any():
         if debug_print: print("DEBUG grad_H: Returning NaN because pi_star contains NaN/Inf.")
         return np.full(M, np.nan)
    pi_norm = np.linalg.norm(pi_star)
    if pi_norm < norm_tolerance:
         if debug_print: print(f"DEBUG grad_H: Returning NaN because pi_star norm {pi_norm:.2e} < {norm_tolerance:.1e}")
         return np.full(M, np.nan)
    if np.isnan(EwX).any() or np.isinf(EwX).any():
        if debug_print: print("DEBUG grad_H: Returning NaN because EwX contains NaN/Inf.")
        return np.full(M, np.nan)

    try:
        pi_T_EwX = pi_star.T @ EwX
        if np.isnan(pi_T_EwX) or np.isinf(pi_T_EwX):
             if debug_print: print(f"DEBUG grad_H: pi_T_EwX is NaN/Inf: {pi_T_EwX}")
             return np.full(M, np.nan)

        for j in range(M):
            Sigma_j = SecondMoments_a_array[j]; r_j = R[:, j]
            pi_T_Sigma_j_pi = pi_star.T @ Sigma_j @ pi_star
            pi_T_r_j = pi_star.T @ r_j

            if np.isnan(pi_T_Sigma_j_pi) or np.isinf(pi_T_Sigma_j_pi):
                 if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_Sigma_j_pi is NaN/Inf: {pi_T_Sigma_j_pi}")
                 grad[j] = np.nan; continue
            if np.isnan(pi_T_r_j) or np.isinf(pi_T_r_j):
                 if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_r_j is NaN/Inf: {pi_T_r_j}")
                 grad[j] = np.nan; continue

            term2 = 2 * pi_T_r_j * pi_T_EwX
            if np.isnan(term2) or np.isinf(term2):
                 if debug_print: print(f"DEBUG grad_H (j={j}): term2 (2*pi_T_r_j*pi_T_EwX) is NaN/Inf: {term2}, pi_T_r_j={pi_T_r_j}, pi_T_EwX={pi_T_EwX}")
                 grad[j] = np.nan; continue

            grad[j] = pi_T_Sigma_j_pi - term2
            if np.isnan(grad[j]) or np.isinf(grad[j]):
                 if debug_print: print(f"DEBUG grad_H (j={j}): final grad[{j}] is NaN/Inf")

    except Exception as e_calc:
         print(f"ERROR in calculate_H_gradient calculation: {e_calc}")
         print(traceback.format_exc())
         return np.full(M, np.nan)

    if np.any(np.isnan(grad)) or np.any(np.isinf(grad)):
        warnings.warn(f"NaN or Inf detected in calculated gradient for w={w}")
        if debug_print: print(f"DEBUG grad_H: Final check found NaN/Inf in gradient: {grad}")
        return np.full(M, np.nan)
    return grad

# --- project_to_simplex 関数 ---
def project_to_simplex(v, z=1):
    n_features = v.shape[0];
    if n_features == 0: return np.array([])
    v_arr = np.asarray(v)
    if np.all(v_arr >= -1e-9) and np.isclose(np.sum(v_arr), z): return np.maximum(v_arr, 0)
    u = np.sort(v_arr)[::-1]; cssv = np.cumsum(u) - z; ind = np.arange(n_features) + 1; cond = u - cssv / ind > 0
    if np.any(cond): rho = ind[cond][-1]; theta = cssv[rho - 1] / float(rho); w = np.maximum(v_arr - theta, 0)
    else:
         w = np.zeros(n_features)
         if z > 0: w[np.argmax(v_arr)] = z
    w_sum = np.sum(w)
    if not np.isclose(w_sum, z):
        if w_sum > 1e-9: w = w * (z / w_sum)
        elif z > 0 :
            w = np.zeros(n_features)
            w[np.argmax(v_arr)] = z
    return np.maximum(w, 0)

# --- frank_wolfe_optimizer 関数 (ActiveSet法を使用、最終解の整合性を高める修正済み) ---
def frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde, initial_w=None, max_outer_iter=250, fw_gap_tol=1e-7, inner_max_iter=350, tolerance=1e-9, psd_make_tolerance=1e-8, qp_regularization=1e-10, debug_print=False, force_iterations=0, return_history=False):
    K, M = R_alpha.shape
    if initial_w is None: w_k = np.ones(M) / M
    else: w_k = project_to_simplex(np.copy(initial_w))
    if w_k is None:
        result = OptimizationResult(False, "Initial projection failed", w_opt=initial_w)
        return (result, []) if return_history else result
    fw_gap = float('inf'); best_w = np.copy(w_k); best_pi = None; best_lam = np.zeros(M); best_H = -float('inf'); final_gHk = np.zeros(M); best_active_set = None
    pi0, ok, p1msg = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance)
    if not ok:
        result = OptimizationResult(False, f"Phase 1 failed: {p1msg}", w_opt=initial_w)
        return (result, []) if return_history else result
    current_w=np.copy(w_k); current_H=-float('inf'); current_pi=None; current_lam=None; current_active_set=None
    inner_solver_args_for_loop = {'max_iter': inner_max_iter, 'tolerance': tolerance, 'regularization_epsilon': qp_regularization}
    history = []

    last_successful_pi = pi0
    last_successful_lam = np.zeros(M)
    last_successful_active_set = tuple()
    last_successful_gHk = np.zeros(M)
    last_successful_fw_gap = float('inf')

    for k in range(max_outer_iter):
        iter_data = {'k': k + 1}
        if return_history: iter_data['w_k'] = np.copy(w_k)
        try: Vk, Ex, ExxT = calculate_Vw(w_k, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
        except Exception as e:
            result = OptimizationResult(False, f"Outer iter {k+1}: Vw failed: {e}", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, active_set_opt=best_active_set)
            if return_history: history.append(iter_data); return (result, history)
            else: return result

        # === Use Active Set for inner solve ===
        pi_init_inner = last_successful_pi if last_successful_pi is not None else pi0
        pk, lk, act_idx_k, _, Vw_reg_k, inner_ok, inner_msg = solve_inner_qp_active_set(
            Vk, R_alpha, mu_tilde, pi_init_inner, **inner_solver_args_for_loop
        )
        # ===================================

        if not inner_ok or pk is None:
            warnings.warn(f"Outer iter {k+1}: Inner QP failed: {inner_msg}. Using last successful state if available.")
            if last_successful_pi is None:
                 result = OptimizationResult(False, f"Outer iter {k+1}: Inner QP failed and no prior success: {inner_msg}",
                                             w_opt=current_w, iterations=k)
                 if return_history: history.append(iter_data); return (result, history)
                 else: return result
            pk = last_successful_pi
            lk = last_successful_lam
            act_idx_k = list(last_successful_active_set)
            final_gHk = last_successful_gHk
            fw_gap = last_successful_fw_gap
            try: Hk = pk.T @ Vk @ pk
            except: Hk = current_H
        else:
            last_successful_pi = pk
            last_successful_lam = lk if lk is not None else np.zeros(M)
            last_successful_active_set = tuple(sorted(act_idx_k)) if act_idx_k is not None else tuple()
            Hk = pk.T @ Vk @ pk
            try:
                gHk = calculate_H_gradient(pk, w_k, R_alpha, SecondMoments_alpha_array, Ex, ExxT, tolerance, debug_print=debug_print)
            except Exception as e:
                result = OptimizationResult(False, f"Outer iter {k+1}: Grad failed: {e}", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
                if return_history: history.append(iter_data); return (result, history)
                else: return result

            final_gHk = gHk if gHk is not None else np.full(M, np.nan)
            last_successful_gHk = final_gHk if not np.isnan(final_gHk).any() else last_successful_gHk

        if return_history: iter_data['H_k'] = Hk
        if return_history: iter_data['grad_H_k_norm'] = np.linalg.norm(final_gHk) if not np.isnan(final_gHk).any() else np.nan

        current_w = np.copy(w_k); current_H = Hk; current_pi = pk; current_lam = last_successful_lam; current_active_set = last_successful_active_set

        if Hk >= best_H - tolerance*1000:
             best_H = Hk; best_w = np.copy(w_k); best_pi = np.copy(pk); best_lam = np.copy(current_lam); best_active_set = current_active_set

        if final_gHk is None or np.isnan(final_gHk).any():
            result = OptimizationResult(False, f"Outer iter {k+1}: Grad NaN.", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
            if return_history: history.append(iter_data); return (result, history)
            else: return result

        grad_norm = np.linalg.norm(final_gHk)
        if grad_norm < tolerance * 10: sk = w_k; sk_idx = -1
        else: sk_idx = np.argmax(final_gHk); sk = np.zeros(M); sk[sk_idx] = 1.0
        if return_history: iter_data['s_k_index'] = sk_idx

        if w_k is None:
             result = OptimizationResult(False, f"k={k} w_k None", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
             if return_history: history.append(iter_data); return (result, history)
             else: return result

        fw_gap = final_gHk.T @ (w_k - sk)
        last_successful_fw_gap = fw_gap
        if return_history: iter_data['fw_gap'] = fw_gap
        gamma = 2.0 / (k + 3.0)
        if return_history: iter_data['gamma_k'] = gamma
        if return_history: history.append(iter_data)

        converged = False
        if k >= force_iterations and not np.isnan(fw_gap):
            if abs(fw_gap) <= fw_gap_tol: converged = True; conv_msg = f"Converged (Gap {abs(fw_gap):.2e})"
        if converged:
            # --- Final Inner Solve for Consistency ---
            final_w = current_w
            final_pi_result = current_pi
            final_lam_result = current_lam
            final_active_set_result = current_active_set
            final_H_result = current_H
            final_grad_result = final_gHk
            final_fw_gap_result = fw_gap

            try:
                final_Vk, final_Ex, final_ExxT = calculate_Vw(final_w, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
                pi0_final, ok_final, _ = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance)
                if ok_final:
                    pi_s_final, lam_s_final, act_idx_final, _, _, inner_ok_final, msg_final = solve_inner_qp_active_set(
                        final_Vk, R_alpha, mu_tilde, pi0_final, # Use stable pi0 for final check
                        **inner_solver_args_for_loop
                    )
                    if inner_ok_final and pi_s_final is not None and lam_s_final is not None:
                        final_pi_result = pi_s_final
                        final_lam_result = lam_s_final
                        final_active_set_result = tuple(sorted(act_idx_final)) if act_idx_final is not None else tuple()
                        final_grad_result = calculate_H_gradient(final_pi_result, final_w, R_alpha, SecondMoments_alpha_array, final_Ex, final_ExxT, tolerance, debug_print=False)
                        if final_grad_result is None or np.isnan(final_grad_result).any(): final_grad_result = np.zeros(M)
                        grad_norm_final = np.linalg.norm(final_grad_result) if not np.isnan(final_grad_result).any() else np.nan
                        if grad_norm_final < tolerance * 10: sk_final = final_w
                        else: sk_idx_final = np.argmax(final_grad_result); sk_final = np.zeros(M); sk_final[sk_idx_final] = 1.0
                        final_fw_gap_result = final_grad_result.T @ (final_w - sk_final) if not np.isnan(final_grad_result).any() else np.nan
                    else:
                        warnings.warn(f"Warning: Final inner QP solve failed after convergence: {msg_final}. Using last iteration's values.")
                else:
                     warnings.warn(f"Warning: Could not find feasible pi0 for final solve after convergence. Using last iteration's values.")
            except Exception as e:
                warnings.warn(f"Error during final inner solve after convergence: {e}. Using last iteration's values.")
            # --- End of Final Inner Solve ---

            result = OptimizationResult(True, conv_msg, w_opt=final_w, pi_opt=final_pi_result, lambda_opt=final_lam_result,
                                        H_opt=final_H_result, grad_H_opt=final_grad_result, iterations=k + 1,
                                        fw_gap=final_fw_gap_result, active_set_opt=final_active_set_result)
            return (result, history) if return_history else result

        # --- Prepare for next iteration ---
        if w_k is None or sk is None:
             result = OptimizationResult(False, f"k={k} w_k/sk None before update", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
             return (result, history) if return_history else result
        w_k_next = (1.0 - gamma) * w_k + gamma * sk; w_k = project_to_simplex(w_k_next)
        if w_k is None:
             result = OptimizationResult(False, f"k={k} proj None", w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, active_set_opt=best_active_set)
             return (result, history) if return_history else result
        pi0 = pk # Use last inner solution as initial guess

    # Max iter reached, perform final solve for consistency
    final_w = current_w
    final_pi_result = current_pi
    final_lam_result = current_lam
    final_active_set_result = current_active_set
    final_H_result = current_H
    final_grad_result = last_successful_gHk
    final_fw_gap_result = last_successful_fw_gap

    try:
        final_Vk, final_Ex, final_ExxT = calculate_Vw(final_w, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
        pi0_final, ok_final, _ = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance) # Use stable pi0
        if ok_final:
            pi_s_final, lam_s_final, act_idx_final, _, _, inner_ok_final, msg_final = solve_inner_qp_active_set(
                final_Vk, R_alpha, mu_tilde, pi0_final, # <-- Use stable pi0_final
                **inner_solver_args_for_loop
            )
            if inner_ok_final and pi_s_final is not None and lam_s_final is not None:
                final_pi_result = pi_s_final
                final_lam_result = lam_s_final
                final_active_set_result = tuple(sorted(act_idx_final)) if act_idx_final is not None else tuple()
                # Recalculate gradient and gap
                final_grad_result = calculate_H_gradient(final_pi_result, final_w, R_alpha, SecondMoments_alpha_array, final_Ex, final_ExxT, tolerance, debug_print=False)
                if final_grad_result is None or np.isnan(final_grad_result).any(): final_grad_result = np.zeros(M)
                grad_norm_final = np.linalg.norm(final_grad_result) if not np.isnan(final_grad_result).any() else np.nan
                if grad_norm_final < tolerance * 10: sk_final = final_w
                else: sk_idx_final = np.argmax(final_grad_result); sk_final = np.zeros(M); sk_final[sk_idx_final] = 1.0
                final_fw_gap_result = final_grad_result.T @ (final_w - sk_final) if not np.isnan(final_grad_result).any() else np.nan
            else:
                warnings.warn(f"Warning: Final inner QP solve failed after max iter: {msg_final}. Using last iteration's values.")
        else:
             warnings.warn(f"Warning: Could not find feasible pi0 for final solve after max iter. Using last iteration's values.")
    except Exception as e:
        warnings.warn(f"Error during final inner solve after max iter: {e}. Using last iteration's values.")

    if return_history:
        grad_norm_final = np.linalg.norm(final_grad_result) if not np.isnan(final_grad_result).any() else np.nan
        if grad_norm_final < tolerance * 10: sk_idx_final = -1
        else: sk_idx_final = np.argmax(final_grad_result) if not np.isnan(final_grad_result).any() else -1
        history.append({
            'k': max_outer_iter + 1, 'w_k': np.copy(final_w), 'H_k': final_H_result,
            'grad_H_k_norm': grad_norm_final, 's_k_index': sk_idx_final, 'fw_gap': final_fw_gap_result,
            'gamma_k': np.nan
        })

    result = OptimizationResult(True, f"Max Iter ({max_outer_iter})", w_opt=final_w, pi_opt=final_pi_result, lambda_opt=final_lam_result,
                                H_opt=final_H_result, grad_H_opt=final_grad_result, iterations=max_outer_iter,
                                fw_gap=final_fw_gap_result, active_set_opt=final_active_set_result)
    return (result, history) if return_history else result


# --- generate_params_profile_switching_symmetric 関数 (対称 + Vm微小差) ---
def generate_params_profile_switching_symmetric(alpha, alpha_max, K=5, M=3,
                                      R_base_sym=np.array([0.02, 0.01, 0.0, -0.01, -0.02]),
                                      sigma_base=np.array([0.18, 0.15, 0.20, 0.12, 0.10]),
                                      Corr_base=np.array([[1.0, 0.4, 0.3, 0.2, 0.1], [0.4, 1.0, 0.5, 0.3, 0.2], [0.3, 0.5, 1.0, 0.6, 0.4], [0.2, 0.3, 0.6, 1.0, 0.5], [0.1, 0.2, 0.4, 0.5, 1.0]]),
                                      r_offset=0.03,
                                      # ★★★ Vm にわずかな差をつける ★★★
                                      s_factor_g = 0.001, # good/neutral vs base
                                      s_factor_b = -0.001, # bad vs base
                                      # ★★★★★★★★★★★★★★★★★★★★★
                                      sigma_min_epsilon=1e-4, psd_tolerance=1e-9):
    """ Generates symmetric parameters with slight differences in Vm """
    assert K == len(R_base_sym) and K == len(sigma_base) and K == Corr_base.shape[0] and M == 3, "Dimension mismatch"

    R_neutral = R_base_sym
    R_good = R_base_sym + r_offset
    R_bad = R_base_sym - r_offset

    # Base covariance
    sigma_neutral = np.maximum(sigma_min_epsilon, sigma_base)
    Corr_neutral = make_psd(Corr_base, psd_tolerance)
    V_base = np.diag(sigma_neutral) @ Corr_neutral @ np.diag(sigma_neutral)

    # Covariances with slight differences
    sigma_good_adj = np.maximum(sigma_min_epsilon, sigma_base * (1 + s_factor_g))
    V_good = np.diag(sigma_good_adj) @ Corr_neutral @ np.diag(sigma_good_adj) # Assume same Corr for simplicity

    sigma_bad_adj = np.maximum(sigma_min_epsilon, sigma_base * (1 + s_factor_b))
    V_bad = np.diag(sigma_bad_adj) @ Corr_neutral @ np.diag(sigma_bad_adj) # Assume same Corr for simplicity

    V_neutral = V_base # V^N = V_base

    # Calculate Sigma^m = V^m + r^m * r^m.T
    Sigma_neutral = V_neutral + np.outer(R_neutral, R_neutral)
    Sigma_good = V_good + np.outer(R_good, R_good)
    Sigma_bad = V_bad + np.outer(R_bad, R_bad)

    # Apply profile switching based on alpha
    beta = np.clip(alpha / alpha_max if alpha_max > 0 else (1.0 if alpha > 0 else 0.0), 0.0, 1.0)

    R_alpha = np.zeros((K, M))
    SecondMoments_a_array = np.zeros((M, K, K))

    # Scenario 0: Mix Good and Neutral
    R_alpha[:, 0] = (1 - beta) * R_good + beta * R_neutral
    SecondMoments_a_array[0, :, :] = (1 - beta) * Sigma_good + beta * Sigma_neutral

    # Scenario 1: Mix Bad and Good
    R_alpha[:, 1] = (1 - beta) * R_bad + beta * R_good
    SecondMoments_a_array[1, :, :] = (1 - beta) * Sigma_bad + beta * Sigma_good

    # Scenario 2: Mix Neutral and Bad
    R_alpha[:, 2] = (1 - beta) * R_neutral + beta * R_bad
    SecondMoments_a_array[2, :, :] = (1 - beta) * Sigma_neutral + beta * Sigma_bad

    return R_alpha, SecondMoments_a_array


# === α依存性分析関数 (ActiveSet法を使用) ===
def analyze_alpha_full_results(alpha_range, param_gen_kwargs, optimizer_kwargs, mu_tilde):
    """ 指定された alpha 範囲で FW解、H*、pi*、grad H*、lambda*、active_set* を追跡する """
    results_over_alpha = []
    K = param_gen_kwargs.get('K', 5); M = param_gen_kwargs.get('M', 3)

    print("\n--- Starting Full Results Analysis over Alpha Range (using Active Set Method) ---")
    total_alphas = len(alpha_range)
    start_loop_time = time.time()

    for idx, alpha in enumerate(alpha_range):
        loop_start_time = time.time()
        print(f"\rAnalyzing alpha = {alpha:.6f} ({idx+1}/{total_alphas}) ... ", end="")

        # === パラメータ生成 ===
        # ★★★ 対称(微小差分)パラメータ生成関数を呼び出す ★★★
        R_alpha, SecondMoments_alpha_array = generate_params_profile_switching_symmetric(alpha, **param_gen_kwargs)
        alpha_result = {'alpha': alpha}

        w_fw_default = np.full(M, np.nan)
        H_star_fw = np.nan
        pi_opt = np.full(K, np.nan)
        grad_H_opt = np.full(M, np.nan)
        lambda_opt = np.full(M, np.nan)
        active_set_opt = tuple()

        try:
            # --- FW Optimizer呼び出し (Active Setを使用) ---
            fw_result = frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde,
                                              initial_w=None, return_history=False,
                                              debug_print=False, # デバッグOFF
                                              **optimizer_kwargs)
            if fw_result.success:
                 w_fw_default = fw_result.w_opt if fw_result.w_opt is not None else np.full(M, np.nan)
                 H_star_fw = fw_result.H_opt if fw_result.H_opt is not None else np.nan
                 pi_opt = fw_result.pi_opt if fw_result.pi_opt is not None else np.full(K, np.nan)
                 grad_H_opt = fw_result.grad_H_opt if fw_result.grad_H_opt is not None else np.full(M, np.nan)
                 lambda_opt = fw_result.lambda_opt if fw_result.lambda_opt is not None else np.full(M, np.nan)
                 active_set_opt = fw_result.active_set_opt if fw_result.active_set_opt is not None else tuple()
            else:
                 w_fw_default = np.full(M, np.nan)
                 H_star_fw = np.nan
                 pi_opt = np.full(K, np.nan)
                 grad_H_opt = np.full(M, np.nan)
                 lambda_opt = np.full(M, np.nan)
                 active_set_opt = tuple()
                 print(f"\n  Warn: FW failed for alpha={alpha:.6f}: {fw_result.message}")

        except Exception as e:
            print(f"\n  Error during FW run for alpha={alpha:.6f}: {e}")
            w_fw_default = np.full(M, np.nan)
            H_star_fw = np.nan
            pi_opt = np.full(K, np.nan)
            grad_H_opt = np.full(M, np.nan)
            lambda_opt = np.full(M, np.nan)
            active_set_opt = tuple()

        alpha_result['w_fw_default'] = w_fw_default
        alpha_result['H_star'] = H_star_fw
        alpha_result['pi_opt'] = pi_opt
        alpha_result['grad_H_opt'] = grad_H_opt
        alpha_result['lambda_opt'] = lambda_opt
        alpha_result['active_set_opt'] = active_set_opt

        results_over_alpha.append(alpha_result)

    print("\n--- Finished Full Results Analysis ---")
    df_results = pd.DataFrame(results_over_alpha)
    if PANDAS_AVAILABLE:
        fw_vecs = np.stack([res.get('w_fw_default', np.full(M, np.nan)) for res in results_over_alpha])
        pi_vecs = np.stack([res.get('pi_opt', np.full(K, np.nan)) for res in results_over_alpha])
        grad_vecs = np.stack([res.get('grad_H_opt', np.full(M, np.nan)) for res in results_over_alpha])
        lambda_vecs = np.stack([res.get('lambda_opt', np.full(M, np.nan)) for res in results_over_alpha])
        active_sets = [str(res.get('active_set_opt', tuple())) for res in results_over_alpha]

        for m in range(M):
            df_results[f'w_fw_{m}'] = fw_vecs[:, m]
            df_results[f'grad_H_{m}'] = grad_vecs[:, m]
            df_results[f'lambda_{m}'] = lambda_vecs[:, m]
        for k in range(K):
            df_results[f'pi_{k}'] = pi_vecs[:, k]

        df_results['active_set'] = active_sets

        df_results = df_results.drop(columns=['w_fw_default', 'pi_opt', 'grad_H_opt', 'lambda_opt', 'active_set_opt'], errors='ignore')

    return df_results if PANDAS_AVAILABLE else results_over_alpha


# === メイン実行ブロック ===
if __name__ == '__main__':
    start_time_main = time.time()
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    warnings.filterwarnings('ignore', category=OptimizeWarning)

    # --- 設定 ---
    K = 5; M = 3;
    # ★★★ mu_tilde を 0.01 に設定 ★★★
    mu_tilde = 0.01
    alpha_max = 1.5

    # ★★★ 対称性を少し崩したパラメータ設定 ★★★
    param_gen_kwargs_symmetric_mild = {
        'K': K, 'M': M, 'alpha_max': alpha_max,
        'R_base_sym': np.array([0.02, 0.01, 0.0, -0.01, -0.02]), # Neutral=0中心
        'sigma_base': np.array([0.18, 0.15, 0.20, 0.12, 0.10]),
        'Corr_base': np.array([[1.0, 0.4, 0.3, 0.2, 0.1], [0.4, 1.0, 0.5, 0.3, 0.2], [0.3, 0.5, 1.0, 0.6, 0.4], [0.2, 0.3, 0.6, 1.0, 0.5], [0.1, 0.2, 0.4, 0.5, 1.0]]),
        'r_offset': 0.03,
        's_factor_g': 0.001,  # Vm にわずかな差をつける
        's_factor_b': -0.001, # Vm にわずかな差をつける
        'sigma_min_epsilon': 1e-4,
        'psd_tolerance': 1e-9
    }

    solver_settings = { # 精度設定
        'max_outer_iter': 500, 'fw_gap_tol': 1e-9,
        'inner_max_iter': 600,
        'tolerance': 1e-11,
        'psd_make_tolerance': 1e-9,
        'qp_regularization': 1e-10, # 元の正則化値
        'force_iterations': 20
    }
    # ★★★ 分析する alpha の範囲 (0.75 付近) ★★★
    alpha_start = 0.700
    alpha_end = 0.800
    alpha_step = 0.001
    num_alpha_steps = int(round((alpha_end - alpha_start) / alpha_step)) + 1
    alpha_range_analyze = np.linspace(alpha_start, alpha_end, num_alpha_steps)
    unique_pt_tolerance = 1e-5 # KKTチェック用

    output_csv_filename = f"alpha_internal_sol_search_{alpha_start:.3f}_{alpha_end:.3f}_activeset_mu001_quasi_sym.csv" # ファイル名変更

    print("-" * 60)
    print(f"--- Searching for Internal Solution around alpha=0.75 [{alpha_start:.3f}, {alpha_end:.3f}] ---")
    print(f"--- (Using ActiveSet, mu_tilde={mu_tilde}, Quasi-Symmetric Params) ---")
    print("-" * 60)

    # --- 分析実行 ---
    results_df = analyze_alpha_full_results(
        alpha_range=alpha_range_analyze,
        param_gen_kwargs=param_gen_kwargs_symmetric_mild, # <-- 対称(微小差分)設定を使用
        optimizer_kwargs=solver_settings,
        mu_tilde=mu_tilde
    )

    # --- 結果表示 & CSV保存 & KKTチェック ---
    if PANDAS_AVAILABLE and isinstance(results_df, pd.DataFrame):
        print("\n--- Analysis Results Summary (DataFrame) ---")
        float_format_func = lambda x: f"{x:.4e}" if pd.notna(x) and isinstance(x, (float, np.number)) else x
        pd.options.display.float_format = float_format_func
        cols_to_show = ['alpha', 'H_star', 'active_set'] + \
                       [f'w_fw_{m}' for m in range(M) if f'w_fw_{m}' in results_df.columns] + \
                       [f'pi_{k}' for k in range(K) if f'pi_{k}' in results_df.columns] + \
                       [f'grad_H_{m}' for m in range(M) if f'grad_H_{m}' in results_df.columns] + \
                       [f'lambda_{m}' for m in range(M) if f'lambda_{m}' in results_df.columns]
        cols_to_show = [col for col in cols_to_show if col in results_df.columns]
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 200): # 全行表示
             print(results_df[cols_to_show].to_string(index=False, na_rep='NaN'))
        pd.reset_option('display.float_format')

        # CSVファイルに保存 (フィルタリングなし、全データ)
        try:
            df_to_save = results_df.copy()
            cols_order = ['alpha', 'H_star', 'active_set'] + \
                         [f'w_fw_{m}' for m in range(M) if f'w_fw_{m}' in df_to_save.columns] + \
                         [f'pi_{k}' for k in range(K) if f'pi_{k}' in df_to_save.columns] + \
                         [f'grad_H_{m}' for m in range(M) if f'grad_H_{m}' in df_to_save.columns] + \
                         [f'lambda_{m}' for m in range(M) if f'lambda_{m}' in df_to_save.columns]
            cols_order = [col for col in cols_order if col in df_to_save.columns]
            df_to_save = df_to_save[cols_order]

            df_to_save.to_csv(output_csv_filename, index=False, float_format='%.8e')
            print(f"\nFull detailed results ({len(df_to_save)} points) saved to: {os.path.abspath(output_csv_filename)}")
        except Exception as e:
            print(f"\nError saving results to CSV: {e}")

        # KKT条件のチェック (簡易版)
        print("\n--- Quick KKT Check (Stationarity w.r.t. w) ---")
        kkt_violations = []
        # unique_pt_tolerance は main スコープで定義済み
        for index, row in results_df.iterrows():
            alpha = row['alpha']
            w_star = np.array([row.get(f'w_fw_{m}', np.nan) for m in range(M)])
            grad_h = np.array([row.get(f'grad_H_{m}', np.nan) for m in range(M)])
            active_set_str = row.get('active_set', '()')

            if np.isnan(w_star).any() or np.isnan(grad_h).any() or np.isclose(np.sum(w_star), 0):
                kkt_violations.append({
                    'alpha': alpha,
                    'active_set_report': active_set_str,
                    'active_set_kkt': 'NaN',
                    'max_violation': np.nan,
                    'grad_consistency_violation': np.nan,
                    'check_result': 'Skipped (NaN or zero w)'
                })
                continue

            check_tol = solver_settings['tolerance'] * 1e3

            max_grad_h = np.max(grad_h)
            nu_estimate = max_grad_h

            stationarity_violation = 0.0
            active_indices_kkt = set()
            for m in range(M):
                if w_star[m] > unique_pt_tolerance:
                    active_indices_kkt.add(m)

            active_grads = grad_h[list(active_indices_kkt)] if active_indices_kkt else []
            grad_consistency_violation = 0.0
            if len(active_grads) > 1:
                grad_consistency_violation = np.max(active_grads) - np.min(active_grads)
                if not np.isclose(grad_consistency_violation, 0.0, atol=check_tol):
                     stationarity_violation = max(stationarity_violation, grad_consistency_violation)

            for m in range(M):
                if m not in active_indices_kkt:
                    nu_for_check = np.min(active_grads) if active_grads else max_grad_h
                    eta_m_estimate = nu_for_check - grad_h[m]
                    if eta_m_estimate < -check_tol:
                        stationarity_violation = max(stationarity_violation, abs(eta_m_estimate))

            kkt_violations.append({
                'alpha': alpha,
                'active_set_report': active_set_str,
                'active_set_kkt': str(tuple(sorted(active_indices_kkt))),
                'max_violation': stationarity_violation,
                'grad_consistency_violation': grad_consistency_violation,
                'check_result': 'OK' if stationarity_violation < check_tol * 10 else 'WARN'
            })

        if PANDAS_AVAILABLE and kkt_violations:
            df_kkt = pd.DataFrame(kkt_violations)
            kkt_cols_order = ['alpha', 'active_set_report','active_set_kkt', 'max_violation', 'grad_consistency_violation', 'check_result']
            kkt_cols_order = [col for col in kkt_cols_order if col in df_kkt.columns]
            with pd.option_context('display.float_format', '{:.4e}'.format, 'display.max_rows', None): # 全行表示
                print(df_kkt[kkt_cols_order].to_string(index=False, na_rep='NaN'))
            print("Notes on KKT Check:")
            print(" - active_set_report: Active set reported by inner solver")
            print(" - active_set_kkt: Active set inferred from w* > tol")
            print(" - max_violation: Max KKT violation (dual infeasibility or comp. slackness based on eta estimate)")
            print(" - grad_consistency_violation: max(active_grads) - min(active_grads) for w_m > tol")
        else:
            print("Could not perform KKT check or pandas not available.")

    elif isinstance(results_df, list):
        print("\n--- Analysis Results Summary (List) ---")
        # (表示省略)
        print("\n(Pandas not available, skipping CSV export)")

    end_time_main = time.time()
    print(f"\nTotal execution time: {end_time_main - start_time_main:.2f} seconds")

------------------------------------------------------------
--- Searching for Internal Solution around alpha=0.75 [0.700, 0.800] ---
--- (Using ActiveSet, mu_tilde=0.01, Quasi-Symmetric Params) ---
------------------------------------------------------------

--- Starting Full Results Analysis over Alpha Range (using Active Set Method) ---
Analyzing alpha = 0.800000 (101/101) ... 
--- Finished Full Results Analysis ---

--- Analysis Results Summary (DataFrame) ---
     alpha     H_star active_set     w_fw_0     w_fw_1     w_fw_2       pi_0       pi_1       pi_2       pi_3        pi_4   grad_H_0   grad_H_1   grad_H_2   lambda_0   lambda_1   lambda_2
7.0000e-01 1.9786e-03     (0, 1) 9.9969e-01 1.5540e-04 1.5540e-04 1.4671e-01 1.2588e-01 1.5709e-02 4.1068e-03 -2.9241e-01 1.8786e-03 1.8762e-03 1.8746e-03 2.4651e-01 1.4921e-01 0.0000e+00
7.0100e-01 1.9786e-03     (0, 2) 9.9969e-01 1.5540e-04 1.5540e-04 1.4671e-01 1.2588e-01 1.5709e-02 4.1068e-03 -2.9241e-01 1.8786e-03 1.8762e-03 1.8746e-03

In [5]:
# -*- coding: utf-8 -*-
"""
5_1_find_internal_solution_mu025_quasi_symmetric.py

Attempts to find an internal solution w* by using quasi-symmetric parameters
(slight difference in Vm) around alpha=0.75, testing mu_tilde = 0.025.
Uses the Active Set method for the inner QP solve.
"""

import numpy as np
import matplotlib.pyplot as plt
import warnings
from scipy.linalg import solve, LinAlgError, eigh, inv, qr
from scipy.optimize import linprog, OptimizeWarning
import traceback
import time
import itertools
import os
import math
from collections import Counter # オプション

# pandas がなければインストールを促す
try:
    import pandas as pd
    PANDAS_AVAILABLE = True
    pd.set_option('display.width', 180) # 表示幅を少し広げる
    pd.set_option('display.float_format', '{:.6e}'.format) # 科学技術表記
    pd.set_option('display.max_rows', 300) # 表示行数を増やす
except ImportError:
    PANDAS_AVAILABLE = False
    print("Warning: pandas library not found. Output formatting will be basic. CSV export disabled.")

# === ここから必要な関数定義 (提供されたコードと同じものが続く) ===
DEFAULT_TOLERANCE = 1e-9

# --- OptimizationResult クラス ---
class OptimizationResult:
    def __init__(self, success, message, w_opt=None, pi_opt=None, lambda_opt=None,
                 H_opt=None, grad_H_opt=None, iterations=None, fw_gap=None,
                 active_set_opt=None):
        self.success = success; self.message = message; self.w_opt = w_opt; self.pi_opt = pi_opt
        self.lambda_opt = lambda_opt; self.H_opt = H_opt; self.grad_H_opt = grad_H_opt
        self.iterations = iterations; self.fw_gap = fw_gap
        self.active_set_opt = active_set_opt

# --- find_feasible_initial_pi 関数 ---
def find_feasible_initial_pi(R, mu_tilde, K, tolerance=1e-8):
    M = R.shape[1]; c = np.zeros(K + 1); c[K] = 1.0
    A_ub = np.hstack((-R.T, -np.ones((M, 1)))); b_ub = -mu_tilde * np.ones(M)
    bounds = [(None, None)] * K + [(0, None)]; opts = {'tol': tolerance, 'disp': False, 'presolve': True}
    result = None; methods_to_try = ['highs', 'highs-ipm', 'highs-ds', 'simplex'] # Removed 'interior-point' as it's deprecated and caused issues
    for method in methods_to_try:
        try:
            # Suppress specific warnings during linprog call if needed
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", OptimizeWarning) # Ignore presolve warnings etc.
                result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method=method, options=opts)
            if result.success: break
        except ValueError: # Handle potential issues like KKT matrix singular
            continue
        except Exception as e: # Catch other unexpected errors
            print(f"\n  Phase 1 LP Exception ({method}): {e}") # Print error for debugging
            return None, False, f"Phase 1 LP failed ({method}): {e}" # Return specific error
    # Check result status carefully
    if result is None or not result.success:
        msg = result.message if result else "No solver succeeded"
        status = result.status if result else -1
        return None, False, f"Phase 1 LP solver failed: {msg} (status={status}, method={method if result else 'N/A'})"

    s = result.x[K]; pi = result.x[:K]

    if np.isnan(pi).any(): # Explicit NaN check
        return None, False, "Phase 1 LP resulted in NaN values for pi."

    # Check feasibility more robustly
    G = -R.T; h = -mu_tilde * np.ones(M); violation = np.max(G @ pi - h)
    feasibility_tol = tolerance * 10000 # Tolerance for feasibility check

    # Phase 1 goal is s=0. Check if s is close to zero.
    if s <= tolerance * 100: # s should ideally be zero
        if violation <= feasibility_tol:
             return pi, True, f"Phase 1 OK (s*={s:.1e}, max_viol={violation:.1e})"
        else:
             # s is small, but initial pi is still infeasible - might happen with loose tol
             return pi, True, f"Phase 1 OK (s*={s:.1e}), WARN: Initial violation {violation:.1e} > {feasibility_tol:.1e}"
    else:
        # s is significantly positive, means the original constraints are likely infeasible
        return None, False, f"Phase 1 Likely Infeasible (s* = {s:.1e} > {tolerance*100:.1e}), max_viol={violation:.1e}"


# --- solve_kkt_system 関数 ---
def solve_kkt_system(Q, G_W, g, tolerance=DEFAULT_TOLERANCE):
    K = Q.shape[0]; n_act = G_W.shape[0] if G_W is not None and G_W.ndim == 2 and G_W.shape[0] > 0 else 0
    if n_act == 0:
        try:
            # Use 'auto' or specify based on Q properties if known (e.g., 'pos')
            # Added check_finite=False for potential minor non-finite inputs
            p = solve(Q, -g, assume_a='sym', check_finite=False)
            l = np.array([]) # No active constraints, lambda is empty
        except LinAlgError: # Matrix might be singular or nearly singular
            return None, None, False
        except ValueError as e: # Input contains NaN, infinity or is too large
             print(f"DEBUG KKT (n_act=0) ValueError: {e}")
             return None, None, False

        # Check residual norm after solve
        if p is None or np.isnan(p).any() or np.isinf(p).any(): return None, None, False
        res_norm = np.linalg.norm(Q @ p + g); g_norm = np.linalg.norm(g)
        # Increased tolerance slightly for numerical stability
        solved_ok = res_norm <= tolerance * 1e4 * (1 + g_norm)
        return p, l, solved_ok
    else:
        kkt_mat = None; rhs = None
        try:
            # Ensure G_W is correctly shaped before blocking
            if G_W.ndim != 2 or G_W.shape[1] != K:
                 print(f"DEBUG KKT (n_act>0) G_W shape error: {G_W.shape}")
                 return None, None, False
            kkt_mat = np.block([[Q, G_W.T], [G_W, np.zeros((n_act, n_act))]])
            rhs = np.concatenate([-g, np.zeros(n_act)])
        except ValueError as e: # If shapes mismatch during block creation
            print(f"DEBUG KKT (n_act>0) Block ValueError: {e}")
            return None, None, False

        try:
            # Added check_finite=False
            sol = solve(kkt_mat, rhs, assume_a='sym', check_finite=False)
            p = sol[:K]; l = sol[K:]
        except LinAlgError: # KKT matrix might be singular
            # Attempt pseudo-inverse as a fallback? Or just fail.
            # For now, just fail. Add pinv later if needed.
            # print(f"DEBUG KKT (n_act>0) LinAlgError. Cond: {np.linalg.cond(kkt_mat)}") # Check condition number
            return None, None, False
        except ValueError as e: # Input contains NaN, infinity or is too large
            print(f"DEBUG KKT (n_act>0) ValueError: {e}")
            return None, None, False

        # Check residual norm after solve
        if sol is None or np.isnan(sol).any() or np.isinf(sol).any(): return None, None, False
        res_norm = np.linalg.norm(kkt_mat @ sol - rhs); rhs_norm = np.linalg.norm(rhs)
        # Increased tolerance slightly for numerical stability
        solved_ok = res_norm <= tolerance * 1e4 * (1 + rhs_norm)
        return p, l, solved_ok

# --- solve_inner_qp_active_set 関数 (より安定性を高める試み) ---
def solve_inner_qp_active_set(Vw, R, mu_tilde, initial_pi, max_iter=350, tolerance=DEFAULT_TOLERANCE, regularization_epsilon=1e-10):
    K = Vw.shape[0]; M = R.shape[1];
    # Ensure Vw is finite before regularization
    if not np.all(np.isfinite(Vw)): return None, None, None, None, None, False, "QP fail: Vw contains NaN/Inf."
    # Regularize Q matrix (ensure positive definite)
    Q_reg = 2 * Vw + 2 * regularization_epsilon * np.eye(K)
    # Check condition number of Q_reg
    cond_Q = np.linalg.cond(Q_reg)
    if cond_Q > 1/tolerance: # If condition number is too high
        warnings.warn(f"QP Warning: Condition number of Q_reg is high ({cond_Q:.2e}). Regularization might be insufficient.")
        # Consider increasing regularization if this happens often
        # Q_reg += 2 * (regularization_epsilon * 10) * np.eye(K)

    G = -R.T; h = -mu_tilde * np.ones(M)

    if initial_pi is None or not np.all(np.isfinite(initial_pi)):
        return None, None, None, None, None, False, "QP fail: Initial pi is None or contains NaN/Inf."

    pi_k = np.copy(initial_pi)
    lam_opt = np.zeros(M)
    W = set() # Active set (indices of constraints treated as equalities)

    # Tolerance for constraint activation/violation checks
    active_tol = tolerance * 100 # Increased tolerance slightly

    # Initial check for feasibility and active set determination
    initial_violations = G @ pi_k - h
    W = set(j for j, viol in enumerate(initial_violations) if viol > -active_tol) # Activate constraints violated or close to boundary
    max_initial_violation = np.max(initial_violations)
    if max_initial_violation > active_tol * 10: # Allow small initial infeasibility
        warnings.warn(f"QP Warning: Initial pi infeasible (max viol: {max_initial_violation:.2e} > {active_tol * 10:.1e}). Active set: {sorted(list(W))}")
        # Attempt to project initial pi onto feasible region? Might be complex. Start anyway.

    active_indices_opt = None # Store the final active set indices

    for i in range(max_iter):
        g_k = Q_reg @ pi_k # Gradient of objective at pi_k
        if not np.all(np.isfinite(g_k)): return None, None, None, None, None, False, f"QP fail:Iter {i+1}: Gradient g_k contains NaN/Inf."

        act = sorted(list(W))
        n_act = len(act)
        G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))

        # Solve KKT system for search direction p_k and Lagrange multipliers lam_Wk
        p_k, lam_Wk, solved = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)

        # Check if KKT solve was successful
        if not solved or p_k is None:
            return None, None, None, None, None, False, f"QP fail:Iter {i+1}: KKT solve failed. ActiveSet={act}. Cond(Q)={cond_Q:.1e}"

        # Check for convergence: If search direction p_k is close to zero
        p_norm = np.linalg.norm(p_k)
        pi_k_norm = np.linalg.norm(pi_k)
        if p_norm <= tolerance * 100 * (1 + pi_k_norm): # Increased tolerance
            # Check optimality conditions (Lagrange multipliers for active constraints)
            is_optimal_point = True
            blocking_constraint_idx = -1
            min_negative_lambda = float('inf')
            # Tolerance for checking non-negativity of lambda
            dual_feas_tol = -tolerance * 100 # Allow slightly negative lambda

            if W: # If there are active constraints
                if lam_Wk is None or len(lam_Wk) != n_act:
                    return None, None, None, None, None, False, f"QP fail:Iter {i+1}: lam_Wk inconsistent? ActiveSet={act}"
                if not np.all(np.isfinite(lam_Wk)):
                     return None, None, None, None, None, False, f"QP fail:Iter {i+1}: lam_Wk contains NaN/Inf. ActiveSet={act}"

                lambda_map = dict(zip(act, lam_Wk))
                for constraint_idx, lagrange_multiplier in lambda_map.items():
                    if lagrange_multiplier < dual_feas_tol:
                        is_optimal_point = False
                        # Find the index of the constraint with the most negative multiplier
                        if lagrange_multiplier < min_negative_lambda:
                            min_negative_lambda = lagrange_multiplier
                            blocking_constraint_idx = constraint_idx

            if is_optimal_point:
                # Optimal solution found
                lam_opt.fill(0.0)
                if W and lam_Wk is not None and len(lam_Wk) == n_act:
                    # Assign non-negative multipliers
                    try:
                        lam_opt[act] = np.maximum(lam_Wk, 0)
                    except IndexError: # Should not happen if act and lam_Wk are consistent
                        return None, None, None, None, None, False, f"QP fail:Iter {i+1}: Index error assigning lambda. ActiveSet={act}"

                # Final check of feasibility
                final_infeas = np.max(G @ pi_k - h)
                msg = f"Optimal found at iter {i+1}."
                if final_infeas > active_tol: # Check against activation tolerance
                    msg += f" (WARN: Final violation {final_infeas:.1e})"
                active_indices_opt = act # Store the active set at optimum
                return pi_k, lam_opt, active_indices_opt, None, Q_reg / 2.0, True, msg
            else:
                # Not optimal: remove the constraint with the most negative multiplier
                if blocking_constraint_idx in W:
                    W.remove(blocking_constraint_idx)
                    # Continue to the next iteration with the updated active set
                    continue
                else:
                    # Should not happen if blocking_constraint_idx was found correctly
                    return None, None, None, None, None, False, f"QP fail:Iter {i+1}: Negative lambda idx {blocking_constraint_idx}, not in W={act}"
        else:
            # Non-zero search direction p_k: Move along p_k
            alpha_k = 1.0 # Initial step length (full step)
            blocking_constraint_idx = -1
            min_step_length = float('inf')
            # Tolerance for checking if a step is blocked by a constraint
            step_tol = tolerance * 100 # Increased tolerance

            # Check inactive constraints to see if any block the step
            for j in range(M):
                if j not in W: # Check only inactive constraints
                    constraint_gradient_dot_p = G[j, :] @ p_k
                    # If moving along p_k violates constraint j (G[j,:] @ (pi_k + alpha*p_k) > h[j])
                    # This corresponds to G[j,:] @ p_k > 0 if constraint j is currently G[j,:] @ pi_k <= h[j]
                    if constraint_gradient_dot_p > step_tol: # Potential to violate
                        # Calculate distance to the boundary of constraint j
                        distance_to_boundary = h[j] - (G[j, :] @ pi_k)
                        if abs(constraint_gradient_dot_p) > 1e-15: # Avoid division by zero
                            alpha_j = distance_to_boundary / constraint_gradient_dot_p
                            # Ensure step is non-negative (should be if distance_to_boundary >= 0)
                            step_j = max(0.0, alpha_j)
                            # Find the smallest step length that hits a boundary
                            if step_j < min_step_length:
                                min_step_length = step_j
                                blocking_constraint_idx = j
                        # else: if constraint_gradient_dot_p is near zero, this constraint won't block significantly

            # Determine the actual step length: minimum of full step (1.0) and blocking step
            alpha_k = min(1.0, min_step_length)

            # Update the current solution
            pi_k += alpha_k * p_k

            # If the step was blocked (alpha_k < 1.0), add the blocking constraint to the active set
            # Check if alpha_k is significantly less than 1
            if alpha_k < 1.0 - step_tol and blocking_constraint_idx != -1:
                if blocking_constraint_idx not in W:
                    W.add(blocking_constraint_idx)

            # Continue to the next iteration
            continue

    # Max iterations reached
    msg = f"Max iter ({max_iter}) reached."
    # Check final feasibility
    final_infeas = np.max(G @ pi_k - h)
    if final_infeas > active_tol * 10: # Check against a larger tolerance
        return None, None, None, None, None, False, f"{msg} Final infeasible ({final_infeas:.1e} > {active_tol*10:.1e}). ActiveSet={sorted(list(W))}"

    # Try to assess optimality at the final point (even if max_iter reached)
    act = sorted(list(W))
    n_act = len(act)
    G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))
    is_likely_optimal = False
    active_constraints_opt = act

    g_k = Q_reg @ pi_k # Final gradient
    if not np.all(np.isfinite(g_k)): return None, None, None, None, None, False, f"{msg} Final gradient g_k contains NaN/Inf."

    p_f, lam_f, solved_f = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)
    final_lambda_estimate = np.zeros(M)

    # Check if the final point satisfies approximate KKT conditions
    # Check stationarity (p_f should be near zero)
    if solved_f and p_f is not None and np.linalg.norm(p_f) <= tolerance * 1000 * (1 + np.linalg.norm(pi_k)): # Looser check
        # Check dual feasibility (lambda for active constraints should be non-negative)
        if n_act > 0:
            if lam_f is not None and len(lam_f) == n_act and np.all(np.isfinite(lam_f)):
                 try:
                     final_lambda_estimate[act] = lam_f
                 except IndexError:
                     pass # Should not happen
                 active_lambdas = final_lambda_estimate[act]
                 if np.all(active_lambdas >= -tolerance * 1000): # Allow slightly negative lambda
                     is_likely_optimal = True
                     msg += " Final KKT check approx OK."
                 else:
                     min_lam = np.min(active_lambdas)
                     msg += f" Final KKT check fails (dual infeasible, min_lam={min_lam:.1e})."
            else:
                 msg += " Final KKT check fails (lam_f invalid or NaN/Inf)."
        else: # No active constraints, optimality requires gradient = 0 (checked by p_f ~ 0)
             is_likely_optimal = True
             msg += " Final KKT check approx OK (unconstrained)."
    else:
        p_norm_f = np.linalg.norm(p_f) if p_f is not None else np.nan
        msg += f" Final KKT check fails (stationarity error, p_norm={p_norm_f:.1e} or solve error)."

    # Return the final solution, even if potentially suboptimal due to max_iter
    lam_opt = np.maximum(final_lambda_estimate, 0) # Ensure non-negativity for return
    return pi_k, lam_opt, active_constraints_opt, None, Q_reg / 2.0, is_likely_optimal, msg

# --- make_psd 関数 ---
def make_psd(matrix, tolerance=1e-8):
    if not np.all(np.isfinite(matrix)):
         warnings.warn("make_psd: Input matrix contains NaN/Inf. Returning as is after symmetrization.")
         sym = (matrix + matrix.T) / 2.0 # Attempt symmetrization anyway
         return sym # Or should return None or raise error?
    sym = (matrix + matrix.T) / 2.0
    try:
        eigenvalues, eigenvectors = np.linalg.eigh(sym)
        min_eigenvalue = np.min(eigenvalues)
        if min_eigenvalue < tolerance:
            # Shift eigenvalues to be at least `tolerance`
            eigenvalues[eigenvalues < tolerance] = tolerance
            # Reconstruct the matrix
            psd_matrix = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T
            # Ensure symmetry again due to potential numerical errors
            return (psd_matrix + psd_matrix.T) / 2.0
        else:
            # Already PSD (or positive definite)
            return sym
    except LinAlgError:
        warnings.warn("make_psd: eigh failed, possibly due to non-finite values or extreme scaling. Returning symmetrized input.")
        return sym # Fallback to returning the symmetrized matrix


# --- calculate_Vw 関数 ---
def calculate_Vw(w, R, SecondMoments_a_array, tolerance=DEFAULT_TOLERANCE, psd_tolerance=1e-8):
    K, M = R.shape
    # Ensure w sums to 1 (or close enough) and non-negative
    w_norm = project_to_simplex(w) # Project w onto simplex for safety
    if not np.isclose(np.sum(w_norm), 1.0):
         raise ValueError(f"calculate_Vw: w does not sum to 1 after projection: {np.sum(w_norm)}")

    # Check inputs for NaN/Inf
    if not np.all(np.isfinite(R)): raise ValueError("calculate_Vw: R contains NaN/Inf.")
    if not np.all(np.isfinite(SecondMoments_a_array)): raise ValueError("calculate_Vw: SecondMoments_a_array contains NaN/Inf.")

    EwX = R @ w_norm
    EwXXT = np.zeros((K, K))
    for m in range(M):
        EwXXT += w_norm[m] * SecondMoments_a_array[m]

    # Check calculated moments for NaN/Inf
    if not np.all(np.isfinite(EwX)): raise ValueError("calculate_Vw: EwX calculation resulted in NaN/Inf.")
    if not np.all(np.isfinite(EwXXT)): raise ValueError("calculate_Vw: EwXXT calculation resulted in NaN/Inf.")

    Vw = EwXXT - np.outer(EwX, EwX)
    if not np.all(np.isfinite(Vw)): raise ValueError("calculate_Vw: Vw calculation resulted in NaN/Inf.")

    Vw_psd = make_psd(Vw, psd_tolerance)
    # Final check after PSD making
    if not np.all(np.isfinite(Vw_psd)): raise ValueError("calculate_Vw: Vw_psd after make_psd contains NaN/Inf.")

    return Vw_psd, EwX, EwXXT

# --- calculate_H_gradient 関数 (ノルムチェック緩和 + NaNチェック強化) ---
def calculate_H_gradient(pi_star, w, R, SecondMoments_a_array, EwX, EwXXT, tolerance=1e-9, debug_print=False):
    M = w.shape[0]; K = R.shape[0]; grad = np.zeros(M)
    norm_tolerance = 1e-12 # Allow very small pi_star

    # --- Input Validation ---
    if pi_star is None: return np.full(M, np.nan) # Grad cannot be computed
    if not np.all(np.isfinite(pi_star)): return np.full(M, np.nan)
    if not np.all(np.isfinite(w)): return np.full(M, np.nan)
    if not np.all(np.isfinite(R)): return np.full(M, np.nan)
    if not np.all(np.isfinite(SecondMoments_a_array)): return np.full(M, np.nan)
    if EwX is None or not np.all(np.isfinite(EwX)): return np.full(M, np.nan)
    if EwXXT is None or not np.all(np.isfinite(EwXXT)): return np.full(M, np.nan)
    # --- End Input Validation ---

    pi_norm = np.linalg.norm(pi_star)
    # If pi_star is numerically zero, gradient is effectively zero (or undefined), return NaN to signal issue
    if pi_norm < norm_tolerance:
        if debug_print: print(f"DEBUG grad_H: Returning NaN because pi_star norm {pi_norm:.2e} < {norm_tolerance:.1e}")
        return np.full(M, np.nan)

    try:
        pi_T_EwX = pi_star.T @ EwX
        if not np.isfinite(pi_T_EwX):
             if debug_print: print(f"DEBUG grad_H: pi_T_EwX is NaN/Inf: {pi_T_EwX}")
             return np.full(M, np.nan)

        for j in range(M):
            Sigma_j = SecondMoments_a_array[j]
            r_j = R[:, j]

            # Term 1: pi^T * Sigma_j * pi
            pi_T_Sigma_j_pi = pi_star.T @ Sigma_j @ pi_star
            if not np.isfinite(pi_T_Sigma_j_pi):
                 if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_Sigma_j_pi is NaN/Inf: {pi_T_Sigma_j_pi}")
                 grad[j] = np.nan; continue # Mark as NaN and skip to next component

            # Intermediate term: pi^T * r_j
            pi_T_r_j = pi_star.T @ r_j
            if not np.isfinite(pi_T_r_j):
                 if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_r_j is NaN/Inf: {pi_T_r_j}")
                 grad[j] = np.nan; continue

            # Term 2: 2 * (pi^T * r_j) * (pi^T * EwX)
            term2 = 2 * pi_T_r_j * pi_T_EwX
            if not np.isfinite(term2):
                 if debug_print: print(f"DEBUG grad_H (j={j}): term2 (2*pi_T_r_j*pi_T_EwX) is NaN/Inf: {term2}, pi_T_r_j={pi_T_r_j}, pi_T_EwX={pi_T_EwX}")
                 grad[j] = np.nan; continue

            # Gradient component j = Term 1 - Term 2
            grad[j] = pi_T_Sigma_j_pi - term2
            if not np.isfinite(grad[j]): # Final check for the component
                 if debug_print: print(f"DEBUG grad_H (j={j}): final grad[{j}] is NaN/Inf")
                 # Keep grad[j] as NaN if calculation resulted in NaN/Inf

    except Exception as e_calc:
         print(f"\nERROR in calculate_H_gradient calculation: {e_calc}")
         print(traceback.format_exc())
         return np.full(M, np.nan) # Return NaN array on unexpected error

    # Final check on the whole gradient vector
    if np.any(~np.isfinite(grad)):
        # warnings.warn(f"NaN or Inf detected in calculated gradient for w={w}. Grad={grad}")
        if debug_print: print(f"DEBUG grad_H: Final check found NaN/Inf in gradient: {grad}")
        # Return the gradient vector possibly containing NaNs
        return grad # Don't replace with all NaNs if only some failed

    return grad


# --- project_to_simplex 関数 ---
def project_to_simplex(v, z=1):
    n_features = v.shape[0]
    if n_features == 0: return np.array([]) # Handle empty input

    # Ensure v is a numpy array and finite
    v_arr = np.asarray(v)
    if not np.all(np.isfinite(v_arr)):
        # Handle non-finite values: Option 1: Raise error
        # raise ValueError("Input vector contains NaN or Inf.")
        # Option 2: Try to recover (e.g., replace with 0 or average), but this can be problematic.
        # Option 3: Return a default valid projection (e.g., uniform) - might hide issues.
        # Let's return uniform for now, with a warning.
        warnings.warn("project_to_simplex: Input vector contains NaN/Inf. Returning uniform projection.")
        return np.full(n_features, z / n_features) if n_features > 0 else np.array([])


    # Check if already on simplex (within tolerance)
    if np.all(v_arr >= -1e-9) and np.isclose(np.sum(v_arr), z):
        return np.maximum(v_arr, 0) # Ensure non-negativity strictly

    # Use the standard projection algorithm (Micchelli, etc.)
    u = np.sort(v_arr)[::-1] # Sort in descending order
    cssv = np.cumsum(u) - z
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0

    # Handle the case where no rho is found (e.g., all elements <= 0 for z > 0)
    if np.any(cond):
        rho = ind[cond][-1]
        theta = cssv[rho - 1] / float(rho)
        w = np.maximum(v_arr - theta, 0)
    else:
        # This case might happen if v has negative components and z=1.
        # Project onto the closest point, which might be a vertex.
        # A simple fallback: assign total weight to the max element if z > 0.
         w = np.zeros(n_features)
         if z > 0:
             w[np.argmax(v_arr)] = z # Assign weight to the largest component index

    # Normalize to ensure sum is exactly z, handling potential floating point errors
    w_sum = np.sum(w)
    if not np.isclose(w_sum, z):
        if w_sum > 1e-9: # Avoid division by zero if sum is tiny
            w = w * (z / w_sum)
        elif z > 0 : # If sum is near zero but should be positive z
            # Fallback again: Assign total weight to the max element's index from original v
            w = np.zeros(n_features)
            w[np.argmax(v_arr)] = z
        # If z=0, w should be all zeros, which is likely covered.

    # Final check for non-negativity
    return np.maximum(w, 0)


# --- frank_wolfe_optimizer 関数 (ActiveSet法を使用、最終解の整合性を高める修正済み) ---
def frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde, initial_w=None, max_outer_iter=250, fw_gap_tol=1e-7, inner_max_iter=350, tolerance=1e-9, psd_make_tolerance=1e-8, qp_regularization=1e-10, debug_print=False, force_iterations=0, return_history=False):
    K, M = R_alpha.shape
    if initial_w is None: w_k = np.ones(M) / M
    else: w_k = project_to_simplex(np.copy(initial_w))

    if w_k is None or not np.all(np.isfinite(w_k)): # Check initial w_k
        result = OptimizationResult(False, "Initial w_k invalid after projection", w_opt=initial_w)
        return (result, []) if return_history else result

    fw_gap = float('inf')
    best_w = np.copy(w_k)
    best_pi = None
    best_lam = np.zeros(M)
    best_H = -float('inf')
    best_grad_H = np.full(M, np.nan)
    best_active_set = None
    history = []

    # --- Phase 1: Find initial feasible pi_0 ---
    pi0, ok, p1msg = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance)
    if not ok:
        result = OptimizationResult(False, f"Phase 1 failed: {p1msg}", w_opt=initial_w)
        return (result, []) if return_history else result
    if pi0 is None or not np.all(np.isfinite(pi0)): # Check pi0 validity
        result = OptimizationResult(False, "Phase 1 returned invalid pi0", w_opt=initial_w)
        return (result, []) if return_history else result
    # --- End Phase 1 ---

    # Initialize tracking variables
    current_w = np.copy(w_k)
    current_H = -float('inf')
    current_pi = np.copy(pi0) # Start with feasible pi0
    current_lam = np.zeros(M)
    current_active_set = tuple()
    current_grad_H = np.full(M, np.nan)
    current_fw_gap = float('inf')

    # QP solver args
    inner_solver_args_for_loop = {'max_iter': inner_max_iter, 'tolerance': tolerance, 'regularization_epsilon': qp_regularization}

    last_successful_pi = np.copy(pi0) # Store the last valid pi found

    for k in range(max_outer_iter):
        iter_data = {'k': k + 1, 'w_k': np.copy(w_k)} if return_history else {}

        # --- Calculate Vw and related terms ---
        try:
            Vk, Ex, ExxT = calculate_Vw(w_k, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
            if not np.all(np.isfinite(Vk)): raise ValueError("Vk contains NaN/Inf")
            if not np.all(np.isfinite(Ex)): raise ValueError("Ex contains NaN/Inf")
        except Exception as e:
            msg = f"Outer iter {k+1}: Vw calculation failed: {e}. Using best known state."
            warnings.warn(msg)
            result = OptimizationResult(False, msg, w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, grad_H_opt=best_grad_H, active_set_opt=best_active_set, iterations=k)
            if return_history: history.append(iter_data); return (result, history)
            else: return result
        # --- End Calculate Vw ---

        # --- Inner QP Solve (using Active Set) ---
        # Use the last successful pi as the initial guess for the inner solver
        pi_init_inner = last_successful_pi if last_successful_pi is not None else pi0

        # Check if pi_init_inner is valid before passing
        if pi_init_inner is None or not np.all(np.isfinite(pi_init_inner)):
             warnings.warn(f"Outer iter {k+1}: Invalid pi_init_inner. Reverting to pi0.")
             pi_init_inner = pi0 # Fallback to the initial feasible point
             if pi_init_inner is None or not np.all(np.isfinite(pi_init_inner)): # If pi0 is also invalid, fail
                  msg = f"Outer iter {k+1}: Both last successful pi and pi0 are invalid. Cannot solve inner QP."
                  result = OptimizationResult(False, msg, w_opt=best_w, iterations=k)
                  if return_history: history.append(iter_data); return (result, history)
                  else: return result

        pk, lk, act_idx_k, _, _, inner_ok, inner_msg = solve_inner_qp_active_set(
            Vk, R_alpha, mu_tilde, pi_init_inner, **inner_solver_args_for_loop
        )
        # --- End Inner QP Solve ---

        # --- Process Inner QP Results ---
        if not inner_ok or pk is None or not np.all(np.isfinite(pk)):
            warnings.warn(f"Outer iter {k+1}: Inner QP failed or returned invalid pi: {inner_msg}. Trying fallback.")
            # Fallback 1: Try solving QP with the default pi0
            if pi_init_inner is not pi0: # Avoid re-solving if already tried pi0
                pi0_check, _, _, _, _, inner_ok_pi0, inner_msg_pi0 = solve_inner_qp_active_set(Vk, R_alpha, mu_tilde, pi0, **inner_solver_args_for_loop)
                if inner_ok_pi0 and pi0_check is not None and np.all(np.isfinite(pi0_check)):
                    warnings.warn(f"Outer iter {k+1}: Inner QP succeeded with pi0 after failing with last_successful_pi.")
                    pk = pi0_check
                    # Need to recalculate lk, act_idx_k if using pi0_check - Requires another KKT solve or accepting potentially inaccurate lambda/active set from solve_inner_qp_active_set
                    # For simplicity, let's re-use the results from the successful call if possible, or reset them
                    # This part is tricky - ideally solve_inner_qp returns consistent results
                    lk = np.zeros(M) # Reset lambda as it's unreliable
                    act_idx_k = [] # Reset active set
                    inner_ok = True # Mark as ok for gradient calculation
                else:
                    warnings.warn(f"Outer iter {k+1}: Inner QP also failed with pi0: {inner_msg_pi0}. Stopping.")
                    msg = f"Outer iter {k+1}: Inner QP failed with both initial guesses. {inner_msg}"
                    result = OptimizationResult(False, msg, w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, grad_H_opt=best_grad_H, active_set_opt=best_active_set, iterations=k)
                    if return_history: history.append(iter_data); return (result, history)
                    else: return result
            else: # Already tried pi0 or pi_init_inner was pi0
                msg = f"Outer iter {k+1}: Inner QP failed: {inner_msg}. Stopping."
                result = OptimizationResult(False, msg, w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, grad_H_opt=best_grad_H, active_set_opt=best_active_set, iterations=k)
                if return_history: history.append(iter_data); return (result, history)
                else: return result

        # Inner QP succeeded (either initially or with fallback)
        last_successful_pi = np.copy(pk) # Update last successful pi
        Hk = pk.T @ Vk @ pk
        current_pi = np.copy(pk)
        current_lam = lk if lk is not None and np.all(np.isfinite(lk)) else np.zeros(M)
        current_active_set = tuple(sorted(act_idx_k)) if act_idx_k is not None else tuple()
        current_H = Hk

        # --- Calculate Gradient of H ---
        try:
            gHk = calculate_H_gradient(pk, w_k, R_alpha, SecondMoments_alpha_array, Ex, ExxT, tolerance, debug_print=debug_print)
            if gHk is None or np.any(~np.isfinite(gHk)):
                raise ValueError("Gradient calculation returned None or NaN/Inf")
            current_grad_H = gHk
        except Exception as e:
            msg = f"Outer iter {k+1}: Gradient calculation failed: {e}. Stopping."
            warnings.warn(msg)
            result = OptimizationResult(False, msg, w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, grad_H_opt=best_grad_H, active_set_opt=best_active_set, iterations=k)
            if return_history: history.append(iter_data); return (result, history)
            else: return result
        # --- End Calculate Gradient ---

        if return_history:
             iter_data['H_k'] = Hk
             iter_data['grad_H_k_norm'] = np.linalg.norm(current_grad_H)
             iter_data['pi_k'] = np.copy(pk)
             iter_data['lam_k'] = np.copy(current_lam)
             iter_data['active_set_k'] = current_active_set

        # --- Update Best Solution Found So Far ---
        # Use a small tolerance for comparing H values
        if Hk > best_H + tolerance * 1e-1: # If significantly better or first valid H
            best_H = Hk; best_w = np.copy(w_k); best_pi = np.copy(pk)
            best_lam = np.copy(current_lam); best_grad_H = np.copy(current_grad_H)
            best_active_set = current_active_set
        elif np.isclose(Hk, best_H, atol=tolerance*1e-1, rtol=tolerance*1e-1): # If close to best H
             # Optional: Update if w is substantially different? Or keep the first one found.
             # Keep the current best for simplicity unless Hk is clearly better.
             pass

        # --- Frank-Wolfe Step ---
        # Find descent direction s_k (vertex maximizing linear approximation)
        grad_norm = np.linalg.norm(current_grad_H)
        # Check if gradient is too small (potential convergence or numerical issue)
        if grad_norm < tolerance * 100: # Increased tolerance
             sk = w_k # Stay at current point if gradient is negligible
             sk_idx = -1 # Indicate no specific vertex direction
        else:
             sk_idx = np.argmax(current_grad_H)
             sk = np.zeros(M); sk[sk_idx] = 1.0
        if return_history: iter_data['s_k_index'] = sk_idx

        # Calculate FW gap
        fw_gap = current_grad_H.T @ (w_k - sk)
        current_fw_gap = fw_gap # Update current gap
        if return_history: iter_data['fw_gap'] = fw_gap

        # --- Check Convergence ---
        converged = False
        # Check gap after force_iterations, ensure gap is finite
        if k >= force_iterations and np.isfinite(fw_gap):
            if abs(fw_gap) <= fw_gap_tol:
                 converged = True
                 conv_msg = f"Converged (Gap {abs(fw_gap):.2e} <= {fw_gap_tol:.1e})"

        if converged:
            # --- Final Check/Refinement ---
            # Optional: Run one last inner QP solve at the converged w_k
            # to ensure pi, lambda, grad_H are consistent with the final w.
            final_w = np.copy(w_k)
            final_pi = np.copy(current_pi)
            final_lam = np.copy(current_lam)
            final_active_set = current_active_set
            final_H = current_H
            final_grad = np.copy(current_grad_H)
            final_gap = current_fw_gap

            try:
                final_Vk, final_Ex, final_ExxT = calculate_Vw(final_w, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
                # Use a known feasible point (pi0) for the final consistency check solve
                pi_f, lam_f, act_f, _, _, ok_f, msg_f = solve_inner_qp_active_set(
                    final_Vk, R_alpha, mu_tilde, pi0, # Use stable pi0
                    **inner_solver_args_for_loop
                )
                if ok_f and pi_f is not None and np.all(np.isfinite(pi_f)):
                     final_pi = pi_f # Update pi
                     final_lam = lam_f if lam_f is not None and np.all(np.isfinite(lam_f)) else np.zeros(M)
                     final_active_set = tuple(sorted(act_f)) if act_f is not None else tuple()
                     final_H = final_pi.T @ final_Vk @ final_pi # Recalculate H

                     # Recalculate gradient and gap for final consistency
                     try:
                         final_grad = calculate_H_gradient(final_pi, final_w, R_alpha, SecondMoments_alpha_array, final_Ex, final_ExxT, tolerance)
                         if final_grad is None or np.any(~np.isfinite(final_grad)): final_grad = np.full(M, np.nan) # Handle potential NaN
                         if np.any(np.isfinite(final_grad)): # Check if grad is valid before gap calc
                              grad_norm_f = np.linalg.norm(final_grad[np.isfinite(final_grad)]) # Norm of finite components
                              if grad_norm_f < tolerance * 100: sk_f = final_w
                              else: sk_idx_f = np.nanargmax(np.nan_to_num(final_grad, nan=-np.inf)); sk_f = np.zeros(M); sk_f[sk_idx_f] = 1.0
                              final_gap = final_grad.T @ (final_w - sk_f) if np.all(np.isfinite(final_grad)) else np.nan
                         else: final_gap = np.nan
                     except Exception as e_grad:
                         warnings.warn(f"Final gradient calculation failed after convergence: {e_grad}")
                         final_grad = np.full(M, np.nan)
                         final_gap = np.nan
                else:
                    warnings.warn(f"Warning: Final inner QP solve failed after convergence ({msg_f}). Using values from last iteration.")
                    # Keep values from last iteration (before final check)
            except Exception as e_final:
                warnings.warn(f"Error during final consistency check after convergence: {e_final}. Using values from last iteration.")
            # --- End Final Check ---

            result = OptimizationResult(True, conv_msg, w_opt=final_w, pi_opt=final_pi, lambda_opt=final_lam,
                                        H_opt=final_H, grad_H_opt=final_grad, iterations=k + 1,
                                        fw_gap=final_gap, active_set_opt=final_active_set)
            if return_history: history.append(iter_data); return (result, history)
            else: return result
        # --- End Convergence Check ---

        # --- Prepare for next iteration ---
        gamma = 2.0 / (k + 3.0) # Standard step size
        if return_history: iter_data['gamma_k'] = gamma
        if return_history: history.append(iter_data)

        w_k_next = (1.0 - gamma) * w_k + gamma * sk
        w_k = project_to_simplex(w_k_next) # Project back onto simplex

        if w_k is None or not np.all(np.isfinite(w_k)): # Check validity after update
             msg = f"Outer iter {k+1}: w_k became invalid after update/projection. Stopping."
             result = OptimizationResult(False, msg, w_opt=best_w, pi_opt=best_pi, H_opt=best_H, lambda_opt=best_lam, grad_H_opt=best_grad_H, active_set_opt=best_active_set, iterations=k+1)
             if return_history: return (result, history) # Return history up to failure
             else: return result
        # Update pi0 for next inner solve warm start (use the latest successful pi)
        pi0 = last_successful_pi # Note: already updated above if QP was successful


    # --- Max Iterations Reached ---
    # Perform final consistency check similar to convergence case
    final_w = np.copy(w_k)
    final_pi = np.copy(current_pi)
    final_lam = np.copy(current_lam)
    final_active_set = current_active_set
    final_H = current_H
    final_grad = np.copy(current_grad_H)
    final_gap = current_fw_gap

    try:
        final_Vk, final_Ex, final_ExxT = calculate_Vw(final_w, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
        # Use stable pi0 for final solve
        pi_f, lam_f, act_f, _, _, ok_f, msg_f = solve_inner_qp_active_set(
            final_Vk, R_alpha, mu_tilde, pi0,
            **inner_solver_args_for_loop
        )
        if ok_f and pi_f is not None and np.all(np.isfinite(pi_f)):
            final_pi = pi_f
            final_lam = lam_f if lam_f is not None and np.all(np.isfinite(lam_f)) else np.zeros(M)
            final_active_set = tuple(sorted(act_f)) if act_f is not None else tuple()
            final_H = final_pi.T @ final_Vk @ final_pi
            try:
                final_grad = calculate_H_gradient(final_pi, final_w, R_alpha, SecondMoments_alpha_array, final_Ex, final_ExxT, tolerance)
                if final_grad is None or np.any(~np.isfinite(final_grad)): final_grad = np.full(M, np.nan)
                if np.any(np.isfinite(final_grad)):
                     grad_norm_f = np.linalg.norm(final_grad[np.isfinite(final_grad)])
                     if grad_norm_f < tolerance * 100: sk_f = final_w
                     else: sk_idx_f = np.nanargmax(np.nan_to_num(final_grad, nan=-np.inf)); sk_f = np.zeros(M); sk_f[sk_idx_f] = 1.0
                     final_gap = final_grad.T @ (final_w - sk_f) if np.all(np.isfinite(final_grad)) else np.nan
                else: final_gap = np.nan
            except Exception as e_grad:
                 warnings.warn(f"Final gradient calculation failed after max_iter: {e_grad}")
                 final_grad = np.full(M, np.nan)
                 final_gap = np.nan
        else:
            warnings.warn(f"Warning: Final inner QP solve failed after max_iter ({msg_f}). Using values from last iteration.")
    except Exception as e_final:
        warnings.warn(f"Error during final consistency check after max_iter: {e_final}. Using values from last iteration.")

    # Add final state to history if requested
    if return_history:
        grad_norm_final = np.linalg.norm(final_grad[np.isfinite(final_grad)]) if np.any(np.isfinite(final_grad)) else np.nan
        if grad_norm_final < tolerance * 100: sk_idx_final = -1
        else: sk_idx_final = np.nanargmax(np.nan_to_num(final_grad, nan=-np.inf)) if np.any(np.isfinite(final_grad)) else -1
        history.append({
            'k': max_outer_iter + 1, 'w_k': np.copy(final_w), 'H_k': final_H,
            'grad_H_k_norm': grad_norm_final, 's_k_index': sk_idx_final, 'fw_gap': final_gap,
            'gamma_k': np.nan, 'pi_k': np.copy(final_pi), 'lam_k': np.copy(final_lam), 'active_set_k': final_active_set
        })

    # Return result indicating max iterations reached
    result = OptimizationResult(True, f"Max Iter ({max_outer_iter}) reached", w_opt=final_w, pi_opt=final_pi, lambda_opt=final_lam,
                                H_opt=final_H, grad_H_opt=final_grad, iterations=max_outer_iter,
                                fw_gap=final_gap, active_set_opt=final_active_set)
    return (result, history) if return_history else result

# --- generate_params_profile_switching_symmetric 関数 (対称 + Vm微小差) ---
def generate_params_profile_switching_symmetric(alpha, alpha_max, K=5, M=3,
                                      R_base_sym=np.array([0.02, 0.01, 0.0, -0.01, -0.02]),
                                      sigma_base=np.array([0.18, 0.15, 0.20, 0.12, 0.10]),
                                      Corr_base=np.array([[1.0, 0.4, 0.3, 0.2, 0.1], [0.4, 1.0, 0.5, 0.3, 0.2], [0.3, 0.5, 1.0, 0.6, 0.4], [0.2, 0.3, 0.6, 1.0, 0.5], [0.1, 0.2, 0.4, 0.5, 1.0]]),
                                      r_offset=0.03,
                                      # ★★★ Vm にわずかな差をつける設定 ★★★
                                      s_factor_g = 0.001, # good/neutral vs base variance factor
                                      s_factor_b = -0.001, # bad vs base variance factor
                                      # ★★★★★★★★★★★★★★★★★★★★★★
                                      sigma_min_epsilon=1e-4, psd_tolerance=1e-9):
    """ Generates quasi-symmetric parameters with slight differences in Vm """
    assert K == len(R_base_sym) and K == len(sigma_base) and K == Corr_base.shape[0] and M == 3, "Dimension mismatch"

    # --- Define base profiles (Good, Neutral, Bad) ---
    R_neutral = R_base_sym
    R_good = R_base_sym + r_offset
    R_bad = R_base_sym - r_offset

    # Ensure base correlation matrix is PSD
    Corr_neutral_psd = make_psd(Corr_base, psd_tolerance)

    # Base covariance for Neutral profile
    sigma_neutral_eff = np.maximum(sigma_min_epsilon, sigma_base)
    V_neutral = np.diag(sigma_neutral_eff) @ Corr_neutral_psd @ np.diag(sigma_neutral_eff)

    # Covariances with slight differences for Good and Bad profiles
    # Good profile: Slightly higher volatility
    sigma_good_adj = np.maximum(sigma_min_epsilon, sigma_base * (1 + s_factor_g))
    V_good = np.diag(sigma_good_adj) @ Corr_neutral_psd @ np.diag(sigma_good_adj)

    # Bad profile: Slightly lower volatility (or adjust factors as needed)
    sigma_bad_adj = np.maximum(sigma_min_epsilon, sigma_base * (1 + s_factor_b))
    V_bad = np.diag(sigma_bad_adj) @ Corr_neutral_psd @ np.diag(sigma_bad_adj)
    # --- End base profiles ---

    # --- Calculate Second Moments (Sigma^m = V^m + r^m * r^m^T) ---
    Sigma_neutral = V_neutral + np.outer(R_neutral, R_neutral)
    Sigma_good = V_good + np.outer(R_good, R_good)
    Sigma_bad = V_bad + np.outer(R_bad, R_bad)
    # --- End Second Moments ---

    # --- Apply profile switching based on alpha ---
    beta = np.clip(alpha / alpha_max if alpha_max > 0 else (1.0 if alpha > 0 else 0.0), 0.0, 1.0)

    R_alpha = np.zeros((K, M))
    SecondMoments_a_array = np.zeros((M, K, K))

    # Scenario 0 (m=0): Mix Good and Neutral (alpha moves from Good towards Neutral)
    R_alpha[:, 0] = (1 - beta) * R_good + beta * R_neutral
    SecondMoments_a_array[0, :, :] = (1 - beta) * Sigma_good + beta * Sigma_neutral

    # Scenario 1 (m=1): Mix Bad and Good (alpha moves from Bad towards Good)
    R_alpha[:, 1] = (1 - beta) * R_bad + beta * R_good
    SecondMoments_a_array[1, :, :] = (1 - beta) * Sigma_bad + beta * Sigma_good

    # Scenario 2 (m=2): Mix Neutral and Bad (alpha moves from Neutral towards Bad)
    R_alpha[:, 2] = (1 - beta) * R_neutral + beta * R_bad
    SecondMoments_a_array[2, :, :] = (1 - beta) * Sigma_neutral + beta * Sigma_bad
    # --- End profile switching ---

    # Final check for validity
    if not np.all(np.isfinite(R_alpha)): raise ValueError("Generated R_alpha contains NaN/Inf.")
    if not np.all(np.isfinite(SecondMoments_a_array)): raise ValueError("Generated SecondMoments_a_array contains NaN/Inf.")

    return R_alpha, SecondMoments_a_array


# === α依存性分析関数 (ActiveSet法を使用) ===
def analyze_alpha_full_results(alpha_range, param_gen_kwargs, optimizer_kwargs, mu_tilde):
    """ 指定された alpha 範囲で FW解、H*、pi*、grad H*、lambda*、active_set* を追跡する """
    results_over_alpha = []
    K = param_gen_kwargs.get('K', 5)
    M = param_gen_kwargs.get('M', 3)
    if K is None or M is None:
         raise ValueError("K and M must be specified in param_gen_kwargs")

    print(f"\n--- Starting Full Results Analysis over Alpha Range [{alpha_range[0]:.4f}, {alpha_range[-1]:.4f}] ---")
    print(f"--- (Using Active Set Method, mu_tilde={mu_tilde}) ---")
    total_alphas = len(alpha_range)
    start_loop_time = time.time()
    successful_runs = 0
    failed_alphas = []

    for idx, alpha in enumerate(alpha_range):
        loop_start_time = time.time()
        print(f"\rAnalyzing alpha = {alpha:.6f} ({idx+1}/{total_alphas}) ... ", end="")

        alpha_result = {'alpha': alpha}
        fw_success = False
        fw_message = "Analysis not run"
        w_fw_default = np.full(M, np.nan)
        H_star_fw = np.nan
        pi_opt = np.full(K, np.nan)
        grad_H_opt = np.full(M, np.nan)
        lambda_opt = np.full(M, np.nan)
        active_set_opt = tuple()
        fw_gap_final = np.nan
        iterations_final = 0

        try:
            # === パラメータ生成 ===
            R_alpha, SecondMoments_alpha_array = generate_params_profile_switching_symmetric(alpha, **param_gen_kwargs)
            # Check parameter validity immediately
            if not np.all(np.isfinite(R_alpha)) or not np.all(np.isfinite(SecondMoments_alpha_array)):
                 raise ValueError("Parameter generation resulted in NaN/Inf.")

            # --- FW Optimizer呼び出し (Active Setを使用) ---
            fw_result = frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde,
                                              initial_w=None, return_history=False,
                                              debug_print=False, # Keep debug off for loops
                                              **optimizer_kwargs)

            fw_success = fw_result.success
            fw_message = fw_result.message
            iterations_final = fw_result.iterations if fw_result.iterations is not None else 0

            if fw_result.success:
                 w_fw_default = fw_result.w_opt if fw_result.w_opt is not None else np.full(M, np.nan)
                 H_star_fw = fw_result.H_opt if fw_result.H_opt is not None else np.nan
                 pi_opt = fw_result.pi_opt if fw_result.pi_opt is not None else np.full(K, np.nan)
                 grad_H_opt = fw_result.grad_H_opt if fw_result.grad_H_opt is not None else np.full(M, np.nan)
                 lambda_opt = fw_result.lambda_opt if fw_result.lambda_opt is not None else np.full(M, np.nan)
                 active_set_opt = fw_result.active_set_opt if fw_result.active_set_opt is not None else tuple()
                 fw_gap_final = fw_result.fw_gap if fw_result.fw_gap is not None else np.nan
                 successful_runs += 1
            else:
                 # Store NaN for results if FW failed, but still record alpha
                 w_fw_default = np.full(M, np.nan)
                 H_star_fw = np.nan
                 pi_opt = np.full(K, np.nan)
                 grad_H_opt = np.full(M, np.nan)
                 lambda_opt = np.full(M, np.nan)
                 active_set_opt = tuple()
                 fw_gap_final = np.nan
                 failed_alphas.append(alpha)
                 # Print failure message immediately
                 print(f"\n  FW failed for alpha={alpha:.6f}: {fw_result.message}")


        except Exception as e:
            print(f"\n  Error during analysis for alpha={alpha:.6f}: {e}")
            # Print traceback for detailed debugging
            # traceback.print_exc()
            fw_success = False
            fw_message = f"Exception: {e}"
            w_fw_default = np.full(M, np.nan)
            H_star_fw = np.nan
            pi_opt = np.full(K, np.nan)
            grad_H_opt = np.full(M, np.nan)
            lambda_opt = np.full(M, np.nan)
            active_set_opt = tuple()
            fw_gap_final = np.nan
            failed_alphas.append(alpha)

        # Store results for this alpha value
        alpha_result['success'] = fw_success
        alpha_result['message'] = fw_message
        alpha_result['w_fw_default'] = w_fw_default
        alpha_result['H_star'] = H_star_fw
        alpha_result['pi_opt'] = pi_opt
        alpha_result['grad_H_opt'] = grad_H_opt
        alpha_result['lambda_opt'] = lambda_opt
        alpha_result['active_set_opt'] = active_set_opt
        alpha_result['fw_gap'] = fw_gap_final
        alpha_result['iterations'] = iterations_final

        results_over_alpha.append(alpha_result)
        loop_end_time = time.time()
        # Optional: print timing per loop
        # print(f" (took {loop_end_time - loop_start_time:.2f}s)")

    # --- Post-loop summary ---
    end_loop_time = time.time()
    print(f"\n--- Finished Full Results Analysis ({total_alphas} points) ---")
    print(f"Total time: {end_loop_time - start_loop_time:.2f} seconds")
    print(f"Successful runs: {successful_runs}/{total_alphas}")
    if failed_alphas:
        print(f"Failed alphas: {failed_alphas}")

    # --- Convert to DataFrame ---
    if not PANDAS_AVAILABLE:
        print("Pandas not available. Returning results as list of dictionaries.")
        return results_over_alpha

    df_results = pd.DataFrame(results_over_alpha)

    # Expand vector columns if they exist and are not all NaN
    if 'w_fw_default' in df_results.columns:
        fw_vecs = np.stack(df_results['w_fw_default'].fillna(pd.Series([np.full(M, np.nan)] * len(df_results))).values)
        if np.any(np.isfinite(fw_vecs)):
            for m in range(M): df_results[f'w_fw_{m}'] = fw_vecs[:, m]

    if 'pi_opt' in df_results.columns:
        pi_vecs = np.stack(df_results['pi_opt'].fillna(pd.Series([np.full(K, np.nan)] * len(df_results))).values)
        if np.any(np.isfinite(pi_vecs)):
            for k in range(K): df_results[f'pi_{k}'] = pi_vecs[:, k]

    if 'grad_H_opt' in df_results.columns:
        grad_vecs = np.stack(df_results['grad_H_opt'].fillna(pd.Series([np.full(M, np.nan)] * len(df_results))).values)
        if np.any(np.isfinite(grad_vecs)):
            for m in range(M): df_results[f'grad_H_{m}'] = grad_vecs[:, m]

    if 'lambda_opt' in df_results.columns:
        lambda_vecs = np.stack(df_results['lambda_opt'].fillna(pd.Series([np.full(M, np.nan)] * len(df_results))).values)
        if np.any(np.isfinite(lambda_vecs)):
            for m in range(M): df_results[f'lambda_{m}'] = lambda_vecs[:, m]

    # Convert active set tuple to string for display/CSV
    if 'active_set_opt' in df_results.columns:
        df_results['active_set'] = df_results['active_set_opt'].apply(lambda x: str(x) if x is not None else '()')

    # Drop original vector/tuple columns
    cols_to_drop = ['w_fw_default', 'pi_opt', 'grad_H_opt', 'lambda_opt', 'active_set_opt']
    df_results = df_results.drop(columns=[col for col in cols_to_drop if col in df_results.columns], errors='ignore')

    return df_results


# === メイン実行ブロック ===
if __name__ == '__main__':
    start_time_main = time.time()
    # Set warning filters
    warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered') # Ignore common numpy warnings
    warnings.filterwarnings('ignore', category=UserWarning) # Ignore other user warnings
    warnings.filterwarnings('ignore', category=OptimizeWarning) # Ignore SciPy Optimize warnings

    # --- 設定 ---
    K = 5; M = 3;
    # ★★★ mu_tilde を 0.025 に設定 ★★★
    mu_tilde = 0.025
    alpha_max = 1.5

    # ★★★ 既存の準対称パラメータ設定を使用 ★★★
    # (generate_params_profile_switching_symmetric で Vm に微小差分がつく)
    param_gen_kwargs_quasi_symmetric = {
        'K': K, 'M': M, 'alpha_max': alpha_max,
        'R_base_sym': np.array([0.02, 0.01, 0.0, -0.01, -0.02]), # Neutral=0中心
        'sigma_base': np.array([0.18, 0.15, 0.20, 0.12, 0.10]),
        'Corr_base': np.array([[1.0, 0.4, 0.3, 0.2, 0.1], [0.4, 1.0, 0.5, 0.3, 0.2], [0.3, 0.5, 1.0, 0.6, 0.4], [0.2, 0.3, 0.6, 1.0, 0.5], [0.1, 0.2, 0.4, 0.5, 1.0]]),
        'r_offset': 0.03,
        's_factor_g': 0.001,  # Vm にわずかな差をつける
        's_factor_b': -0.001, # Vm にわずかな差をつける
        'sigma_min_epsilon': 1e-4,
        'psd_tolerance': 1e-9
    }

    # FW Optimizer と Inner QP の設定 (前回と同様)
    solver_settings = {
        'max_outer_iter': 500, 'fw_gap_tol': 1e-9,
        'inner_max_iter': 600, # Increase inner iterations slightly
        'tolerance': 1e-11,   # Numerical tolerances
        'psd_make_tolerance': 1e-9,
        'qp_regularization': 1e-10, # Regularization for inner QP
        'force_iterations': 50 # Run at least 50 iterations before checking FW gap
    }

    # ★★★ 分析する alpha の範囲 (0.75 付近に絞る) ★★★
    alpha_start = 0.700
    alpha_end = 0.800
    num_alpha_steps = 101 # 約 0.001 ステップ
    alpha_range_analyze = np.linspace(alpha_start, alpha_end, num_alpha_steps)

    # KKTチェック用許容誤差
    unique_pt_tolerance = 1e-4 # Threshold to consider w_m > 0 for KKT check

    # 出力ファイル名
    output_csv_filename = f"alpha_internal_sol_search_{alpha_start:.3f}_{alpha_end:.3f}_activeset_mu{int(mu_tilde*1000):03d}_quasi_sym.csv"

    print("-" * 70)
    print(f"--- Searching for Internal Solution around alpha=0.75 [{alpha_start:.3f}, {alpha_end:.3f}] ---")
    print(f"--- (Using ActiveSet, mu_tilde={mu_tilde}, Quasi-Symmetric Params) ---")
    print(f"--- Output CSV: {output_csv_filename} ---")
    print("-" * 70)

    # --- 分析実行 ---
    results_df = analyze_alpha_full_results(
        alpha_range=alpha_range_analyze,
        param_gen_kwargs=param_gen_kwargs_quasi_symmetric, # <-- 準対称設定を使用
        optimizer_kwargs=solver_settings,
        mu_tilde=mu_tilde # <-- 新しい mu_tilde を使用
    )

    # --- 結果表示 & CSV保存 & KKTチェック ---
    if PANDAS_AVAILABLE and isinstance(results_df, pd.DataFrame) and not results_df.empty:
        print("\n--- Analysis Results Summary (DataFrame) ---")
        # Select and order columns for display
        cols_display = ['alpha', 'success', 'H_star', 'fw_gap', 'iterations', 'active_set']
        cols_w = [f'w_fw_{m}' for m in range(M) if f'w_fw_{m}' in results_df.columns]
        cols_pi = [f'pi_{k}' for k in range(K) if f'pi_{k}' in results_df.columns]
        cols_grad = [f'grad_H_{m}' for m in range(M) if f'grad_H_{m}' in results_df.columns]
        cols_lam = [f'lambda_{m}' for m in range(M) if f'lambda_{m}' in results_df.columns]
        cols_to_show = cols_display + cols_w + cols_grad #+ cols_pi + cols_lam # Keep output concise
        cols_to_show = [col for col in cols_to_show if col in results_df.columns]

        # Display settings
        float_format_func = lambda x: f"{x:.4e}" if pd.notna(x) and isinstance(x, (float, np.number)) else x
        pd.options.display.float_format = float_format_func
        # Print the results DataFrame to console
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 200):
             print(results_df[cols_to_show].to_string(index=False, na_rep='NaN'))
        pd.reset_option('display.float_format') # Reset float format

        # --- CSVファイルに保存 ---
        try:
            # Order columns for CSV saving (more comprehensive)
            cols_csv_order = ['alpha', 'success', 'message', 'H_star', 'fw_gap', 'iterations', 'active_set'] + \
                             cols_w + cols_pi + cols_grad + cols_lam
            cols_csv_order = [col for col in cols_csv_order if col in results_df.columns]
            df_to_save = results_df[cols_csv_order]

            df_to_save.to_csv(output_csv_filename, index=False, float_format='%.8e')
            print(f"\nFull detailed results ({len(df_to_save)} points) saved to: {os.path.abspath(output_csv_filename)}")
        except Exception as e:
            print(f"\nError saving results to CSV: {e}")
        # --- End CSV Save ---

        # --- KKT条件のチェック (簡易版) ---
        print("\n--- Quick KKT Check (Stationarity w.r.t. w) ---")
        kkt_violations = []
        check_tol = solver_settings.get('tolerance', 1e-9) * 1e3 # Tolerance for KKT checks

        for index, row in results_df.iterrows():
            alpha = row['alpha']
            if not row.get('success', False): # Skip failed runs
                 kkt_violations.append({
                    'alpha': alpha, 'active_set_report': row.get('active_set', 'N/A'), 'active_set_kkt': 'N/A',
                    'max_violation': np.nan, 'grad_consistency_violation': np.nan, 'check_result': 'FW Fail'
                 })
                 continue

            w_star = np.array([row.get(f'w_fw_{m}', np.nan) for m in range(M)])
            grad_h = np.array([row.get(f'grad_H_{m}', np.nan) for m in range(M)])
            active_set_str = row.get('active_set', '()')

            # Check if w* or grad_h are valid
            if np.isnan(w_star).any() or np.isnan(grad_h).any() or np.isclose(np.sum(w_star), 0):
                kkt_violations.append({
                    'alpha': alpha, 'active_set_report': active_set_str, 'active_set_kkt': 'NaN',
                    'max_violation': np.nan, 'grad_consistency_violation': np.nan, 'check_result': 'Skipped (NaN/zero w/grad)'
                })
                continue

            # Identify numerically active weights for KKT check
            active_indices_kkt = set(m for m in range(M) if w_star[m] > unique_pt_tolerance)

            # Check 1: Consistency of gradients for active weights
            grad_consistency_violation = 0.0
            if len(active_indices_kkt) > 1:
                 active_grads = grad_h[list(active_indices_kkt)]
                 grad_consistency_violation = np.max(active_grads) - np.min(active_grads)

            # Check 2: Optimality condition for inactive weights
            # Estimate nu* = max(active_grads) or max(all grads) if none active
            nu_estimate = np.max(grad_h[list(active_indices_kkt)]) if active_indices_kkt else np.max(grad_h)
            stationarity_violation = grad_consistency_violation # Start with consistency violation

            for m in range(M):
                if m not in active_indices_kkt:
                    # Check if grad_h[m] <= nu_estimate (within tolerance)
                    violation = grad_h[m] - nu_estimate
                    if violation > check_tol: # If inactive grad is significantly larger than nu_estimate
                        stationarity_violation = max(stationarity_violation, violation)

            # Check Result based on violations
            is_internal = len(active_indices_kkt) == M
            result_str = 'OK'
            if stationarity_violation > check_tol * 10: result_str = 'WARN (KKT Viol)'
            if is_internal: result_str += ' [INTERNAL]'
            elif len(active_indices_kkt) == 1: result_str += ' [VERTEX]'
            elif len(active_indices_kkt) == 2: result_str += ' [EDGE]'


            kkt_violations.append({
                'alpha': alpha,
                'active_set_report': active_set_str,
                'active_set_kkt': str(tuple(sorted(active_indices_kkt))),
                'max_violation': stationarity_violation,
                'grad_consistency_violation': grad_consistency_violation,
                'check_result': result_str
            })

        # Display KKT Check Results
        if PANDAS_AVAILABLE and kkt_violations:
            df_kkt = pd.DataFrame(kkt_violations)
            kkt_cols_order = ['alpha', 'active_set_kkt', 'max_violation', 'grad_consistency_violation', 'check_result']
            #kkt_cols_order = ['alpha', 'active_set_report','active_set_kkt', 'max_violation', 'grad_consistency_violation', 'check_result'] # More verbose option
            kkt_cols_order = [col for col in kkt_cols_order if col in df_kkt.columns]
            with pd.option_context('display.float_format', '{:.4e}'.format, 'display.max_rows', None): # All rows
                print(df_kkt[kkt_cols_order].to_string(index=False, na_rep='NaN'))
            print("\nNotes on KKT Check:")
            # print(" - active_set_report: Active set reported by inner solver for pi*") # Verbose option
            print(" - active_set_kkt: Active set inferred from w* > tolerance for outer problem")
            print(" - max_violation: Max KKT violation (max(grad_consistency, max(grad_inactive - nu_est)))")
            print(" - grad_consistency_violation: max(active_grads) - min(active_grads) for w_m > tol")
            print(" - check_result: OK/WARN indicates KKT satisfaction; [INTERNAL/EDGE/VERTEX] indicates solution type.")
        else:
            print("Could not perform KKT check or pandas not available.")
        # --- End KKT Check ---

    elif isinstance(results_df, list) or (isinstance(results_df, pd.DataFrame) and results_df.empty):
        print("\n--- Analysis Results Summary ---")
        print("No results generated or DataFrame is empty.")
        if not PANDAS_AVAILABLE: print("(Pandas not available, skipping CSV export)")

    end_time_main = time.time()
    print(f"\nTotal execution time: {end_time_main - start_time_main:.2f} seconds")

----------------------------------------------------------------------
--- Searching for Internal Solution around alpha=0.75 [0.700, 0.800] ---
--- (Using ActiveSet, mu_tilde=0.025, Quasi-Symmetric Params) ---
--- Output CSV: alpha_internal_sol_search_0.700_0.800_activeset_mu025_quasi_sym.csv ---
----------------------------------------------------------------------

--- Starting Full Results Analysis over Alpha Range [0.7000, 0.8000] ---
--- (Using Active Set Method, mu_tilde=0.025) ---
Analyzing alpha = 0.700000 (1/101) ... 
  FW failed for alpha=0.700000: Outer iter 2: Inner QP failed: QP fail:Iter 1: KKT solve failed. ActiveSet=[0, 1, 2]. Cond(Q)=1.2e+01. Stopping.
Analyzing alpha = 0.701000 (2/101) ... 
  FW failed for alpha=0.701000: Outer iter 2: Inner QP failed: QP fail:Iter 1: KKT solve failed. ActiveSet=[0, 1, 2]. Cond(Q)=1.2e+01. Stopping.
Analyzing alpha = 0.702000 (3/101) ... 
  FW failed for alpha=0.702000: Outer iter 9: Inner QP failed: QP fail:Iter 1: KKT solve failed. 

In [6]:
# -*- coding: utf-8 -*-
"""
5_2_find_internal_solution_mu022_reg_quasi_symmetric.py

Attempts to find an internal solution w* by using quasi-symmetric parameters
around alpha=0.75, testing mu_tilde = 0.022 and increased QP regularization (1e-9).
Uses the Active Set method for the inner QP solve.
"""

import numpy as np
import matplotlib.pyplot as plt
import warnings
from scipy.linalg import solve, LinAlgError, eigh, inv, qr, lstsq, pinv # Import lstsq, pinv for potential fallback
from scipy.optimize import linprog, OptimizeWarning
import traceback
import time
import itertools
import os
import math
from collections import Counter # オプション

# pandas がなければインストールを促す
try:
    import pandas as pd
    PANDAS_AVAILABLE = True
    pd.set_option('display.width', 180) # 表示幅を少し広げる
    pd.set_option('display.float_format', '{:.6e}'.format) # 科学技術表記
    pd.set_option('display.max_rows', 300) # 表示行数を増やす
except ImportError:
    PANDAS_AVAILABLE = False
    print("Warning: pandas library not found. Output formatting will be basic. CSV export disabled.")

# === ここから必要な関数定義 ===
DEFAULT_TOLERANCE = 1e-9 # General tolerance

# --- OptimizationResult クラス (変更なし) ---
class OptimizationResult:
    def __init__(self, success, message, w_opt=None, pi_opt=None, lambda_opt=None,
                 H_opt=None, grad_H_opt=None, iterations=None, fw_gap=None,
                 active_set_opt=None):
        self.success = success; self.message = message; self.w_opt = w_opt; self.pi_opt = pi_opt
        self.lambda_opt = lambda_opt; self.H_opt = H_opt; self.grad_H_opt = grad_H_opt
        self.iterations = iterations; self.fw_gap = fw_gap
        self.active_set_opt = active_set_opt

# --- find_feasible_initial_pi 関数 (失敗時メッセージ改善) ---
def find_feasible_initial_pi(R, mu_tilde, K, tolerance=1e-8):
    M = R.shape[1]; c = np.zeros(K + 1); c[K] = 1.0
    A_ub = np.hstack((-R.T, -np.ones((M, 1)))); b_ub = -mu_tilde * np.ones(M)
    bounds = [(None, None)] * K + [(0, None)]; opts = {'tol': tolerance, 'disp': False, 'presolve': True}
    result = None; methods_to_try = ['highs', 'highs-ipm', 'highs-ds', 'simplex']
    last_method_tried = 'None'
    for method in methods_to_try:
        last_method_tried = method
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", OptimizeWarning)
                result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method=method, options=opts)
            if result.success: break
        except ValueError as e:
             # print(f"DEBUG Phase 1 ({method}) ValueError: {e}") # Debug print
             continue
        except Exception as e:
            # print(f"\n  Phase 1 LP Exception ({method}): {e}") # Debug print
            return None, False, f"Phase 1 LP failed ({method}): {e}"
    if result is None or not result.success:
        msg = result.message if result else "No solver succeeded"
        status = result.status if result else -1
        # Add more context to the failure message
        max_expected_return = -np.inf
        try: # Estimate max return with equal weights if possible
            if K > 0 and M > 0 and R is not None and np.all(np.isfinite(R)):
                 equal_pi = np.ones(K) / K
                 expected_returns = equal_pi @ R
                 if np.all(np.isfinite(expected_returns)):
                      max_expected_return = np.max(expected_returns)
        except: pass # Ignore errors during estimation

        fail_reason = f"Phase 1 LP solver failed: {msg} (status={status}, method={last_method_tried})"
        if np.isfinite(max_expected_return) and mu_tilde > max_expected_return + tolerance: # Check if mu_tilde might be too high
             fail_reason += f" - mu_tilde ({mu_tilde:.4f}) > max_equal_weight_return ({max_expected_return:.4f})?"
        return None, False, fail_reason

    s = result.x[K]; pi = result.x[:K]
    if np.isnan(pi).any(): return None, False, "Phase 1 LP resulted in NaN values for pi."
    G = -R.T; h = -mu_tilde * np.ones(M); violation = np.max(G @ pi - h)
    feasibility_tol = tolerance * 10000
    if s <= tolerance * 1000: # Increased tolerance for s slightly
        if violation <= feasibility_tol:
             return pi, True, f"Phase 1 OK (s*={s:.1e}, max_viol={violation:.1e})"
        else:
             return pi, True, f"Phase 1 OK (s*={s:.1e}), WARN: Initial violation {violation:.1e} > {feasibility_tol:.1e}"
    else:
        return None, False, f"Phase 1 Likely Infeasible (s* = {s:.1e} > {tolerance*1000:.1e}), max_viol={violation:.1e}"


# --- solve_kkt_system 関数 (KKT行列条件数チェック追加) ---
def solve_kkt_system(Q, G_W, g, tolerance=DEFAULT_TOLERANCE):
    K = Q.shape[0]; n_act = G_W.shape[0] if G_W is not None and G_W.ndim == 2 and G_W.shape[0] > 0 else 0
    kkt_cond = np.nan # Initialize KKT condition number

    # Input checks
    if not np.all(np.isfinite(Q)) or not np.all(np.isfinite(g)): return None, None, False, np.nan
    if n_act > 0 and (G_W is None or not np.all(np.isfinite(G_W))): return None, None, False, np.nan

    if n_act == 0:
        try:
            # Calculate condition number before solving
            try: kkt_cond = np.linalg.cond(Q)
            except LinAlgError: kkt_cond = np.inf

            p = solve(Q, -g, assume_a='sym', check_finite=False)
            l = np.array([])
        except LinAlgError: return None, None, False, kkt_cond # Return condition number even on failure
        except ValueError: return None, None, False, kkt_cond
        if p is None or np.isnan(p).any() or np.isinf(p).any(): return None, None, False, kkt_cond
        res_norm = np.linalg.norm(Q @ p + g); g_norm = np.linalg.norm(g)
        solved_ok = res_norm <= tolerance * 1e4 * (1 + g_norm)
        return p, l, solved_ok, kkt_cond
    else:
        kkt_mat = None; rhs = None
        try:
            if G_W.ndim != 2 or G_W.shape[1] != K: return None, None, False, kkt_cond
            kkt_mat = np.block([[Q, G_W.T], [G_W, np.zeros((n_act, n_act))]])
            rhs = np.concatenate([-g, np.zeros(n_act)])
            # Calculate condition number of KKT matrix (can be expensive)
            try: kkt_cond = np.linalg.cond(kkt_mat)
            except LinAlgError: kkt_cond = np.inf # Handle case where cond itself fails
        except ValueError: return None, None, False, kkt_cond
        except Exception as e: # Catch any other error during block creation
            print(f"DEBUG KKT block matrix creation error: {e}")
            return None, None, False, kkt_cond

        try:
            sol = solve(kkt_mat, rhs, assume_a='sym', check_finite=False)
            p = sol[:K]; l = sol[K:]
        # Keep returning kkt_cond on failure
        except LinAlgError: return None, None, False, kkt_cond
        except ValueError: return None, None, False, kkt_cond
        except Exception as e:
             print(f"DEBUG KKT solve unexpected error: {e}")
             return None, None, False, kkt_cond

        if sol is None or np.isnan(sol).any() or np.isinf(sol).any():
             return None, None, False, kkt_cond

        # Check residual norm after solve
        res_norm = np.linalg.norm(kkt_mat @ sol - rhs); rhs_norm = np.linalg.norm(rhs)
        solved_ok = res_norm <= tolerance * 1e4 * (1 + rhs_norm)
        return p, l, solved_ok, kkt_cond


# --- solve_inner_qp_active_set 関数 (KKT失敗時の情報追加, 初期ActiveSet修正) ---
def solve_inner_qp_active_set(Vw, R, mu_tilde, initial_pi, max_iter=350, tolerance=DEFAULT_TOLERANCE, regularization_epsilon=1e-10):
    K = Vw.shape[0]; M = R.shape[1];
    if not np.all(np.isfinite(Vw)): return None, None, None, None, None, False, "QP fail: Vw contains NaN/Inf."

    # Regularization (using the increased value passed from solver_settings)
    Q_reg = 2 * Vw + 2 * regularization_epsilon * np.eye(K)
    q_cond = np.inf
    try: q_cond = np.linalg.cond(Q_reg)
    except LinAlgError: pass # Keep q_cond as inf if cond fails
    if q_cond > 1 / tolerance:
        warnings.warn(f"QP Warning: Condition number of Q_reg is high ({q_cond:.2e}). Regularization epsilon={regularization_epsilon:.1e}")

    G = -R.T; h = -mu_tilde * np.ones(M)

    if initial_pi is None or not np.all(np.isfinite(initial_pi)):
        return None, None, None, None, None, False, "QP fail: Initial pi is None or contains NaN/Inf."

    pi_k = np.copy(initial_pi)
    lam_opt = np.zeros(M)
    W = set() # Active set

    active_tol = tolerance * 100 # Tolerance for constraint activation

    # --- Initial Active Set Determination ---
    initial_violations = G @ pi_k - h
    max_initial_violation = np.max(initial_violations)
    # If initial pi is significantly infeasible, start with only clearly violated constraints active
    if max_initial_violation > active_tol * 10:
        warnings.warn(f"QP Warning: Initial pi infeasible (max viol: {max_initial_violation:.2e} > {active_tol * 10:.1e}).")
        # Only activate constraints violated by more than active_tol
        W = set(j for j, viol in enumerate(initial_violations) if viol > active_tol)
        # print(f"DEBUG: Infeasible initial pi. Starting Active Set: {sorted(list(W))}")
    else:
        # If initial pi is feasible or near feasible, activate constraints "close" to the boundary
        W = set(j for j, viol in enumerate(initial_violations) if viol > -active_tol)
    # --- End Initial Active Set Determination ---

    active_indices_opt = None

    for i in range(max_iter):
        g_k = Q_reg @ pi_k
        if not np.all(np.isfinite(g_k)): return None, None, None, None, None, False, f"QP fail:Iter {i+1}: Gradient g_k contains NaN/Inf."

        act = sorted(list(W))
        n_act = len(act)
        G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))

        # Solve KKT system
        p_k, lam_Wk, solved, kkt_cond = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)

        if not solved or p_k is None:
            # Include condition numbers in the failure message
            return None, None, None, None, None, False, f"QP fail:Iter {i+1}: KKT solve failed. ActiveSet={act}. Q_cond={q_cond:.1e}, KKT_cond={kkt_cond:.1e}"

        p_norm = np.linalg.norm(p_k); pi_k_norm = np.linalg.norm(pi_k)
        # Check convergence: If search direction p_k is close to zero
        if p_norm <= tolerance * 100 * (1 + pi_k_norm):
            is_optimal_point = True; blocking_constraint_idx = -1; min_negative_lambda = float('inf')
            dual_feas_tol = -tolerance * 100 # Allow slightly negative lambda

            if W: # Check dual feasibility if constraints are active
                if lam_Wk is None or len(lam_Wk) != n_act: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: lam_Wk inconsistent? ActiveSet={act}"
                if not np.all(np.isfinite(lam_Wk)): return None, None, None, None, None, False, f"QP fail:Iter {i+1}: lam_Wk contains NaN/Inf. ActiveSet={act}"
                lambda_map = dict(zip(act, lam_Wk))
                for constraint_idx, lagrange_multiplier in lambda_map.items():
                    if lagrange_multiplier < dual_feas_tol:
                        is_optimal_point = False
                        if lagrange_multiplier < min_negative_lambda:
                            min_negative_lambda = lagrange_multiplier; blocking_constraint_idx = constraint_idx

            if is_optimal_point: # Optimal solution found
                lam_opt.fill(0.0)
                if W and lam_Wk is not None:
                     try: lam_opt[act] = np.maximum(lam_Wk, 0)
                     except IndexError: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: Index error assign lambda. ActiveSet={act}"
                final_infeas = np.max(G @ pi_k - h)
                msg = f"Optimal found at iter {i+1}."
                if final_infeas > active_tol: msg += f" (WARN: Final violation {final_infeas:.1e})"
                active_indices_opt = act
                return pi_k, lam_opt, active_indices_opt, None, Q_reg / 2.0, True, msg
            else: # Not optimal: remove constraint with most negative multiplier
                if blocking_constraint_idx in W:
                    W.remove(blocking_constraint_idx); continue
                else: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: Neg lambda idx {blocking_constraint_idx}, not in W={act}"
        else: # Non-zero search direction p_k: Move along p_k
            alpha_k = 1.0; blocking_constraint_idx = -1; min_step_length = float('inf')
            step_tol = tolerance * 100

            # Find maximum step length alpha_k before hitting an inactive constraint
            for j in range(M):
                if j not in W:
                    constraint_gradient_dot_p = G[j, :] @ p_k
                    if constraint_gradient_dot_p > step_tol: # Potential to violate constraint j
                        distance_to_boundary = h[j] - (G[j, :] @ pi_k)
                        if abs(constraint_gradient_dot_p) > 1e-15: # Avoid division by zero
                             alpha_j = distance_to_boundary / constraint_gradient_dot_p
                             if np.isfinite(alpha_j) and alpha_j >= -tolerance:
                                 step_j = max(0.0, alpha_j)
                                 if step_j < min_step_length:
                                     min_step_length = step_j; blocking_constraint_idx = j

            alpha_k = min(1.0, min_step_length) # Determine step length

            # Safeguard for tiny steps (might indicate cycling or slow progress)
            if alpha_k * p_norm < tolerance * (1 + pi_k_norm):
                 # warnings.warn(f"QP Iter {i+1}: Tiny step size ({alpha_k:.1e}). May indicate slow progress or cycling.")
                 # Decide whether to stop or continue if step is negligible
                 # Let's continue for now, but monitor if this causes issues.
                 pass

            # Update solution
            pi_k += alpha_k * p_k
            if not np.all(np.isfinite(pi_k)):
                 return None, None, None, None, None, False, f"QP fail:Iter {i+1}: pi_k became NaN/Inf after step."

            # Add blocking constraint to active set if step was blocked
            if alpha_k < 1.0 - step_tol and blocking_constraint_idx != -1:
                if blocking_constraint_idx not in W: W.add(blocking_constraint_idx)

            continue # Continue to next iteration

    # Max iterations reached
    msg = f"Max iter ({max_iter}) reached."
    final_infeas = np.max(G @ pi_k - h)
    if final_infeas > active_tol * 100:
        return None, None, None, None, None, False, f"{msg} Final infeasible ({final_infeas:.1e} > {active_tol*100:.1e}). ActiveSet={sorted(list(W))}"

    # Assess optimality at final point
    act = sorted(list(W)); n_act = len(act)
    G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))
    is_likely_optimal = False; active_constraints_opt = act

    g_k = Q_reg @ pi_k
    if not np.all(np.isfinite(g_k)): return None, None, None, None, None, False, f"{msg} Final gradient g_k contains NaN/Inf."

    p_f, lam_f, solved_f, kkt_cond_f = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)
    final_lambda_estimate = np.zeros(M)

    if solved_f and p_f is not None and np.linalg.norm(p_f) <= tolerance * 1000 * (1 + np.linalg.norm(pi_k)):
        if n_act > 0:
            if lam_f is not None and len(lam_f) == n_act and np.all(np.isfinite(lam_f)):
                 try: final_lambda_estimate[act] = lam_f
                 except IndexError: pass
                 active_lambdas = final_lambda_estimate[act]
                 if np.all(active_lambdas >= -tolerance * 1000):
                     is_likely_optimal = True; msg += " Final KKT check approx OK."
                 else: msg += f" Final KKT check fails (dual infeasible, min_lam={np.min(active_lambdas):.1e})."
            else: msg += " Final KKT check fails (lam_f invalid)."
        else: is_likely_optimal = True; msg += " Final KKT check approx OK (unconstrained)."
    else:
        p_norm_f = np.linalg.norm(p_f) if p_f is not None else np.nan
        msg += f" Final KKT check fails (stationarity error, p_norm={p_norm_f:.1e} or solve error). KKT_cond={kkt_cond_f:.1e}"

    lam_opt = np.maximum(final_lambda_estimate, 0)
    return pi_k, lam_opt, active_constraints_opt, None, Q_reg / 2.0, is_likely_optimal, msg


# --- make_psd 関数 (変更なし) ---
def make_psd(matrix, tolerance=1e-8):
    if not np.all(np.isfinite(matrix)):
         warnings.warn("make_psd: Input matrix contains NaN/Inf. Returning as is after symmetrization.")
         sym = (matrix + matrix.T) / 2.0
         return sym
    sym = (matrix + matrix.T) / 2.0
    try:
        eigenvalues, eigenvectors = np.linalg.eigh(sym)
        min_eigenvalue = np.min(eigenvalues)
        if min_eigenvalue < tolerance:
            eigenvalues[eigenvalues < tolerance] = tolerance
            psd_matrix = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T
            return (psd_matrix + psd_matrix.T) / 2.0
        else:
            return sym
    except LinAlgError:
        warnings.warn("make_psd: eigh failed. Returning symmetrized input.")
        return sym

# --- calculate_Vw 関数 (変更なし) ---
def calculate_Vw(w, R, SecondMoments_a_array, tolerance=DEFAULT_TOLERANCE, psd_tolerance=1e-8):
    K, M = R.shape
    w_norm = project_to_simplex(w)
    if not np.isclose(np.sum(w_norm), 1.0):
         raise ValueError(f"calculate_Vw: w does not sum to 1 after projection: sum(w)={np.sum(w_norm):.4f}")
    if not np.all(np.isfinite(R)): raise ValueError("calculate_Vw: R contains NaN/Inf.")
    if not np.all(np.isfinite(SecondMoments_a_array)): raise ValueError("calculate_Vw: SecondMoments_a_array contains NaN/Inf.")
    EwX = R @ w_norm
    EwXXT = np.zeros((K, K));
    for m in range(M): EwXXT += w_norm[m] * SecondMoments_a_array[m]
    if not np.all(np.isfinite(EwX)): raise ValueError("calculate_Vw: EwX calculation resulted in NaN/Inf.")
    if not np.all(np.isfinite(EwXXT)): raise ValueError("calculate_Vw: EwXXT calculation resulted in NaN/Inf.")
    Vw = EwXXT - np.outer(EwX, EwX);
    if not np.all(np.isfinite(Vw)): raise ValueError("calculate_Vw: Vw calculation resulted in NaN/Inf.")
    Vw_psd = make_psd(Vw, psd_tolerance)
    if not np.all(np.isfinite(Vw_psd)): raise ValueError("calculate_Vw: Vw_psd after make_psd contains NaN/Inf.")
    return Vw_psd, EwX, EwXXT

# --- calculate_H_gradient 関数 (変更なし) ---
def calculate_H_gradient(pi_star, w, R, SecondMoments_a_array, EwX, EwXXT, tolerance=1e-9, debug_print=False):
    M = w.shape[0]; K = R.shape[0]; grad = np.zeros(M)
    norm_tolerance = 1e-12
    if pi_star is None: return np.full(M, np.nan)
    if not np.all(np.isfinite(pi_star)): return np.full(M, np.nan)
    if not np.all(np.isfinite(w)): return np.full(M, np.nan)
    if not np.all(np.isfinite(R)): return np.full(M, np.nan)
    if not np.all(np.isfinite(SecondMoments_a_array)): return np.full(M, np.nan)
    if EwX is None or not np.all(np.isfinite(EwX)): return np.full(M, np.nan)
    if EwXXT is None or not np.all(np.isfinite(EwXXT)): return np.full(M, np.nan)
    pi_norm = np.linalg.norm(pi_star)
    if pi_norm < norm_tolerance:
        if debug_print: print(f"DEBUG grad_H: Ret NaN pi_norm {pi_norm:.2e} < {norm_tolerance:.1e}")
        return np.full(M, np.nan)
    try:
        pi_T_EwX = pi_star.T @ EwX
        if not np.isfinite(pi_T_EwX):
             if debug_print: print(f"DEBUG grad_H: pi_T_EwX is NaN/Inf: {pi_T_EwX}")
             return np.full(M, np.nan)
        for j in range(M):
            Sigma_j = SecondMoments_a_array[j]; r_j = R[:, j]
            pi_T_Sigma_j_pi = pi_star.T @ Sigma_j @ pi_star
            if not np.isfinite(pi_T_Sigma_j_pi):
                 if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_Sigma_j_pi is NaN/Inf")
                 grad[j] = np.nan; continue
            pi_T_r_j = pi_star.T @ r_j
            if not np.isfinite(pi_T_r_j):
                 if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_r_j is NaN/Inf")
                 grad[j] = np.nan; continue
            term2 = 2 * pi_T_r_j * pi_T_EwX
            if not np.isfinite(term2):
                 if debug_print: print(f"DEBUG grad_H (j={j}): term2 is NaN/Inf")
                 grad[j] = np.nan; continue
            grad[j] = pi_T_Sigma_j_pi - term2
            if not np.isfinite(grad[j]):
                 if debug_print: print(f"DEBUG grad_H (j={j}): final grad[{j}] is NaN/Inf")
    except Exception as e_calc:
         print(f"\nERROR in calculate_H_gradient: {e_calc}\n{traceback.format_exc()}")
         return np.full(M, np.nan)
    if np.any(~np.isfinite(grad)):
        if debug_print: print(f"DEBUG grad_H: Final check found NaN/Inf: {grad}")
        return grad
    return grad

# --- project_to_simplex 関数 (変更なし) ---
def project_to_simplex(v, z=1):
    n_features = v.shape[0];
    if n_features == 0: return np.array([])
    v_arr = np.asarray(v)
    if not np.all(np.isfinite(v_arr)):
        warnings.warn("project_to_simplex: Input contains NaN/Inf. Returning uniform.")
        return np.full(n_features, z / n_features) if n_features > 0 else np.array([])
    if np.all(v_arr >= -1e-9) and np.isclose(np.sum(v_arr), z): return np.maximum(v_arr, 0)
    u = np.sort(v_arr)[::-1]; cssv = np.cumsum(u) - z; ind = np.arange(n_features) + 1; cond = u - cssv / ind > 0
    if np.any(cond): rho = ind[cond][-1]; theta = cssv[rho - 1] / float(rho); w = np.maximum(v_arr - theta, 0)
    else:
         w = np.zeros(n_features)
         if z > 0: w[np.argmax(v_arr)] = z
    w_sum = np.sum(w)
    if not np.isclose(w_sum, z):
        if w_sum > 1e-9: w = w * (z / w_sum)
        elif z > 0 : w = np.zeros(n_features); w[np.argmax(v_arr)] = z
    return np.maximum(w, 0)


# --- frank_wolfe_optimizer 関数 (失敗時の結果保持改善) ---
def frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde, initial_w=None, max_outer_iter=250, fw_gap_tol=1e-7, inner_max_iter=350, tolerance=1e-9, psd_make_tolerance=1e-8, qp_regularization=1e-10, debug_print=False, force_iterations=0, return_history=False):
    K, M = R_alpha.shape
    if initial_w is None: w_k = np.ones(M) / M
    else: w_k = project_to_simplex(np.copy(initial_w))
    if w_k is None or not np.all(np.isfinite(w_k)):
        result = OptimizationResult(False, "Initial w_k invalid after projection", w_opt=initial_w)
        return (result, []) if return_history else result

    pi0, ok, p1msg = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance)
    if not ok:
        result = OptimizationResult(False, f"Phase 1 failed: {p1msg}", w_opt=w_k) # Use w_k here
        return (result, []) if return_history else result
    if pi0 is None or not np.all(np.isfinite(pi0)):
        result = OptimizationResult(False, "Phase 1 returned invalid pi0", w_opt=w_k)
        return (result, []) if return_history else result

    best_w = np.copy(w_k); best_pi = None; best_lam = np.zeros(M); best_H = -float('inf')
    best_grad_H = np.full(M, np.nan); best_active_set = None; best_fw_gap = float('inf')
    last_successful_pi = np.copy(pi0)
    history = []

    inner_solver_args_for_loop = {'max_iter': inner_max_iter, 'tolerance': tolerance, 'regularization_epsilon': qp_regularization}
    converged = False; k = 0; final_msg = ""

    for k in range(max_outer_iter):
        iter_data = {'k': k + 1, 'w_k': np.copy(w_k)} if return_history else {}

        try: # Calculate Vw
            Vk, Ex, ExxT = calculate_Vw(w_k, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
            if not np.all(np.isfinite(Vk)) or not np.all(np.isfinite(Ex)): raise ValueError("Vk or Ex NaN/Inf")
        except Exception as e:
            final_msg = f"Outer iter {k+1}: Vw calculation failed: {e}. Stopping."
            warnings.warn(final_msg); break # Exit loop on Vw failure

        # Inner QP Solve
        pi_init_inner = last_successful_pi if last_successful_pi is not None else pi0
        if pi_init_inner is None or not np.all(np.isfinite(pi_init_inner)): pi_init_inner = pi0
        if pi_init_inner is None or not np.all(np.isfinite(pi_init_inner)):
            final_msg = f"Outer iter {k+1}: Invalid initial pi for inner QP. Stopping."; break

        pk, lk, act_idx_k, _, _, inner_ok, inner_msg = solve_inner_qp_active_set(
            Vk, R_alpha, mu_tilde, pi_init_inner, **inner_solver_args_for_loop
        )

        if not inner_ok or pk is None or not np.all(np.isfinite(pk)): # Inner QP Failed
            tried_pi0_fallback = False
            if pi_init_inner is not pi0:
                 tried_pi0_fallback = True
                 warnings.warn(f"Outer iter {k+1}: Inner QP failed ({inner_msg}). Retrying with pi0.")
                 pk_pi0, lk_pi0, act_idx_k_pi0, _, _, inner_ok_pi0, inner_msg_pi0 = solve_inner_qp_active_set(
                     Vk, R_alpha, mu_tilde, pi0, **inner_solver_args_for_loop
                 )
                 if inner_ok_pi0 and pk_pi0 is not None and np.all(np.isfinite(pk_pi0)):
                     warnings.warn(f"Outer iter {k+1}: Inner QP fallback with pi0 succeeded.")
                     pk = pk_pi0; lk = lk_pi0; act_idx_k = act_idx_k_pi0; inner_ok = True
                 else:
                     warnings.warn(f"Outer iter {k+1}: Inner QP fallback with pi0 also failed ({inner_msg_pi0}).")
                     inner_ok = False
            if not inner_ok:
                 final_msg = f"Outer iter {k+1}: Inner QP failed persistently ({inner_msg})."; break

        # Inner QP Succeeded
        last_successful_pi = np.copy(pk)
        Hk = pk.T @ Vk @ pk
        current_pi = np.copy(pk)
        current_lam = lk if lk is not None and np.all(np.isfinite(lk)) else np.zeros(M)
        current_active_set = tuple(sorted(act_idx_k)) if act_idx_k is not None else tuple()
        current_H = Hk

        try: # Calculate Gradient
            gHk = calculate_H_gradient(pk, w_k, R_alpha, SecondMoments_alpha_array, Ex, ExxT, tolerance, debug_print=debug_print)
            if gHk is None or np.any(~np.isfinite(gHk)): raise ValueError("Grad calc returned None or NaN/Inf")
            current_grad_H = gHk
        except Exception as e:
            final_msg = f"Outer iter {k+1}: Gradient calculation failed: {e}. Stopping."; break

        # Update History
        if return_history:
             iter_data['H_k'] = Hk; iter_data['grad_H_k_norm'] = np.linalg.norm(current_grad_H)
             iter_data['pi_k'] = np.copy(pk); iter_data['lam_k'] = np.copy(current_lam)
             iter_data['active_set_k'] = current_active_set

        # Update Best Solution Found
        if np.isfinite(Hk) and (best_pi is None or Hk > best_H + tolerance * 1e-1): # Added check for best_pi is None
             best_H = Hk; best_w = np.copy(w_k); best_pi = np.copy(pk)
             best_lam = np.copy(current_lam); best_grad_H = np.copy(current_grad_H)
             best_active_set = current_active_set

        # Frank-Wolfe Step
        grad_norm = np.linalg.norm(current_grad_H)
        if grad_norm < tolerance * 100: sk = w_k; sk_idx = -1
        else: sk_idx = np.argmax(current_grad_H); sk = np.zeros(M); sk[sk_idx] = 1.0
        if return_history: iter_data['s_k_index'] = sk_idx

        fw_gap = current_grad_H.T @ (w_k - sk)
        current_fw_gap = fw_gap
        if return_history: iter_data['fw_gap'] = fw_gap

        # Update best FW gap if current H is the best found so far
        if best_pi is not None and np.isclose(Hk, best_H, atol=tolerance*1e-1, rtol=tolerance*1e-1) and np.isfinite(fw_gap):
             best_fw_gap = fw_gap

        # Check Convergence
        if k >= force_iterations and np.isfinite(fw_gap):
            if abs(fw_gap) <= fw_gap_tol:
                 converged = True
                 final_msg = f"Converged (Gap {abs(fw_gap):.2e} <= {fw_gap_tol:.1e})"
                 break # Exit loop

        # Prepare for next iteration
        gamma = 2.0 / (k + 3.0)
        if return_history: iter_data['gamma_k'] = gamma; history.append(iter_data)
        w_k_next = (1.0 - gamma) * w_k + gamma * sk
        w_k = project_to_simplex(w_k_next)
        if w_k is None or not np.all(np.isfinite(w_k)):
             final_msg = f"Outer iter {k+1}: w_k invalid after update. Stopping."; break

    # Loop Finished (Converged, Max Iter, or Error)
    final_iters = k + 1 # Number of iterations completed (or max_iter)
    if not final_msg: # If loop finished by max_iter without prior error message
         final_msg = f"Max Iter ({max_outer_iter}) reached"

    # --- Final Result Preparation ---
    # Use the 'best' values found during the run
    final_w = best_w
    final_pi = best_pi
    final_lam = best_lam
    final_active_set = best_active_set
    final_H = best_H
    final_grad = best_grad_H
    final_gap = best_fw_gap

    # --- Optional Final Consistency Check ---
    # If a best solution was found (best_pi is not None), run one last inner QP
    if final_pi is not None and np.all(np.isfinite(final_w)):
        try:
            final_Vk, final_Ex, final_ExxT = calculate_Vw(final_w, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
            pi_f, lam_f, act_f, _, _, ok_f, msg_f = solve_inner_qp_active_set(
                final_Vk, R_alpha, mu_tilde, pi0, **inner_solver_args_for_loop # Use stable pi0
            )
            if ok_f and pi_f is not None and np.all(np.isfinite(pi_f)):
                 # Update results with the consistent final values
                 final_pi = pi_f
                 final_lam = lam_f if lam_f is not None and np.all(np.isfinite(lam_f)) else np.zeros(M)
                 final_active_set = tuple(sorted(act_f)) if act_f is not None else tuple()
                 final_H = final_pi.T @ final_Vk @ final_pi
                 try: # Recalculate final gradient and gap
                     final_grad = calculate_H_gradient(final_pi, final_w, R_alpha, SecondMoments_alpha_array, final_Ex, final_ExxT, tolerance)
                     if final_grad is None or np.any(~np.isfinite(final_grad)): final_grad = np.full(M, np.nan)
                     if np.any(np.isfinite(final_grad)):
                          grad_norm_f = np.linalg.norm(final_grad[np.isfinite(final_grad)])
                          if grad_norm_f < tolerance * 100: sk_f = final_w
                          else: sk_idx_f = np.nanargmax(np.nan_to_num(final_grad, nan=-np.inf)); sk_f = np.zeros(M); sk_f[sk_idx_f] = 1.0
                          final_gap = final_grad.T @ (final_w - sk_f) if np.all(np.isfinite(final_grad)) else np.nan
                     else: final_gap = np.nan
                 except Exception as e_grad:
                      warnings.warn(f"Final gradient calculation failed after loop: {e_grad}")
                      final_grad = np.full(M, np.nan); final_gap = np.nan
            else:
                warnings.warn(f"Warning: Final inner QP solve failed ({msg_f}). Using best values from iterations.")
        except Exception as e_final:
            warnings.warn(f"Error during final consistency check: {e_final}. Using best values from iterations.")
    # --- End Final Check ---

    # Determine success based on whether a valid H was ever found
    success_flag = best_pi is not None and np.isfinite(best_H)

    if return_history and success_flag: # Add final state if successful run
         grad_norm_final = np.linalg.norm(final_grad[np.isfinite(final_grad)]) if np.any(np.isfinite(final_grad)) else np.nan
         if grad_norm_final < tolerance * 100: sk_idx_final = -1
         else: sk_idx_final = np.nanargmax(np.nan_to_num(final_grad, nan=-np.inf)) if np.any(np.isfinite(final_grad)) else -1
         history.append({
            'k': final_iters + 1, 'w_k': np.copy(final_w), 'H_k': final_H,
            'grad_H_k_norm': grad_norm_final, 's_k_index': sk_idx_final, 'fw_gap': final_gap,
            'gamma_k': np.nan, 'pi_k': np.copy(final_pi), 'lam_k': np.copy(final_lam), 'active_set_k': final_active_set
         })

    # Create result object using the best values found
    result = OptimizationResult(success_flag, final_msg, w_opt=final_w, pi_opt=final_pi, lambda_opt=final_lam,
                                H_opt=final_H, grad_H_opt=final_grad, iterations=final_iters,
                                fw_gap=final_gap, active_set_opt=final_active_set)

    return (result, history) if return_history else result


# --- generate_params_profile_switching_symmetric 関数 (変更なし) ---
def generate_params_profile_switching_symmetric(alpha, alpha_max, K=5, M=3,
                                      R_base_sym=np.array([0.02, 0.01, 0.0, -0.01, -0.02]),
                                      sigma_base=np.array([0.18, 0.15, 0.20, 0.12, 0.10]),
                                      Corr_base=np.array([[1.0, 0.4, 0.3, 0.2, 0.1], [0.4, 1.0, 0.5, 0.3, 0.2], [0.3, 0.5, 1.0, 0.6, 0.4], [0.2, 0.3, 0.6, 1.0, 0.5], [0.1, 0.2, 0.4, 0.5, 1.0]]),
                                      r_offset=0.03,
                                      s_factor_g = 0.001, s_factor_b = -0.001,
                                      sigma_min_epsilon=1e-4, psd_tolerance=1e-9):
    assert K == len(R_base_sym) and K == len(sigma_base) and K == Corr_base.shape[0] and M == 3, "Dimension mismatch"
    R_neutral = R_base_sym; R_good = R_base_sym + r_offset; R_bad = R_base_sym - r_offset
    Corr_neutral_psd = make_psd(Corr_base, psd_tolerance)
    sigma_neutral_eff = np.maximum(sigma_min_epsilon, sigma_base)
    V_neutral = np.diag(sigma_neutral_eff) @ Corr_neutral_psd @ np.diag(sigma_neutral_eff)
    sigma_good_adj = np.maximum(sigma_min_epsilon, sigma_base * (1 + s_factor_g))
    V_good = np.diag(sigma_good_adj) @ Corr_neutral_psd @ np.diag(sigma_good_adj)
    sigma_bad_adj = np.maximum(sigma_min_epsilon, sigma_base * (1 + s_factor_b))
    V_bad = np.diag(sigma_bad_adj) @ Corr_neutral_psd @ np.diag(sigma_bad_adj)
    Sigma_neutral = V_neutral + np.outer(R_neutral, R_neutral)
    Sigma_good = V_good + np.outer(R_good, R_good)
    Sigma_bad = V_bad + np.outer(R_bad, R_bad)
    beta = np.clip(alpha / alpha_max if alpha_max > 0 else (1.0 if alpha > 0 else 0.0), 0.0, 1.0)
    R_alpha = np.zeros((K, M)); SecondMoments_a_array = np.zeros((M, K, K))
    R_alpha[:, 0] = (1 - beta) * R_good + beta * R_neutral
    SecondMoments_a_array[0, :, :] = (1 - beta) * Sigma_good + beta * Sigma_neutral
    R_alpha[:, 1] = (1 - beta) * R_bad + beta * R_good
    SecondMoments_a_array[1, :, :] = (1 - beta) * Sigma_bad + beta * Sigma_good
    R_alpha[:, 2] = (1 - beta) * R_neutral + beta * R_bad
    SecondMoments_a_array[2, :, :] = (1 - beta) * Sigma_neutral + beta * Sigma_bad
    if not np.all(np.isfinite(R_alpha)): raise ValueError("Generated R_alpha contains NaN/Inf.")
    if not np.all(np.isfinite(SecondMoments_a_array)): raise ValueError("Generated SecondMoments_a_array contains NaN/Inf.")
    return R_alpha, SecondMoments_a_array

# === α依存性分析関数 (変更なし) ===
def analyze_alpha_full_results(alpha_range, param_gen_kwargs, optimizer_kwargs, mu_tilde):
    results_over_alpha = []
    K = param_gen_kwargs.get('K', 5)
    M = param_gen_kwargs.get('M', 3)
    if K is None or M is None: raise ValueError("K and M must be specified")

    print(f"\n--- Starting Full Results Analysis over Alpha Range [{alpha_range[0]:.4f}, {alpha_range[-1]:.4f}] ---")
    print(f"--- (Using Active Set Method, mu_tilde={mu_tilde}) ---")
    total_alphas = len(alpha_range); start_loop_time = time.time()
    successful_runs = 0; failed_alphas = []

    for idx, alpha in enumerate(alpha_range):
        loop_start_time = time.time()
        print(f"\rAnalyzing alpha = {alpha:.6f} ({idx+1}/{total_alphas}) ... ", end="")
        alpha_result = {'alpha': alpha}
        fw_success = False; fw_message = "Analysis not run"; w_fw_default = np.full(M, np.nan)
        H_star_fw = np.nan; pi_opt = np.full(K, np.nan); grad_H_opt = np.full(M, np.nan)
        lambda_opt = np.full(M, np.nan); active_set_opt = tuple(); fw_gap_final = np.nan
        iterations_final = 0

        try:
            R_alpha, SecondMoments_alpha_array = generate_params_profile_switching_symmetric(alpha, **param_gen_kwargs)
            if not np.all(np.isfinite(R_alpha)) or not np.all(np.isfinite(SecondMoments_alpha_array)):
                 raise ValueError("Parameter generation resulted in NaN/Inf.")

            fw_result = frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde,
                                              initial_w=None, return_history=False,
                                              debug_print=False, **optimizer_kwargs)
            fw_success = fw_result.success; fw_message = fw_result.message
            iterations_final = fw_result.iterations if fw_result.iterations is not None else 0

            # Store results regardless of success, using NaNs for failed runs
            w_fw_default = fw_result.w_opt if fw_result.w_opt is not None else np.full(M, np.nan)
            H_star_fw = fw_result.H_opt if fw_result.H_opt is not None else np.nan
            pi_opt = fw_result.pi_opt if fw_result.pi_opt is not None else np.full(K, np.nan)
            grad_H_opt = fw_result.grad_H_opt if fw_result.grad_H_opt is not None else np.full(M, np.nan)
            lambda_opt = fw_result.lambda_opt if fw_result.lambda_opt is not None else np.full(M, np.nan)
            active_set_opt = fw_result.active_set_opt if fw_result.active_set_opt is not None else tuple()
            fw_gap_final = fw_result.fw_gap if fw_result.fw_gap is not None else np.nan

            if fw_success: successful_runs += 1
            else:
                 failed_alphas.append(alpha)
                 print(f"\n  FW failed for alpha={alpha:.6f}: {fw_result.message}")

        except Exception as e:
            print(f"\n  Error during analysis for alpha={alpha:.6f}: {e}")
            fw_success = False; fw_message = f"Exception: {e}"; w_fw_default = np.full(M, np.nan)
            H_star_fw = np.nan; pi_opt = np.full(K, np.nan); grad_H_opt = np.full(M, np.nan)
            lambda_opt = np.full(M, np.nan); active_set_opt = tuple(); fw_gap_final = np.nan
            failed_alphas.append(alpha)

        alpha_result['success'] = fw_success; alpha_result['message'] = fw_message
        alpha_result['w_fw_default'] = w_fw_default; alpha_result['H_star'] = H_star_fw
        alpha_result['pi_opt'] = pi_opt; alpha_result['grad_H_opt'] = grad_H_opt
        alpha_result['lambda_opt'] = lambda_opt; alpha_result['active_set_opt'] = active_set_opt
        alpha_result['fw_gap'] = fw_gap_final; alpha_result['iterations'] = iterations_final
        results_over_alpha.append(alpha_result)

    end_loop_time = time.time()
    print(f"\n--- Finished Full Results Analysis ({total_alphas} points) ---")
    print(f"Total time: {end_loop_time - start_loop_time:.2f} seconds")
    print(f"Successful runs: {successful_runs}/{total_alphas}")
    if failed_alphas: print(f"Failed alphas (count: {len(failed_alphas)}): {failed_alphas[:5]}...") # Show only first few failed

    if not PANDAS_AVAILABLE: return results_over_alpha
    df_results = pd.DataFrame(results_over_alpha)
    if df_results.empty: return df_results # Return empty DataFrame if no results

    # Expand vector columns carefully, checking for existence first
    if 'w_fw_default' in df_results.columns:
        w_vecs = np.stack(df_results['w_fw_default'].apply(lambda x: x if isinstance(x, np.ndarray) and x.shape == (M,) else np.full(M, np.nan)).values)
        if np.any(np.isfinite(w_vecs)):
            for m in range(M): df_results[f'w_fw_{m}'] = w_vecs[:, m]
    if 'pi_opt' in df_results.columns:
        pi_vecs = np.stack(df_results['pi_opt'].apply(lambda x: x if isinstance(x, np.ndarray) and x.shape == (K,) else np.full(K, np.nan)).values)
        if np.any(np.isfinite(pi_vecs)):
            for k in range(K): df_results[f'pi_{k}'] = pi_vecs[:, k]
    if 'grad_H_opt' in df_results.columns:
        grad_vecs = np.stack(df_results['grad_H_opt'].apply(lambda x: x if isinstance(x, np.ndarray) and x.shape == (M,) else np.full(M, np.nan)).values)
        if np.any(np.isfinite(grad_vecs)):
            for m in range(M): df_results[f'grad_H_{m}'] = grad_vecs[:, m]
    if 'lambda_opt' in df_results.columns:
        lambda_vecs = np.stack(df_results['lambda_opt'].apply(lambda x: x if isinstance(x, np.ndarray) and x.shape == (M,) else np.full(M, np.nan)).values)
        if np.any(np.isfinite(lambda_vecs)):
            for m in range(M): df_results[f'lambda_{m}'] = lambda_vecs[:, m]
    if 'active_set_opt' in df_results.columns:
        df_results['active_set'] = df_results['active_set_opt'].apply(lambda x: str(x) if x is not None else '()')

    cols_to_drop = ['w_fw_default', 'pi_opt', 'grad_H_opt', 'lambda_opt', 'active_set_opt']
    df_results = df_results.drop(columns=[col for col in cols_to_drop if col in df_results.columns], errors='ignore')
    return df_results


# === メイン実行ブロック ===
if __name__ == '__main__':
    start_time_main = time.time()
    warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered')
    warnings.filterwarnings('ignore', category=UserWarning)
    warnings.filterwarnings('ignore', category=OptimizeWarning)

    # --- 設定 ---
    K = 5; M = 3;
    # ★★★ mu_tilde を 0.022 に変更 ★★★
    mu_tilde = 0.022
    alpha_max = 1.5

    # 準対称パラメータ設定 (変更なし)
    param_gen_kwargs_quasi_symmetric = {
        'K': K, 'M': M, 'alpha_max': alpha_max,
        'R_base_sym': np.array([0.02, 0.01, 0.0, -0.01, -0.02]),
        'sigma_base': np.array([0.18, 0.15, 0.20, 0.12, 0.10]),
        'Corr_base': np.array([[1.0, 0.4, 0.3, 0.2, 0.1], [0.4, 1.0, 0.5, 0.3, 0.2], [0.3, 0.5, 1.0, 0.6, 0.4], [0.2, 0.3, 0.6, 1.0, 0.5], [0.1, 0.2, 0.4, 0.5, 1.0]]),
        'r_offset': 0.03, 's_factor_g': 0.001, 's_factor_b': -0.001,
        'sigma_min_epsilon': 1e-4, 'psd_tolerance': 1e-9
    }

    # FW Optimizer と Inner QP の設定
    solver_settings = {
        'max_outer_iter': 500, 'fw_gap_tol': 1e-9,
        'inner_max_iter': 600,
        'tolerance': 1e-11,
        'psd_make_tolerance': 1e-9,
        # ★★★ 正則化項を少し増やす ★★★
        'qp_regularization': 1e-9, # Increased from 1e-10 to 1e-9
        'force_iterations': 50
    }

    # 分析する alpha の範囲 (変更なし)
    alpha_start = 0.700
    alpha_end = 0.800
    num_alpha_steps = 101
    alpha_range_analyze = np.linspace(alpha_start, alpha_end, num_alpha_steps)
    unique_pt_tolerance = 1e-4

    # 出力ファイル名 (mu_tilde 値と正則化項のオーダーを反映)
    output_csv_filename = f"alpha_internal_sol_search_{alpha_start:.3f}_{alpha_end:.3f}_activeset_mu{int(mu_tilde*1000):03d}_quasi_sym_reg{abs(int(math.log10(solver_settings['qp_regularization'])))}.csv"

    print("-" * 70)
    print(f"--- Searching for Internal Solution around alpha=0.75 [{alpha_start:.3f}, {alpha_end:.3f}] ---")
    print(f"--- (Using ActiveSet, mu_tilde={mu_tilde}, Quasi-Symmetric Params, QP Reg={solver_settings['qp_regularization']:.1e}) ---") # Print regularization
    print(f"--- Output CSV: {output_csv_filename} ---")
    print("-" * 70)

    # --- 分析実行 ---
    results_df = analyze_alpha_full_results(
        alpha_range=alpha_range_analyze,
        param_gen_kwargs=param_gen_kwargs_quasi_symmetric,
        optimizer_kwargs=solver_settings,
        mu_tilde=mu_tilde
    )

    # --- 結果表示 & CSV保存 & KKTチェック ---
    if PANDAS_AVAILABLE and isinstance(results_df, pd.DataFrame) and not results_df.empty:
        print("\n--- Analysis Results Summary (DataFrame) ---")
        cols_display = ['alpha', 'success', 'H_star', 'fw_gap', 'iterations', 'active_set']
        cols_w = [f'w_fw_{m}' for m in range(M) if f'w_fw_{m}' in results_df.columns]
        cols_grad = [f'grad_H_{m}' for m in range(M) if f'grad_H_{m}' in results_df.columns]
        cols_to_show = cols_display + cols_w + cols_grad
        cols_to_show = [col for col in cols_to_show if col in results_df.columns]
        float_format_func = lambda x: f"{x:.4e}" if pd.notna(x) and isinstance(x, (float, np.number)) else x
        pd.options.display.float_format = float_format_func
        with pd.option_context('display.max_rows', 100, 'display.max_columns', None, 'display.width', 200): # Limit rows for console
             print(results_df[cols_to_show].to_string(index=False, na_rep='NaN'))
        pd.reset_option('display.float_format')

        # --- CSVファイルに保存 ---
        try:
            cols_pi = [f'pi_{k}' for k in range(K) if f'pi_{k}' in results_df.columns]
            cols_lam = [f'lambda_{m}' for m in range(M) if f'lambda_{m}' in results_df.columns]
            cols_csv_order = ['alpha', 'success', 'message', 'H_star', 'fw_gap', 'iterations', 'active_set'] + \
                             cols_w + cols_pi + cols_grad + cols_lam
            cols_csv_order = [col for col in cols_csv_order if col in results_df.columns]
            df_to_save = results_df[cols_csv_order]
            df_to_save.to_csv(output_csv_filename, index=False, float_format='%.8e')
            print(f"\nFull detailed results ({len(df_to_save)} points) saved to: {os.path.abspath(output_csv_filename)}")
        except Exception as e: print(f"\nError saving results to CSV: {e}")

        # --- KKT条件のチェック ---
        print("\n--- Quick KKT Check (Stationarity w.r.t. w) ---")
        kkt_violations = []
        check_tol = solver_settings.get('tolerance', 1e-9) * 1e4 # KKT tolerance

        for index, row in results_df.iterrows():
            alpha = row['alpha']
            # Use the 'success' flag from the DataFrame
            if not row.get('success', False):
                 kkt_violations.append({
                    'alpha': alpha, 'active_set_kkt': 'N/A',
                    'max_violation': np.nan, 'grad_consistency_violation': np.nan, 'check_result': 'FW Fail'
                 })
                 continue
            w_star = np.array([row.get(f'w_fw_{m}', np.nan) for m in range(M)])
            grad_h = np.array([row.get(f'grad_H_{m}', np.nan) for m in range(M)])
            if np.isnan(w_star).any() or np.isnan(grad_h).any() or np.isclose(np.sum(w_star), 0):
                kkt_violations.append({
                    'alpha': alpha, 'active_set_kkt': 'NaN',
                    'max_violation': np.nan, 'grad_consistency_violation': np.nan, 'check_result': 'Skipped (NaN)'
                })
                continue

            active_indices_kkt = set(m for m in range(M) if w_star[m] > unique_pt_tolerance)
            grad_consistency_violation = 0.0
            if len(active_indices_kkt) > 1:
                 active_grads = grad_h[list(active_indices_kkt)]
                 if np.all(np.isfinite(active_grads)):
                      grad_consistency_violation = np.max(active_grads) - np.min(active_grads)
                 else: grad_consistency_violation = np.nan

            stationarity_violation = grad_consistency_violation if np.isfinite(grad_consistency_violation) else 0.0
            nu_estimate = np.nan
            if active_indices_kkt:
                 valid_active_grads = grad_h[list(active_indices_kkt)]
                 valid_active_grads = valid_active_grads[np.isfinite(valid_active_grads)]
                 if len(valid_active_grads) > 0: nu_estimate = np.max(valid_active_grads)
            if not np.isfinite(nu_estimate):
                 valid_grads = grad_h[np.isfinite(grad_h)]
                 if len(valid_grads) > 0: nu_estimate = np.max(valid_grads)

            if np.isfinite(nu_estimate):
                for m in range(M):
                    if m not in active_indices_kkt and np.isfinite(grad_h[m]):
                        violation = grad_h[m] - nu_estimate
                        if violation > check_tol:
                            stationarity_violation = max(stationarity_violation, violation)
            else: stationarity_violation = np.nan

            is_internal = len(active_indices_kkt) == M
            result_str = ''
            if not np.isfinite(stationarity_violation): result_str = 'WARN (NaN KKT)'
            elif stationarity_violation > check_tol * 10: result_str = 'WARN (KKT Viol)'
            else: result_str = 'OK'
            if is_internal: result_str += ' [INTERNAL]'
            elif len(active_indices_kkt) == 2: result_str += ' [EDGE]'
            elif len(active_indices_kkt) == 1: result_str += ' [VERTEX]'
            elif len(active_indices_kkt) == 0: result_str += ' [EMPTY?]' # Should not happen if w sums to 1

            kkt_violations.append({
                'alpha': alpha, 'active_set_kkt': str(tuple(sorted(active_indices_kkt))),
                'max_violation': stationarity_violation, 'grad_consistency_violation': grad_consistency_violation,
                'check_result': result_str
            })

        if PANDAS_AVAILABLE and kkt_violations:
            df_kkt = pd.DataFrame(kkt_violations)
            kkt_cols_order = ['alpha', 'active_set_kkt', 'max_violation', 'grad_consistency_violation', 'check_result']
            kkt_cols_order = [col for col in kkt_cols_order if col in df_kkt.columns]
            with pd.option_context('display.float_format', '{:.4e}'.format, 'display.max_rows', None):
                print(df_kkt[kkt_cols_order].to_string(index=False, na_rep='NaN'))
            print("\nNotes on KKT Check:")
            print(" - active_set_kkt: Active set inferred from w* > tolerance for outer problem")
            print(" - max_violation: Max KKT violation (max(grad_consistency, max(grad_inactive - nu_est)))")
            print(" - grad_consistency_violation: max(active_grads) - min(active_grads) for w_m > tol")
            print(" - check_result: OK/WARN indicates KKT satisfaction; [...] indicates solution type.")
        else: print("Could not perform KKT check or pandas not available.")

    elif isinstance(results_df, list) or (isinstance(results_df, pd.DataFrame) and results_df.empty):
        print("\n--- Analysis Results Summary ---")
        print("No results generated or DataFrame is empty.")
        if not PANDAS_AVAILABLE: print("(Pandas not available, skipping CSV export)")

    end_time_main = time.time()
    print(f"\nTotal execution time: {end_time_main - start_time_main:.2f} seconds")

----------------------------------------------------------------------
--- Searching for Internal Solution around alpha=0.75 [0.700, 0.800] ---
--- (Using ActiveSet, mu_tilde=0.022, Quasi-Symmetric Params, QP Reg=1.0e-09) ---
--- Output CSV: alpha_internal_sol_search_0.700_0.800_activeset_mu022_quasi_sym_reg9.csv ---
----------------------------------------------------------------------

--- Starting Full Results Analysis over Alpha Range [0.7000, 0.8000] ---
--- (Using Active Set Method, mu_tilde=0.022) ---
Analyzing alpha = 0.800000 (101/101) ... 
--- Finished Full Results Analysis (101 points) ---
Total time: 32.79 seconds
Successful runs: 101/101

--- Analysis Results Summary (DataFrame) ---
     alpha  success     H_star      fw_gap  iterations active_set     w_fw_0     w_fw_1     w_fw_2   grad_H_0   grad_H_1   grad_H_2
7.0000e-01     True 9.5764e-03 -9.9093e-10         143     (0, 1) 9.9994e-01 3.2375e-05 3.2375e-05 9.0924e-03 9.0809e-03 9.0732e-03
7.0100e-01     True 9.5763e-03 

In [7]:
# -*- coding: utf-8 -*-
"""
5_3_find_internal_solution_mu022_reg_quasi_sym_larger_diff.py

Attempts to find an internal solution w* by using quasi-symmetric parameters
with LARGER differences in Vm (s_factor = +/- 0.01) around alpha=0.75.
Uses mu_tilde = 0.022 and increased QP regularization (1e-9).
Uses the Active Set method for the inner QP solve.
"""

import numpy as np
import matplotlib.pyplot as plt
import warnings
from scipy.linalg import solve, LinAlgError, eigh, inv, qr, lstsq, pinv
from scipy.optimize import linprog, OptimizeWarning
import traceback
import time
import itertools
import os
import math
from collections import Counter

# pandas がなければインストールを促す
try:
    import pandas as pd
    PANDAS_AVAILABLE = True
    pd.set_option('display.width', 180)
    pd.set_option('display.float_format', '{:.6e}'.format)
    pd.set_option('display.max_rows', 300)
except ImportError:
    PANDAS_AVAILABLE = False
    print("Warning: pandas library not found. Output formatting will be basic. CSV export disabled.")

# === 必要な関数定義 (前回から変更なし) ===
DEFAULT_TOLERANCE = 1e-9

# --- OptimizationResult クラス ---
class OptimizationResult:
    def __init__(self, success, message, w_opt=None, pi_opt=None, lambda_opt=None,
                 H_opt=None, grad_H_opt=None, iterations=None, fw_gap=None,
                 active_set_opt=None):
        self.success = success; self.message = message; self.w_opt = w_opt; self.pi_opt = pi_opt
        self.lambda_opt = lambda_opt; self.H_opt = H_opt; self.grad_H_opt = grad_H_opt
        self.iterations = iterations; self.fw_gap = fw_gap
        self.active_set_opt = active_set_opt

# --- find_feasible_initial_pi 関数 ---
def find_feasible_initial_pi(R, mu_tilde, K, tolerance=1e-8):
    M = R.shape[1]; c = np.zeros(K + 1); c[K] = 1.0
    A_ub = np.hstack((-R.T, -np.ones((M, 1)))); b_ub = -mu_tilde * np.ones(M)
    bounds = [(None, None)] * K + [(0, None)]; opts = {'tol': tolerance, 'disp': False, 'presolve': True}
    result = None; methods_to_try = ['highs', 'highs-ipm', 'highs-ds', 'simplex']
    last_method_tried = 'None'
    for method in methods_to_try:
        last_method_tried = method
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", OptimizeWarning)
                result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method=method, options=opts)
            if result.success: break
        except ValueError: continue
        except Exception as e: return None, False, f"Phase 1 LP failed ({method}): {e}"
    if result is None or not result.success:
        msg = result.message if result else "No solver succeeded"; status = result.status if result else -1
        max_expected_return = -np.inf
        try:
             if K > 0 and M > 0 and R is not None and np.all(np.isfinite(R)):
                  equal_pi = np.ones(K) / K; expected_returns = equal_pi @ R
                  if np.all(np.isfinite(expected_returns)): max_expected_return = np.max(expected_returns)
        except: pass
        fail_reason = f"Phase 1 LP solver failed: {msg} (status={status}, method={last_method_tried})"
        if np.isfinite(max_expected_return) and mu_tilde > max_expected_return + tolerance:
             fail_reason += f" - mu_tilde ({mu_tilde:.4f}) may be too high?"
        return None, False, fail_reason
    s = result.x[K]; pi = result.x[:K]
    if np.isnan(pi).any(): return None, False, "Phase 1 LP resulted in NaN values for pi."
    G = -R.T; h = -mu_tilde * np.ones(M); violation = np.max(G @ pi - h)
    feasibility_tol = tolerance * 10000
    if s <= tolerance * 1000:
        if violation <= feasibility_tol: return pi, True, f"Phase 1 OK (s*={s:.1e}, max_viol={violation:.1e})"
        else: return pi, True, f"Phase 1 OK (s*={s:.1e}), WARN: Initial violation {violation:.1e}"
    else: return None, False, f"Phase 1 Likely Infeasible (s*={s:.1e}), max_viol={violation:.1e}"

# --- solve_kkt_system 関数 ---
def solve_kkt_system(Q, G_W, g, tolerance=DEFAULT_TOLERANCE):
    K = Q.shape[0]; n_act = G_W.shape[0] if G_W is not None and G_W.ndim == 2 and G_W.shape[0] > 0 else 0
    kkt_cond = np.nan
    if not np.all(np.isfinite(Q)) or not np.all(np.isfinite(g)): return None, None, False, np.nan
    if n_act > 0 and (G_W is None or not np.all(np.isfinite(G_W))): return None, None, False, np.nan
    if n_act == 0:
        try:
            try: kkt_cond = np.linalg.cond(Q)
            except LinAlgError: kkt_cond = np.inf
            p = solve(Q, -g, assume_a='sym', check_finite=False); l = np.array([])
        except LinAlgError: return None, None, False, kkt_cond
        except ValueError: return None, None, False, kkt_cond
        if p is None or np.isnan(p).any() or np.isinf(p).any(): return None, None, False, kkt_cond
        res_norm = np.linalg.norm(Q @ p + g); g_norm = np.linalg.norm(g)
        solved_ok = res_norm <= tolerance * 1e4 * (1 + g_norm)
        return p, l, solved_ok, kkt_cond
    else:
        kkt_mat = None; rhs = None
        try:
            if G_W.ndim != 2 or G_W.shape[1] != K: return None, None, False, kkt_cond
            kkt_mat = np.block([[Q, G_W.T], [G_W, np.zeros((n_act, n_act))]])
            rhs = np.concatenate([-g, np.zeros(n_act)])
            try: kkt_cond = np.linalg.cond(kkt_mat)
            except LinAlgError: kkt_cond = np.inf
        except ValueError: return None, None, False, kkt_cond
        except Exception as e: return None, None, False, kkt_cond
        try: sol = solve(kkt_mat, rhs, assume_a='sym', check_finite=False); p = sol[:K]; l = sol[K:]
        except LinAlgError: return None, None, False, kkt_cond
        except ValueError: return None, None, False, kkt_cond
        except Exception: return None, None, False, kkt_cond
        if sol is None or np.isnan(sol).any() or np.isinf(sol).any(): return None, None, False, kkt_cond
        res_norm = np.linalg.norm(kkt_mat @ sol - rhs); rhs_norm = np.linalg.norm(rhs)
        solved_ok = res_norm <= tolerance * 1e4 * (1 + rhs_norm)
        return p, l, solved_ok, kkt_cond

# --- solve_inner_qp_active_set 関数 ---
def solve_inner_qp_active_set(Vw, R, mu_tilde, initial_pi, max_iter=350, tolerance=DEFAULT_TOLERANCE, regularization_epsilon=1e-10):
    K = Vw.shape[0]; M = R.shape[1];
    if not np.all(np.isfinite(Vw)): return None, None, None, None, None, False, "QP fail: Vw NaN/Inf."
    Q_reg = 2 * Vw + 2 * regularization_epsilon * np.eye(K); q_cond = np.inf
    try: q_cond = np.linalg.cond(Q_reg)
    except LinAlgError: pass
    if q_cond > 1 / tolerance: warnings.warn(f"QP Warning: Q_reg cond high ({q_cond:.2e}). Reg={regularization_epsilon:.1e}")
    G = -R.T; h = -mu_tilde * np.ones(M)
    if initial_pi is None or not np.all(np.isfinite(initial_pi)): return None, None, None, None, None, False, "QP fail: Initial pi invalid."
    pi_k = np.copy(initial_pi); lam_opt = np.zeros(M); W = set(); active_tol = tolerance * 100
    initial_violations = G @ pi_k - h; max_initial_violation = np.max(initial_violations)
    if max_initial_violation > active_tol * 10:
        warnings.warn(f"QP Warning: Initial pi infeasible (max viol: {max_initial_violation:.2e}).")
        W = set(j for j, viol in enumerate(initial_violations) if viol > active_tol)
    else: W = set(j for j, viol in enumerate(initial_violations) if viol > -active_tol)
    active_indices_opt = None
    for i in range(max_iter):
        g_k = Q_reg @ pi_k
        if not np.all(np.isfinite(g_k)): return None, None, None, None, None, False, f"QP fail:Iter {i+1}: g_k NaN/Inf."
        act = sorted(list(W)); n_act = len(act)
        G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))
        p_k, lam_Wk, solved, kkt_cond = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)
        if not solved or p_k is None: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: KKT solve failed. ActSet={act}. Q_cond={q_cond:.1e}, KKT_cond={kkt_cond:.1e}"
        p_norm = np.linalg.norm(p_k); pi_k_norm = np.linalg.norm(pi_k)
        if p_norm <= tolerance * 100 * (1 + pi_k_norm):
            is_optimal_point = True; blocking_constraint_idx = -1; min_negative_lambda = float('inf'); dual_feas_tol = -tolerance * 100
            if W:
                if lam_Wk is None or len(lam_Wk) != n_act: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: lam_Wk inconsistent? ActSet={act}"
                if not np.all(np.isfinite(lam_Wk)): return None, None, None, None, None, False, f"QP fail:Iter {i+1}: lam_Wk NaN/Inf. ActSet={act}"
                lambda_map = dict(zip(act, lam_Wk))
                for constr_idx, lam_val in lambda_map.items():
                    if lam_val < dual_feas_tol: is_optimal_point = False
                    if lam_val < min_negative_lambda: min_negative_lambda = lam_val; blocking_constraint_idx = constr_idx
            if is_optimal_point:
                lam_opt.fill(0.0)
                if W and lam_Wk is not None:
                     try: lam_opt[act] = np.maximum(lam_Wk, 0)
                     except IndexError: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: Index err assign lambda. ActSet={act}"
                final_infeas = np.max(G @ pi_k - h); msg = f"Optimal found iter {i+1}."
                if final_infeas > active_tol: msg += f" (WARN: Final viol {final_infeas:.1e})"
                active_indices_opt = act
                return pi_k, lam_opt, active_indices_opt, None, Q_reg / 2.0, True, msg
            else:
                if blocking_constraint_idx in W: W.remove(blocking_constraint_idx); continue
                else: return None, None, None, None, None, False, f"QP fail:Iter {i+1}: Neg lam idx {blocking_constraint_idx}, not in W={act}"
        else:
            alpha_k = 1.0; blocking_constraint_idx = -1; min_step_length = float('inf'); step_tol = tolerance * 100
            for j in range(M):
                if j not in W:
                    constr_grad_dot_p = G[j, :] @ p_k
                    if constr_grad_dot_p > step_tol:
                        dist_to_bound = h[j] - (G[j, :] @ pi_k)
                        if abs(constr_grad_dot_p) > 1e-15:
                             alpha_j = dist_to_bound / constr_grad_dot_p
                             if np.isfinite(alpha_j) and alpha_j >= -tolerance:
                                 step_j = max(0.0, alpha_j)
                                 if step_j < min_step_length: min_step_length = step_j; blocking_constraint_idx = j
            alpha_k = min(1.0, min_step_length)
            pi_k += alpha_k * p_k
            if not np.all(np.isfinite(pi_k)): return None, None, None, None, None, False, f"QP fail:Iter {i+1}: pi_k NaN/Inf after step."
            if alpha_k < 1.0 - step_tol and blocking_constraint_idx != -1:
                if blocking_constraint_idx not in W: W.add(blocking_constraint_idx)
            continue
    msg = f"Max iter ({max_iter}) reached."; final_infeas = np.max(G @ pi_k - h)
    if final_infeas > active_tol * 100: return None, None, None, None, None, False, f"{msg} Final infeasible ({final_infeas:.1e}). ActSet={sorted(list(W))}"
    act = sorted(list(W)); n_act = len(act); G_Wk = G[act, :] if n_act > 0 else np.empty((0, K))
    is_likely_optimal = False; active_constraints_opt = act; g_k = Q_reg @ pi_k
    if not np.all(np.isfinite(g_k)): return None, None, None, None, None, False, f"{msg} Final g_k NaN/Inf."
    p_f, lam_f, solved_f, kkt_cond_f = solve_kkt_system(Q_reg, G_Wk, g_k, tolerance)
    final_lambda_estimate = np.zeros(M)
    if solved_f and p_f is not None and np.linalg.norm(p_f) <= tolerance * 1000 * (1 + np.linalg.norm(pi_k)):
        if n_act > 0:
            if lam_f is not None and len(lam_f) == n_act and np.all(np.isfinite(lam_f)):
                 try: final_lambda_estimate[act] = lam_f
                 except IndexError: pass
                 active_lambdas = final_lambda_estimate[act]
                 if np.all(active_lambdas >= -tolerance * 1000): is_likely_optimal = True; msg += " Final KKT approx OK."
                 else: msg += f" Final KKT fails (dual infeas, min_lam={np.min(active_lambdas):.1e})."
            else: msg += " Final KKT fails (lam_f invalid)."
        else: is_likely_optimal = True; msg += " Final KKT approx OK (unconstrained)."
    else: p_norm_f = np.linalg.norm(p_f) if p_f is not None else np.nan; msg += f" Final KKT fails (stationarity p_norm={p_norm_f:.1e} or solve). KKT_cond={kkt_cond_f:.1e}"
    lam_opt = np.maximum(final_lambda_estimate, 0)
    return pi_k, lam_opt, active_constraints_opt, None, Q_reg / 2.0, is_likely_optimal, msg

# --- make_psd 関数 ---
def make_psd(matrix, tolerance=1e-8):
    if not np.all(np.isfinite(matrix)): warnings.warn("make_psd: Input NaN/Inf."); sym = (matrix + matrix.T) / 2.0; return sym
    sym = (matrix + matrix.T) / 2.0
    try:
        eigenvalues, eigenvectors = np.linalg.eigh(sym); min_eigenvalue = np.min(eigenvalues)
        if min_eigenvalue < tolerance: eigenvalues[eigenvalues < tolerance] = tolerance; psd_matrix = eigenvectors @ np.diag(eigenvalues) @ eigenvectors.T; return (psd_matrix + psd_matrix.T) / 2.0
        else: return sym
    except LinAlgError: warnings.warn("make_psd: eigh failed."); return sym

# --- calculate_Vw 関数 ---
def calculate_Vw(w, R, SecondMoments_a_array, tolerance=DEFAULT_TOLERANCE, psd_tolerance=1e-8):
    K, M = R.shape; w_norm = project_to_simplex(w)
    if not np.isclose(np.sum(w_norm), 1.0): raise ValueError(f"calculate_Vw: w sum err: {np.sum(w_norm):.4f}")
    if not np.all(np.isfinite(R)): raise ValueError("calculate_Vw: R NaN/Inf.")
    if not np.all(np.isfinite(SecondMoments_a_array)): raise ValueError("calculate_Vw: SecondMoments NaN/Inf.")
    EwX = R @ w_norm; EwXXT = np.zeros((K, K));
    for m in range(M): EwXXT += w_norm[m] * SecondMoments_a_array[m]
    if not np.all(np.isfinite(EwX)): raise ValueError("calculate_Vw: EwX NaN/Inf.")
    if not np.all(np.isfinite(EwXXT)): raise ValueError("calculate_Vw: EwXXT NaN/Inf.")
    Vw = EwXXT - np.outer(EwX, EwX);
    if not np.all(np.isfinite(Vw)): raise ValueError("calculate_Vw: Vw NaN/Inf.")
    Vw_psd = make_psd(Vw, psd_tolerance)
    if not np.all(np.isfinite(Vw_psd)): raise ValueError("calculate_Vw: Vw_psd NaN/Inf.")
    return Vw_psd, EwX, EwXXT

# --- calculate_H_gradient 関数 ---
def calculate_H_gradient(pi_star, w, R, SecondMoments_a_array, EwX, EwXXT, tolerance=1e-9, debug_print=False):
    M = w.shape[0]; K = R.shape[0]; grad = np.zeros(M); norm_tolerance = 1e-12
    if pi_star is None or not np.all(np.isfinite(pi_star)): return np.full(M, np.nan)
    if not np.all(np.isfinite(w)): return np.full(M, np.nan)
    if not np.all(np.isfinite(R)): return np.full(M, np.nan)
    if not np.all(np.isfinite(SecondMoments_a_array)): return np.full(M, np.nan)
    if EwX is None or not np.all(np.isfinite(EwX)): return np.full(M, np.nan)
    if EwXXT is None or not np.all(np.isfinite(EwXXT)): return np.full(M, np.nan)
    pi_norm = np.linalg.norm(pi_star)
    if pi_norm < norm_tolerance:
      if debug_print: print(f"DEBUG grad_H: Ret NaN pi_norm {pi_norm:.2e}"); return np.full(M, np.nan)
    try:
        pi_T_EwX = pi_star.T @ EwX
        if not np.isfinite(pi_T_EwX): if debug_print: print(f"DEBUG grad_H: pi_T_EwX NaN/Inf"); return np.full(M, np.nan)
        for j in range(M):
            Sigma_j = SecondMoments_a_array[j]; r_j = R[:, j]
            pi_T_Sigma_j_pi = pi_star.T @ Sigma_j @ pi_star
            if not np.isfinite(pi_T_Sigma_j_pi): if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_Sigma_j_pi NaN/Inf"); grad[j] = np.nan; continue
            pi_T_r_j = pi_star.T @ r_j
            if not np.isfinite(pi_T_r_j): if debug_print: print(f"DEBUG grad_H (j={j}): pi_T_r_j NaN/Inf"); grad[j] = np.nan; continue
            term2 = 2 * pi_T_r_j * pi_T_EwX
            if not np.isfinite(term2): if debug_print: print(f"DEBUG grad_H (j={j}): term2 NaN/Inf"); grad[j] = np.nan; continue
            grad[j] = pi_T_Sigma_j_pi - term2
            if not np.isfinite(grad[j]): if debug_print: print(f"DEBUG grad_H (j={j}): final grad[{j}] NaN/Inf")
    except Exception as e_calc: print(f"\nERROR in calculate_H_gradient: {e_calc}\n{traceback.format_exc()}"); return np.full(M, np.nan)
    if np.any(~np.isfinite(grad)): if debug_print: print(f"DEBUG grad_H: Final check NaN/Inf: {grad}"); return grad
    return grad

# --- project_to_simplex 関数 ---
def project_to_simplex(v, z=1):
    n_features = v.shape[0];
    if n_features == 0: return np.array([])
    v_arr = np.asarray(v)
    if not np.all(np.isfinite(v_arr)): warnings.warn("project_to_simplex: Input NaN/Inf."); return np.full(n_features, z / n_features) if n_features > 0 else np.array([])
    if np.all(v_arr >= -1e-9) and np.isclose(np.sum(v_arr), z): return np.maximum(v_arr, 0)
    u = np.sort(v_arr)[::-1]; cssv = np.cumsum(u) - z; ind = np.arange(n_features) + 1; cond = u - cssv / ind > 0
    if np.any(cond): rho = ind[cond][-1]; theta = cssv[rho - 1] / float(rho); w = np.maximum(v_arr - theta, 0)
    else: w = np.zeros(n_features);
    if z > 0 and not np.any(cond): w[np.argmax(v_arr)] = z # Fallback if needed
    w_sum = np.sum(w)
    if not np.isclose(w_sum, z):
        if w_sum > 1e-9: w = w * (z / w_sum)
        elif z > 0 : w = np.zeros(n_features); w[np.argmax(v_arr)] = z
    return np.maximum(w, 0)

# --- frank_wolfe_optimizer 関数 ---
def frank_wolfe_optimizer(R_alpha, SecondMoments_alpha_array, mu_tilde, initial_w=None, max_outer_iter=250, fw_gap_tol=1e-7, inner_max_iter=350, tolerance=1e-9, psd_make_tolerance=1e-8, qp_regularization=1e-10, debug_print=False, force_iterations=0, return_history=False):
    K, M = R_alpha.shape
    if initial_w is None: w_k = np.ones(M) / M
    else: w_k = project_to_simplex(np.copy(initial_w))
    if w_k is None or not np.all(np.isfinite(w_k)): return (OptimizationResult(False, "Initial w_k invalid", w_opt=initial_w), []) if return_history else OptimizationResult(False, "Initial w_k invalid", w_opt=initial_w)
    pi0, ok, p1msg = find_feasible_initial_pi(R_alpha, mu_tilde, K, tolerance)
    if not ok: return (OptimizationResult(False, f"Phase 1 failed: {p1msg}", w_opt=w_k), []) if return_history else OptimizationResult(False, f"Phase 1 failed: {p1msg}", w_opt=w_k)
    if pi0 is None or not np.all(np.isfinite(pi0)): return (OptimizationResult(False, "Phase 1 invalid pi0", w_opt=w_k), []) if return_history else OptimizationResult(False, "Phase 1 invalid pi0", w_opt=w_k)
    best_w = np.copy(w_k); best_pi = None; best_lam = np.zeros(M); best_H = -float('inf')
    best_grad_H = np.full(M, np.nan); best_active_set = None; best_fw_gap = float('inf')
    last_successful_pi = np.copy(pi0); history = []; converged = False; k = 0; final_msg = ""
    inner_solver_args_for_loop = {'max_iter': inner_max_iter, 'tolerance': tolerance, 'regularization_epsilon': qp_regularization}
    for k in range(max_outer_iter):
        iter_data = {'k': k + 1, 'w_k': np.copy(w_k)} if return_history else {}
        try: Vk, Ex, ExxT = calculate_Vw(w_k, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
        except Exception as e: final_msg = f"Outer iter {k+1}: Vw calc failed: {e}."; warnings.warn(final_msg); break
        pi_init_inner = last_successful_pi if last_successful_pi is not None else pi0
        if pi_init_inner is None or not np.all(np.isfinite(pi_init_inner)): pi_init_inner = pi0
        if pi_init_inner is None or not np.all(np.isfinite(pi_init_inner)): final_msg = f"Outer iter {k+1}: Invalid inner pi init."; break
        pk, lk, act_idx_k, _, _, inner_ok, inner_msg = solve_inner_qp_active_set(Vk, R_alpha, mu_tilde, pi_init_inner, **inner_solver_args_for_loop)
        if not inner_ok or pk is None or not np.all(np.isfinite(pk)):
            tried_pi0_fallback = False
            if pi_init_inner is not pi0:
                 tried_pi0_fallback = True; warnings.warn(f"Outer iter {k+1}: Inner QP failed ({inner_msg}). Retry w/ pi0.")
                 pk_pi0, lk_pi0, act_idx_k_pi0, _, _, inner_ok_pi0, inner_msg_pi0 = solve_inner_qp_active_set(Vk, R_alpha, mu_tilde, pi0, **inner_solver_args_for_loop)
                 if inner_ok_pi0 and pk_pi0 is not None and np.all(np.isfinite(pk_pi0)): warnings.warn(f"Outer iter {k+1}: Inner QP fallback OK."); pk = pk_pi0; lk = lk_pi0; act_idx_k = act_idx_k_pi0; inner_ok = True
                 else: warnings.warn(f"Outer iter {k+1}: Inner QP fallback failed ({inner_msg_pi0})."); inner_ok = False
            if not inner_ok: final_msg = f"Outer iter {k+1}: Inner QP failed persist ({inner_msg})."; break
        last_successful_pi = np.copy(pk); Hk = pk.T @ Vk @ pk
        current_pi = np.copy(pk); current_lam = lk if lk is not None and np.all(np.isfinite(lk)) else np.zeros(M)
        current_active_set = tuple(sorted(act_idx_k)) if act_idx_k is not None else tuple(); current_H = Hk
        try:
            gHk = calculate_H_gradient(pk, w_k, R_alpha, SecondMoments_alpha_array, Ex, ExxT, tolerance, debug_print=debug_print)
            if gHk is None or np.any(~np.isfinite(gHk)): raise ValueError("Grad calc None/NaN/Inf")
            current_grad_H = gHk
        except Exception as e: final_msg = f"Outer iter {k+1}: Grad calc failed: {e}."; break
        if return_history: iter_data['H_k'] = Hk; iter_data['grad_H_k_norm'] = np.linalg.norm(current_grad_H); iter_data['pi_k'] = np.copy(pk); iter_data['lam_k'] = np.copy(current_lam); iter_data['active_set_k'] = current_active_set
        if np.isfinite(Hk) and (best_pi is None or Hk > best_H + tolerance * 1e-1): best_H = Hk; best_w = np.copy(w_k); best_pi = np.copy(pk); best_lam = np.copy(current_lam); best_grad_H = np.copy(current_grad_H); best_active_set = current_active_set
        grad_norm = np.linalg.norm(current_grad_H);
        if grad_norm < tolerance * 100: sk = w_k; sk_idx = -1
        else: sk_idx = np.argmax(current_grad_H); sk = np.zeros(M); sk[sk_idx] = 1.0
        if return_history: iter_data['s_k_index'] = sk_idx
        fw_gap = current_grad_H.T @ (w_k - sk); current_fw_gap = fw_gap
        if return_history: iter_data['fw_gap'] = fw_gap
        if best_pi is not None and np.isclose(Hk, best_H, atol=tolerance*1e-1, rtol=tolerance*1e-1) and np.isfinite(fw_gap): best_fw_gap = fw_gap
        if k >= force_iterations and np.isfinite(fw_gap):
            if abs(fw_gap) <= fw_gap_tol: converged = True; final_msg = f"Converged (Gap {abs(fw_gap):.2e})"; break
        gamma = 2.0 / (k + 3.0)
        if return_history: iter_data['gamma_k'] = gamma; history.append(iter_data)
        w_k_next = (1.0 - gamma) * w_k + gamma * sk; w_k = project_to_simplex(w_k_next)
        if w_k is None or not np.all(np.isfinite(w_k)): final_msg = f"Outer iter {k+1}: w_k invalid after update."; break
    final_iters = k + 1;
    if not final_msg: final_msg = f"Max Iter ({max_outer_iter}) reached"
    final_w = best_w; final_pi = best_pi; final_lam = best_lam; final_active_set = best_active_set
    final_H = best_H; final_grad = best_grad_H; final_gap = best_fw_gap
    if final_pi is not None and np.all(np.isfinite(final_w)):
        try:
            final_Vk, final_Ex, final_ExxT = calculate_Vw(final_w, R_alpha, SecondMoments_alpha_array, tolerance=tolerance, psd_tolerance=psd_make_tolerance)
            pi_f, lam_f, act_f, _, _, ok_f, msg_f = solve_inner_qp_active_set(final_Vk, R_alpha, mu_tilde, pi0, **inner_solver_args_for_loop)
            if ok_f and pi_f is not None and np.all(np.isfinite(pi_f)):
                 final_pi = pi_f; final_lam = lam_f if lam_f is not None and np.all(np.isfinite(lam_f)) else np.zeros(M)
                 final_active_set = tuple(sorted(act_f)) if act_f is not None else tuple(); final_H = final_pi.T @ final_Vk @ final_pi
                 try:
                     final_grad = calculate_H_gradient(final_pi, final_w, R_alpha, SecondMoments_alpha_array, final_Ex, final_ExxT, tolerance)
                     if final_grad is None or np.any(~np.isfinite(final_grad)): final_grad = np.full(M, np.nan)
                     if np.any(np.isfinite(final_grad)):
                          grad_norm_f = np.linalg.norm(final_grad[np.isfinite(final_grad)])
                          if grad_norm_f < tolerance * 100: sk_f = final_w
                          else: sk_idx_f = np.nanargmax(np.nan_to_num(final_grad, nan=-np.inf)); sk_f = np.zeros(M); sk_f[sk_idx_f] = 1.0
                          final_gap = final_grad.T @ (final_w - sk_f) if np.all(np.isfinite(final_grad)) else np.nan
                     else: final_gap = np.nan
                 except Exception as e_grad: warnings.warn(f"Final grad calc failed: {e_grad}"); final_grad = np.full(M, np.nan); final_gap = np.nan
            else: warnings.warn(f"Warning: Final inner QP solve failed ({msg_f}). Using best iter values.")
        except Exception as e_final: warnings.warn(f"Error during final check: {e_final}. Using best iter values.")
    success_flag = best_pi is not None and np.isfinite(best_H)
    if return_history and success_flag:
         grad_norm_final = np.linalg.norm(final_grad[np.isfinite(final_grad)]) if np.any(np.isfinite(final_grad)) else np.nan
         if grad_norm_final < tolerance * 100: sk_idx_final = -1
         else: sk_idx_final = np.nanargmax(np.nan_to_num(final_grad, nan=-np.inf)) if np.any(np.isfinite(final_grad)) else -1
         history.append({'k': final_iters + 1, 'w_k': np.copy(final_w), 'H_k': final_H, 'grad_H_k_norm': grad_norm_final, 's_k_index': sk_idx_final, 'fw_gap': final_gap, 'gamma_k': np.nan, 'pi_k': np.copy(final_pi), 'lam_k': np.copy(final_lam), 'active_set_k': final_active_set})
    result = OptimizationResult(success_flag, final_msg, w_opt=final_w, pi_opt=final_pi, lambda_opt=final_lam, H_opt=final_H, grad_H_opt=final_grad, iterations=final_iters, fw_gap=final_gap, active_set_opt=final_active_set)
    return (result, history) if return_history else result

# --- generate_params_profile_switching_symmetric 関数 ---
# (s_factor_g, s_factor_b を kwargs から受け取るように変更)
def generate_params_profile_switching_symmetric(alpha, alpha_max, K=5, M=3,
                                      R_base_sym=np.array([0.02, 0.01, 0.0, -0.01, -0.02]),
                                      sigma_base=np.array([0.18, 0.15, 0.20, 0.12, 0.10]),
                                      Corr_base=np.array([[1.0, 0.4, 0.3, 0.2, 0.1], [0.4, 1.0, 0.5, 0.3, 0.2], [0.3, 0.5, 1.0, 0.6, 0.4], [0.2, 0.3, 0.6, 1.0, 0.5], [0.1, 0.2, 0.4, 0.5, 1.0]]),
                                      r_offset=0.03,
                                      # ★★ s_factor を kwargs で受け取る ★★
                                      s_factor_g = 0.001, # Default value if not provided
                                      s_factor_b = -0.001,# Default value if not provided
                                      # ★★★★★★★★★★★★★★★★★★★★★★
                                      sigma_min_epsilon=1e-4, psd_tolerance=1e-9):
    """ Generates quasi-symmetric parameters with controllable differences in Vm """
    assert K == len(R_base_sym) and K == len(sigma_base) and K == Corr_base.shape[0] and M == 3, "Dimension mismatch"
    R_neutral = R_base_sym; R_good = R_base_sym + r_offset; R_bad = R_base_sym - r_offset
    Corr_neutral_psd = make_psd(Corr_base, psd_tolerance)
    sigma_neutral_eff = np.maximum(sigma_min_epsilon, sigma_base)
    V_neutral = np.diag(sigma_neutral_eff) @ Corr_neutral_psd @ np.diag(sigma_neutral_eff)
    # Use s_factor_g, s_factor_b passed via kwargs
    sigma_good_adj = np.maximum(sigma_min_epsilon, sigma_base * (1 + s_factor_g))
    V_good = np.diag(sigma_good_adj) @ Corr_neutral_psd @ np.diag(sigma_good_adj)
    sigma_bad_adj = np.maximum(sigma_min_epsilon, sigma_base * (1 + s_factor_b))
    V_bad = np.diag(sigma_bad_adj) @ Corr_neutral_psd @ np.diag(sigma_bad_adj)
    Sigma_neutral = V_neutral + np.outer(R_neutral, R_neutral)
    Sigma_good = V_good + np.outer(R_good, R_good); Sigma_bad = V_bad + np.outer(R_bad, R_bad)
    beta = np.clip(alpha / alpha_max if alpha_max > 0 else (1.0 if alpha > 0 else 0.0), 0.0, 1.0)
    R_alpha = np.zeros((K, M)); SecondMoments_a_array = np.zeros((M, K, K))
    R_alpha[:, 0] = (1 - beta) * R_good + beta * R_neutral
    SecondMoments_a_array[0, :, :] = (1 - beta) * Sigma_good + beta * Sigma_neutral
    R_alpha[:, 1] = (1 - beta) * R_bad + beta * R_good
    SecondMoments_a_array[1, :, :] = (1 - beta) * Sigma_bad + beta * Sigma_good
    R_alpha[:, 2] = (1 - beta) * R_neutral + beta * R_bad
    SecondMoments_a_array[2, :, :] = (1 - beta) * Sigma_neutral + beta * Sigma_bad
    if not np.all(np.isfinite(R_alpha)): raise ValueError("Gen R_alpha NaN/Inf.")
    if not np.all(np.isfinite(SecondMoments_a_array)): raise ValueError("Gen SecondMoments NaN/Inf.")
    return R_alpha, SecondMoments_a_array

# === analyze_alpha_full_results 関数 (変更なし) ===
def analyze_alpha_full_results(alpha_range, param_gen_kwargs, optimizer_kwargs, mu_tilde):
    results_over_alpha = []
    K = param_gen_kwargs.get('K', 5); M = param_gen_kwargs.get('M', 3)
    if K is None or M is None: raise ValueError("K and M must be specified")
    print(f"\n--- Starting Full Results Analysis [{alpha_range[0]:.4f}, {alpha_range[-1]:.4f}] (mu={mu_tilde:.4f}) ---")
    total_alphas = len(alpha_range); start_loop_time = time.time(); successful_runs = 0; failed_alphas = []
    for idx, alpha in enumerate(alpha_range):
        print(f"\rAnalyzing alpha = {alpha:.6f} ({idx+1}/{total_alphas}) ... ", end="")
        alpha_result = {'alpha': alpha}; fw_success = False; fw_message = "Not run"; w_fw = np.full(M, np.nan)
        H_star = np.nan; pi_opt = np.full(K, np.nan); grad_H = np.full(M, np.nan); lam_opt = np.full(M, np.nan)
        act_set = tuple(); fw_gap = np.nan; iters = 0
        try:
            R_alpha, SecMoments = generate_params_profile_switching_symmetric(alpha, **param_gen_kwargs)
            fw_result = frank_wolfe_optimizer(R_alpha, SecMoments, mu_tilde, initial_w=None, return_history=False, debug_print=False, **optimizer_kwargs)
            fw_success = fw_result.success; fw_message = fw_result.message; iters = fw_result.iterations
            w_fw = fw_result.w_opt if fw_result.w_opt is not None else np.full(M, np.nan)
            H_star = fw_result.H_opt if fw_result.H_opt is not None else np.nan
            pi_opt = fw_result.pi_opt if fw_result.pi_opt is not None else np.full(K, np.nan)
            grad_H = fw_result.grad_H_opt if fw_result.grad_H_opt is not None else np.full(M, np.nan)
            lam_opt = fw_result.lambda_opt if fw_result.lambda_opt is not None else np.full(M, np.nan)
            act_set = fw_result.active_set_opt if fw_result.active_set_opt is not None else tuple()
            fw_gap = fw_result.fw_gap if fw_result.fw_gap is not None else np.nan
            if fw_success: successful_runs += 1
            else: failed_alphas.append(alpha); print(f"\n  FW fail alpha={alpha:.6f}: {fw_result.message}")
        except Exception as e:
            print(f"\n  Error alpha={alpha:.6f}: {e}"); fw_success = False; fw_message = f"Exception: {e}"; failed_alphas.append(alpha)
        alpha_result['success'] = fw_success; alpha_result['message'] = fw_message; alpha_result['w_fw_default'] = w_fw
        alpha_result['H_star'] = H_star; alpha_result['pi_opt'] = pi_opt; alpha_result['grad_H_opt'] = grad_H
        alpha_result['lambda_opt'] = lam_opt; alpha_result['active_set_opt'] = act_set; alpha_result['fw_gap'] = fw_gap; alpha_result['iterations'] = iters
        results_over_alpha.append(alpha_result)
    end_loop_time = time.time(); print(f"\n--- Finished Analysis ({total_alphas} pts) --- Time: {end_loop_time - start_loop_time:.2f}s")
    print(f"Successful runs: {successful_runs}/{total_alphas}")
    if failed_alphas: print(f"Failed alphas ({len(failed_alphas)}): {failed_alphas[:5]}...")
    if not PANDAS_AVAILABLE: return results_over_alpha
    df_results = pd.DataFrame(results_over_alpha)
    if df_results.empty: return df_results
    if 'w_fw_default' in df_results.columns:
        w_vecs = np.stack(df_results['w_fw_default'].apply(lambda x: x if isinstance(x, np.ndarray) and x.shape == (M,) else np.full(M, np.nan)).values)
        if np.any(np.isfinite(w_vecs)):
            for m in range(M): df_results[f'w_fw_{m}'] = w_vecs[:, m]
    if 'pi_opt' in df_results.columns:
        pi_vecs = np.stack(df_results['pi_opt'].apply(lambda x: x if isinstance(x, np.ndarray) and x.shape == (K,) else np.full(K, np.nan)).values)
        if np.any(np.isfinite(pi_vecs)):
            for k in range(K): df_results[f'pi_{k}'] = pi_vecs[:, k]
    if 'grad_H_opt' in df_results.columns:
        grad_vecs = np.stack(df_results['grad_H_opt'].apply(lambda x: x if isinstance(x, np.ndarray) and x.shape == (M,) else np.full(M, np.nan)).values)
        if np.any(np.isfinite(grad_vecs)):
            for m in range(M): df_results[f'grad_H_{m}'] = grad_vecs[:, m]
    if 'lambda_opt' in df_results.columns:
        lambda_vecs = np.stack(df_results['lambda_opt'].apply(lambda x: x if isinstance(x, np.ndarray) and x.shape == (M,) else np.full(M, np.nan)).values)
        if np.any(np.isfinite(lambda_vecs)):
            for m in range(M): df_results[f'lambda_{m}'] = lambda_vecs[:, m]
    if 'active_set_opt' in df_results.columns:
        df_results['active_set'] = df_results['active_set_opt'].apply(lambda x: str(x) if x is not None else '()')
    cols_to_drop = ['w_fw_default', 'pi_opt', 'grad_H_opt', 'lambda_opt', 'active_set_opt']
    df_results = df_results.drop(columns=[col for col in cols_to_drop if col in df_results.columns], errors='ignore')
    return df_results

# === メイン実行ブロック ===
if __name__ == '__main__':
    start_time_main = time.time()
    warnings.filterwarnings('ignore', category=RuntimeWarning, message='invalid value encountered')
    warnings.filterwarnings('ignore', category=UserWarning); warnings.filterwarnings('ignore', category=OptimizeWarning)

    # --- 設定 ---
    K = 5; M = 3;
    mu_tilde = 0.022 # 維持
    alpha_max = 1.5

    # ★★★ 準対称パラメータ設定 (差異を拡大) ★★★
    param_gen_kwargs_quasi_symmetric_larger_diff = {
        'K': K, 'M': M, 'alpha_max': alpha_max,
        'R_base_sym': np.array([0.02, 0.01, 0.0, -0.01, -0.02]),
        'sigma_base': np.array([0.18, 0.15, 0.20, 0.12, 0.10]),
        'Corr_base': np.array([[1.0, 0.4, 0.3, 0.2, 0.1], [0.4, 1.0, 0.5, 0.3, 0.2], [0.3, 0.5, 1.0, 0.6, 0.4], [0.2, 0.3, 0.6, 1.0, 0.5], [0.1, 0.2, 0.4, 0.5, 1.0]]),
        'r_offset': 0.03, # 維持
        # ★★ Vm の差を拡大 ★★
        's_factor_g': 0.01,  # 0.001 -> 0.01
        's_factor_b': -0.01, # -0.001 -> -0.01
        # ★★★★★★★★★★★★★★
        'sigma_min_epsilon': 1e-4, 'psd_tolerance': 1e-9
    }

    # ソルバー設定 (正則化項は 1e-9 のまま)
    solver_settings = {
        'max_outer_iter': 500, 'fw_gap_tol': 1e-9, 'inner_max_iter': 600,
        'tolerance': 1e-11, 'psd_make_tolerance': 1e-9,
        'qp_regularization': 1e-9, # 維持
        'force_iterations': 50
    }

    # alpha 範囲 (変更なし)
    alpha_start = 0.700; alpha_end = 0.800; num_alpha_steps = 101
    alpha_range_analyze = np.linspace(alpha_start, alpha_end, num_alpha_steps)
    unique_pt_tolerance = 1e-4

    # 出力ファイル名 (差分拡大を反映)
    s_factor_abs_int = abs(int(math.log10(param_gen_kwargs_quasi_symmetric_larger_diff['s_factor_g']))) if param_gen_kwargs_quasi_symmetric_larger_diff['s_factor_g'] != 0 else 0
    output_csv_filename = f"alpha_internal_sol_search_{alpha_start:.3f}_{alpha_end:.3f}_activeset_mu{int(mu_tilde*1000):03d}_diff{s_factor_abs_int}_reg9.csv"

    print("-" * 70)
    print(f"--- Searching Internal Solution [{alpha_start:.3f}, {alpha_end:.3f}] ---")
    print(f"--- (mu={mu_tilde}, Larger Vm Diff: s_factor=+/-{param_gen_kwargs_quasi_symmetric_larger_diff['s_factor_g']:.2f}, Reg={solver_settings['qp_regularization']:.1e}) ---")
    print(f"--- Output CSV: {output_csv_filename} ---")
    print("-" * 70)

    # --- 分析実行 ---
    results_df = analyze_alpha_full_results(
        alpha_range=alpha_range_analyze,
        param_gen_kwargs=param_gen_kwargs_quasi_symmetric_larger_diff, # <-- 差分拡大版を使用
        optimizer_kwargs=solver_settings,
        mu_tilde=mu_tilde
    )

    # --- 結果表示 & CSV保存 & KKTチェック ---
    if PANDAS_AVAILABLE and isinstance(results_df, pd.DataFrame) and not results_df.empty:
        print("\n--- Analysis Results Summary (DataFrame) ---")
        cols_display = ['alpha', 'success', 'H_star', 'fw_gap', 'iterations', 'active_set']
        cols_w = [f'w_fw_{m}' for m in range(M) if f'w_fw_{m}' in results_df.columns]
        cols_grad = [f'grad_H_{m}' for m in range(M) if f'grad_H_{m}' in results_df.columns]
        cols_to_show = cols_display + cols_w + cols_grad
        cols_to_show = [col for col in cols_to_show if col in results_df.columns]
        float_format_func = lambda x: f"{x:.4e}" if pd.notna(x) and isinstance(x, (float, np.number)) else x
        pd.options.display.float_format = float_format_func
        with pd.option_context('display.max_rows', 100, 'display.max_columns', None, 'display.width', 200):
             print(results_df[cols_to_show].to_string(index=False, na_rep='NaN'))
        pd.reset_option('display.float_format')

        # --- CSV保存 ---
        try:
            cols_pi = [f'pi_{k}' for k in range(K) if f'pi_{k}' in results_df.columns]
            cols_lam = [f'lambda_{m}' for m in range(M) if f'lambda_{m}' in results_df.columns]
            cols_csv_order = ['alpha', 'success', 'message', 'H_star', 'fw_gap', 'iterations', 'active_set'] + cols_w + cols_pi + cols_grad + cols_lam
            cols_csv_order = [col for col in cols_csv_order if col in results_df.columns]
            df_to_save = results_df[cols_csv_order]; df_to_save.to_csv(output_csv_filename, index=False, float_format='%.8e')
            print(f"\nFull detailed results saved to: {os.path.abspath(output_csv_filename)}")
        except Exception as e: print(f"\nError saving CSV: {e}")

        # --- KKTチェック ---
        print("\n--- Quick KKT Check (Stationarity w.r.t. w) ---")
        kkt_violations = []
        check_tol = solver_settings.get('tolerance', 1e-9) * 1e4
        for index, row in results_df.iterrows():
            alpha = row['alpha']
            if not row.get('success', False):
                 kkt_violations.append({'alpha': alpha, 'active_set_kkt': 'N/A', 'max_violation': np.nan, 'grad_consistency_violation': np.nan, 'check_result': 'FW Fail'})
                 continue
            w_star = np.array([row.get(f'w_fw_{m}', np.nan) for m in range(M)])
            grad_h = np.array([row.get(f'grad_H_{m}', np.nan) for m in range(M)])
            if np.isnan(w_star).any() or np.isnan(grad_h).any() or np.isclose(np.sum(w_star), 0):
                kkt_violations.append({'alpha': alpha, 'active_set_kkt': 'NaN', 'max_violation': np.nan, 'grad_consistency_violation': np.nan, 'check_result': 'Skipped (NaN)'})
                continue
            active_indices_kkt = set(m for m in range(M) if w_star[m] > unique_pt_tolerance)
            grad_consistency_violation = 0.0
            if len(active_indices_kkt) > 1:
                 active_grads = grad_h[list(active_indices_kkt)]; valid_grads = active_grads[np.isfinite(active_grads)]
                 if len(valid_grads) > 1: grad_consistency_violation = np.max(valid_grads) - np.min(valid_grads)
                 else: grad_consistency_violation = 0.0 if len(valid_grads)==1 else np.nan
            stationarity_violation = grad_consistency_violation if np.isfinite(grad_consistency_violation) else 0.0
            nu_estimate = np.nan
            if active_indices_kkt:
                 valid_active_grads = grad_h[list(active_indices_kkt)]; valid_active_grads = valid_active_grads[np.isfinite(valid_active_grads)]
                 if len(valid_active_grads) > 0: nu_estimate = np.max(valid_active_grads)
            if not np.isfinite(nu_estimate): valid_grads = grad_h[np.isfinite(grad_h)];
            if len(valid_grads)>0: nu_estimate = np.max(valid_grads)
            if np.isfinite(nu_estimate):
                for m in range(M):
                    if m not in active_indices_kkt and np.isfinite(grad_h[m]):
                        violation = grad_h[m] - nu_estimate
                        if violation > check_tol: stationarity_violation = max(stationarity_violation, violation)
            else: stationarity_violation = np.nan
            is_internal = len(active_indices_kkt) == M; result_str = ''
            if not np.isfinite(stationarity_violation): result_str = 'WARN (NaN KKT)'
            elif stationarity_violation > check_tol * 10: result_str = 'WARN (KKT Viol)'
            else: result_str = 'OK'
            if is_internal: result_str += ' [INTERNAL]'
            elif len(active_indices_kkt) == 2: result_str += ' [EDGE]'
            elif len(active_indices_kkt) == 1: result_str += ' [VERTEX]'
            elif len(active_indices_kkt) == 0: result_str += ' [EMPTY?]'
            kkt_violations.append({'alpha': alpha, 'active_set_kkt': str(tuple(sorted(active_indices_kkt))), 'max_violation': stationarity_violation, 'grad_consistency_violation': grad_consistency_violation, 'check_result': result_str})
        if PANDAS_AVAILABLE and kkt_violations:
            df_kkt = pd.DataFrame(kkt_violations)
            kkt_cols_order = ['alpha', 'active_set_kkt', 'max_violation', 'grad_consistency_violation', 'check_result']
            kkt_cols_order = [col for col in kkt_cols_order if col in df_kkt.columns]
            with pd.option_context('display.float_format', '{:.4e}'.format, 'display.max_rows', None): print(df_kkt[kkt_cols_order].to_string(index=False, na_rep='NaN'))
            print("\nNotes on KKT Check:")
            print(" - active_set_kkt: Active set inferred from w* > tolerance")
            print(" - max_violation: Max KKT violation")
            print(" - grad_consistency_violation: max(active_grads) - min(active_grads)")
            print(" - check_result: OK/WARN indicates KKT satisfaction; [...] solution type.")
        else: print("Could not perform KKT check or pandas not available.")

    elif isinstance(results_df, list) or (isinstance(results_df, pd.DataFrame) and results_df.empty):
        print("\n--- Analysis Results Summary ---")
        print("No results generated or DataFrame is empty.")
        if not PANDAS_AVAILABLE: print("(Pandas not available, skipping CSV export)")

    end_time_main = time.time()
    print(f"\nTotal execution time: {end_time_main - start_time_main:.2f} seconds")

SyntaxError: invalid syntax (<ipython-input-7-262d1fa57b1d>, line 241)